In [ ]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# !pip install evaluate wav2clip

# # core/utils.py
# import os
# import random
# import numpy as np
# import torch
# from PIL import Image

# def set_seed(seed: int = 42):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)

# def to_pil(x):
#     if isinstance(x, Image.Image): return x.convert("RGB")
#     if isinstance(x, str): return Image.open(x).convert("RGB")
#     if isinstance(x, np.ndarray): return Image.fromarray(x).convert("RGB")
#     raise ValueError("Expected path/np.ndarray/PIL.Image")

# def load_audio(path: str, target_sr: int) -> np.ndarray:
#     try:
#         import torchaudio
#     except Exception as e:
#         raise RuntimeError("Требуется torchaudio: pip install torchaudio") from e
#     waveform, sr = torchaudio.load(path)
#     if waveform.size(0) > 1: waveform = waveform.mean(dim=0, keepdim=True)
#     if sr != target_sr: waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)
#     return waveform.squeeze(0).numpy().astype(np.float32)

# def safe_load(component_cls, checkpoint: str, cache_dir: str = "./model_cache",
#               local_files_only: bool = None, **kwargs):
#     if local_files_only is None:
#         local_files_only = os.environ.get("HF_HUB_OFFLINE", "0") == "1"
#     name = getattr(component_cls, "__name__", "")
#     if "Tokenizer" in name:
#         kwargs.setdefault("use_fast", True)
#     return component_cls.from_pretrained(
#         checkpoint, cache_dir=cache_dir, local_files_only=local_files_only, **kwargs
#     )

# # core/types.py
# import torch

# def preferred_device():
#     if torch.cuda.is_available():
#         return torch.device("cuda")
#     if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
#         return torch.device("mps")
#     return torch.device("cpu")

# # core/device.py
# from typing import Any, Dict, List, Protocol, Literal
# import torch

# Modality = Literal["text", "image", "audio"]

# class Processor(Protocol):
#     modality: Modality
#     def prepare_batch(self, raw_items: List[Any]) -> Dict[str, torch.Tensor]:
#         ...

# class Encoder(Protocol):
#     modality: Modality
#     embed_dim: int
#     def encode_batch(self, inputs: Dict[str, torch.Tensor]) -> torch.Tensor:
#         ...

# class ItemAggregator(Protocol):
#     def aggregate(self, embs: torch.Tensor, counts: List[int], max_items: int, how: str) -> torch.Tensor:
#         ...

# class Fusion(Protocol):
#     def __call__(self, zs: Dict[Modality, torch.Tensor]) -> torch.Tensor:
#         ...

# # data/tokenization.py
# from functools import lru_cache
# from typing import Dict, List, Optional
# import numpy as np
# import torch

# class BatchTokenizer:
#     def __init__(self, tokenizer, max_length=512, cache_size=10000, batch_size=256, padding_strategy="max_length"):
#         assert padding_strategy in {"max_length", "dynamic"}
#         self.tok = tokenizer
#         self.max_length = max_length
#         self.batch_size = batch_size
#         self.padding_strategy = padding_strategy
#         self._cache = lru_cache(maxsize=cache_size)(self._tokenize_single)
#         self.is_fast = hasattr(tokenizer, "is_fast") and tokenizer.is_fast
#         if self.is_fast:
#             print("✓ Используется Fast Tokenizer")

#     def _tokenize_single(self, text: str) -> Dict[str, np.ndarray]:
#         res = self.tok(text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
#         return {k: v.squeeze(0).cpu().numpy() for k, v in res.items()}

#     def tokenize_batch(self, texts: List[str], use_cache: bool = True) -> Dict[str, torch.Tensor]:
#         if self.padding_strategy == "dynamic":
#             res = self.tok(texts, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt")
#             return {k: (v.long() if k in ("input_ids","attention_mask","token_type_ids") else v) for k, v in res.items()}
#         if use_cache and len(texts) < 100:
#             items = [self._cache(t) for t in texts]
#             keys = items[0].keys()
#             out = {}
#             for k in keys:
#                 dtype = torch.long if k in ("input_ids","attention_mask","token_type_ids") else torch.float32
#                 out[k] = torch.tensor(np.stack([it[k] for it in items]), dtype=dtype)
#             return out
#         res = self.tok(texts, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
#         return {k: (v.long() if k in ("input_ids","attention_mask","token_type_ids") else v) for k, v in res.items()}

#     def tokenize_dataset_lazy(self, texts: List[str], batch_size: Optional[int] = None):
#         b = batch_size or self.batch_size
#         for i in range(0, len(texts), b):
#             yield self.tokenize_batch(texts[i:i+b], use_cache=False)

#     def clear_cache(self):
#         self._cache.cache_clear()

# # data/dataset.py
# import gc
# from typing import Any, Dict, List, Optional, Callable
# import numpy as np
# import pandas as pd
# import torch
# from torch.utils.data import Dataset

# class MultiModalDataset(Dataset):
#     """
#     Универсальный датасет для классификации и регрессии.
#     Поддерживает:
#       - text pretokenize: BatchTokenizer или кастомные функции (single/batched)
#       - dynamic/max паддинг (dynamic отключает предтокенизацию)
#       - мульти-изображения и мульти-аудио на сэмпл
#     """
#     def __init__(
#         self,
#         df: pd.DataFrame,
#         target_col: Optional[str],                 # для регрессии — колонка с np.ndarray; для классификации — метка
#         task: str = "regression",                  # "regression" | "classification"
#         label2id: Optional[Dict[Any, int]] = None, # для классификации
#         text_columns: Optional[List[str]] = None,
#         image_columns: Optional[List[str]] = None,
#         audio_columns: Optional[List[str]] = None,
#         text_tokenizer=None,                       # BatchTokenizer
#         text_tokenizer_fn: Optional[Callable] = None,
#         text_tokenizer_fn_batched: Optional[Callable] = None,
#         special_tokens: Optional[Dict[str, Any]] = None,
#         pretokenize: bool = False,
#         pretokenize_batch_size: int = 256,
#         tokenizer_returns_tensors: bool = False,
#         deduplicate_texts: bool = True
#     ):
#         super().__init__()
#         self.df = df.reset_index(drop=True)
#         self.target_col = target_col
#         self.task = task.lower()
#         self.label2id = label2id or {}
#         self.text_columns = text_columns or []
#         self.image_columns = image_columns or []
#         self.audio_columns = audio_columns or []

#         self.batch_tokenizer = text_tokenizer
#         self.text_tokenizer_fn = text_tokenizer_fn
#         self.text_tokenizer_fn_batched = text_tokenizer_fn_batched

#         self.special_tokens = special_tokens or {"sep": " [SEP] "}
#         self.tokenizer_returns_tensors = tokenizer_returns_tensors

#         self._N = len(self.df)

#         # labels
#         if self.task == "classification":
#             if self.target_col is None:
#                 y = np.zeros(self._N, dtype=np.int64)
#             else:
#                 y = self.df[self.target_col].map(self.label2id).fillna(0).astype(int).values
#             self._labels = torch.tensor(y, dtype=torch.long)
#         else:
#             self._labels = self._prepare_labels_regression(self.df, self.target_col)

#         # lists for images/audios
#         self._image_lists = self._collect_multi_values(self.df, self.image_columns) if self.image_columns else None
#         self._audio_lists = self._collect_multi_values(self.df, self.audio_columns) if self.audio_columns else None

#         self._tok_bank: Optional[Dict[str, torch.Tensor]] = None
#         self._has_text = bool(self.text_columns)

#         # dynamic padding -> disable pretokenization
#         if pretokenize and self.batch_tokenizer is not None and getattr(self.batch_tokenizer, "padding_strategy", "max_length") == "dynamic":
#             print("⚠ Предтокенизация отключена: выбран dynamic-паддинг для текста.")
#             pretokenize = False

#         if self._has_text and pretokenize:
#             if self.batch_tokenizer is not None and self.text_tokenizer_fn is None and self.text_tokenizer_fn_batched is None:
#                 self._pretokenize_with_batch_tokenizer(pretokenize_batch_size)
#             else:
#                 self._pretokenize_with_custom_fn(pretokenize_batch_size, deduplicate_texts)

#     # labels for regression: [N, K]
#     def _prepare_labels_regression(self, df: pd.DataFrame, target_col: Optional[str]) -> torch.Tensor:
#         if target_col is None or target_col not in df.columns:
#             return torch.zeros((len(df), 1), dtype=torch.float32)
#         labels_list = []
#         for i in range(len(df)):
#             v = df.iloc[i][target_col]
#             if isinstance(v, (list, tuple, np.ndarray)):
#                 arr = np.asarray(v, dtype=np.float32)
#             else:
#                 try:
#                     arr = np.asarray([float(v)], dtype=np.float32)
#                 except Exception:
#                     arr = np.asarray([0.0], dtype=np.float32)
#             labels_list.append(arr)
#         K = max(a.shape[0] for a in labels_list) if labels_list else 1
#         out = np.zeros((len(labels_list), K), dtype=np.float32)
#         for i, a in enumerate(labels_list):
#             out[i, :a.shape[0]] = a
#         return torch.tensor(out, dtype=torch.float32)

#     def _join_text(self, row: pd.Series) -> str:
#         sep = self.special_tokens.get("sep", " [SEP] ")
#         return sep.join([("" if pd.isna(row[c]) else str(row[c])) for c in self.text_columns])

#     @staticmethod
#     def _as_list(v):
#         if v is None or (isinstance(v, float) and np.isnan(v)):
#             return []
#         if isinstance(v, (list, tuple)):
#             return list(v)
#         return [v]

#     def _collect_multi_values(self, df: pd.DataFrame, columns: List[str]) -> List[List[Any]]:
#         out = []
#         as_list = self._as_list
#         for _, row in df.iterrows():
#             lst: List[Any] = []
#             for c in columns:
#                 if c in row:
#                     lst.extend([x for x in as_list(row[c]) if x is not None])
#             out.append(lst)
#         return out

#     def _pretokenize_with_batch_tokenizer(self, batch_size: int):
#         print("Предтокенизация с BatchTokenizer...")
#         texts = [self._join_text(self.df.iloc[i]) for i in range(self._N)]
#         banks: Dict[str, List[torch.Tensor]] = {}
#         for start in range(0, self._N, batch_size):
#             batch = texts[start:start + batch_size]
#             tok = self.batch_tokenizer.tokenize_batch(batch, use_cache=False)
#             for k in tok:
#                 if k in ("input_ids","attention_mask","token_type_ids"):
#                     tok[k] = tok[k].long()
#                 else:
#                     tok[k] = tok[k].to(torch.float32)
#             for k, v in tok.items():
#                 banks.setdefault(k, []).append(v)
#         self._tok_bank = {k: torch.cat(v_parts, dim=0).contiguous() for k, v_parts in banks.items()}
#         shapes = {k: tuple(v.shape) for k, v in self._tok_bank.items()}
#         print(f"✓ Предтокенизация завершена: {self._N} образцов | keys={list(self._tok_bank.keys())}, shapes={shapes}")

#     def _pretokenize_with_custom_fn(self, batch_size: int, deduplicate_texts: bool):
#         cols = list(self.text_columns)
#         if self.text_tokenizer_fn_batched is not None:
#             print("Предтокенизация кастомной batched-функцией...")
#             first_end = min(self._N, max(8, batch_size))
#             batch_data = []
#             for i in range(first_end):
#                 row = self.df.iloc[i]
#                 d = {c: ("" if pd.isna(row[c]) else str(row[c])) for c in cols}
#                 batch_data.append(d)
#             first_tok = self.text_tokenizer_fn_batched(batch_data, self.special_tokens)
#             if not isinstance(first_tok, dict):
#                 raise ValueError("text_tokenizer_fn_batched должна возвращать dict тензоров [B, L]")
#             bank: Dict[str, torch.Tensor] = {}
#             for k, t in first_tok.items():
#                 if not torch.is_tensor(t): t = torch.tensor(t)
#                 bank[k] = torch.empty((self._N, t.size(1)), dtype=t.dtype)
#                 bank[k][:first_end] = t[:first_end]
#             for start in range(first_end, self._N, batch_size):
#                 end = min(self._N, start + batch_size)
#                 batch_data = []
#                 for i in range(start, end):
#                     row = self.df.iloc[i]
#                     d = {c: ("" if pd.isna(row[c]) else str(row[c])) for c in cols}
#                     batch_data.append(d)
#                 tok = self.text_tokenizer_fn_batched(batch_data, self.special_tokens)
#                 for k, t in tok.items():
#                     if not torch.is_tensor(t): t = torch.tensor(t)
#                     bank[k][start:end] = t
#             self._tok_bank = {k: v.contiguous() for k, v in bank.items()}
#             shapes = {k: tuple(v.shape) for k, v in self._tok_bank.items()}
#             print(f"✓ Предтокенизация кастомной batched-функцией завершена: shapes={shapes}")
#             return

#         print("Предтокенизация кастомной single-функцией...")
#         col_arrays = [self.df[c].astype(str).where(~self.df[c].isna(), other="").tolist() for c in cols]
#         first_td = {c: col_arrays[i][0] for i, c in enumerate(cols)}
#         first_tok = self.text_tokenizer_fn(first_td, self.special_tokens)
#         if not isinstance(first_tok, dict):
#             raise ValueError("custom text_tokenizer_fn должна возвращать dict тензоров")
#         for k, t in first_tok.items():
#             if not torch.is_tensor(t): first_tok[k] = torch.tensor(t)
#         bank: Dict[str, torch.Tensor] = {k: torch.empty((self._N, *t.shape), dtype=t.dtype) for k, t in first_tok.items()}
#         for k, t in first_tok.items():
#             bank[k][0].copy_(t)

#         cache: Dict[tuple, Dict[str, torch.Tensor]] = {}
#         if deduplicate_texts:
#             key0 = tuple(first_td.get(c, "") for c in cols)
#             cache[key0] = {k: v.clone() for k, v in first_tok.items()}

#         for i in range(1, self._N):
#             td = {c: col_arrays[j][i] for j, c in enumerate(cols)}
#             if deduplicate_texts:
#                 key = tuple(td.get(c, "") for c in cols)
#                 tok = cache.get(key)
#                 if tok is None:
#                     tok = self.text_tokenizer_fn(td, self.special_tokens)
#                     for k, t in tok.items():
#                         if not torch.is_tensor(t): tok[k] = torch.tensor(t)
#                     cache[key] = {k: v.clone() for k, v in tok.items()}
#             else:
#                 tok = self.text_tokenizer_fn(td, self.special_tokens)
#                 for k, t in tok.items():
#                     if not torch.is_tensor(t): tok[k] = torch.tensor(t)
#             for k, t in tok.items():
#                 bank[k][i].copy_(t)

#         self._tok_bank = {k: v.contiguous() for k, v in bank.items()}
#         shapes = {k: tuple(v.shape) for k, v in self._tok_bank.items()}
#         print(f"✓ Предтокенизация кастомной single-функцией завершена: shapes={shapes}")

#     def __len__(self):
#         return self._N

#     def __getitem__(self, idx: int) -> Dict[str, Any]:
#         item: Dict[str, Any] = {}
#         item["labels"] = self._labels[idx]  # [K] float32 для регрессии, int64 для классификации
#         if self.text_columns:
#             if self._tok_bank is not None:
#                 item["text_tokens"] = {k: v[idx] for k, v in self._tok_bank.items()}
#             else:
#                 item["text"] = self._join_text(self.df.iloc[idx])
#         if self._image_lists is not None:
#             item["images"] = self._image_lists[idx]
#         if self._audio_lists is not None:
#             item["audios"] = self._audio_lists[idx]
#         return item

#     def get_cache_stats(self):
#         has = self._tok_bank is not None
#         sizes = {k: tuple(v.shape) for k, v in (self._tok_bank or {}).items()}
#         return {"has_pretokenized": has, "shapes": sizes, "N": self._N}

#     def clear_cache(self):
#         self._tok_bank = None
#         gc.collect()
#         if torch.cuda.is_available():
#             torch.cuda.empty_cache()

# # data/collate.py
# from typing import Any, Dict, List, Optional
# import numpy as np
# import torch
# # from mmkit.core.utils import to_pil, load_audio

# class MultiModalCollator:
#     """
#     Готовит backend_inputs для модели.
#     - Если в item есть "text_tokens": стакает их.
#     - Иначе использует processors["text"] для батчевой токенизации.
#     - Изображения/аудио: разворачивает в плоский список + считает counts.
#     """
#     def __init__(self, processors: Dict[str, Any], task: str = "regression", audio_sr_fallback: int = 16000):
#         self.processors = processors
#         self.task = task.lower()
#         self.audio_sr_fallback = audio_sr_fallback

#     def __call__(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
#         labels = [b.get("labels") for b in batch]
#         if labels and isinstance(labels[0], torch.Tensor):
#             labels = torch.stack(labels)
#         else:
#             if self.task == "classification":
#                 labels = torch.tensor(labels, dtype=torch.long)
#             else:
#                 labels = torch.tensor(labels, dtype=torch.float32)

#         out = {"labels": labels, "backend_inputs": {}}
#         # text
#         if "text" in self.processors or any(("text_tokens" in b) for b in batch):
#             if "text_tokens" in batch[0]:
#                 t0 = batch[0]["text_tokens"]
#                 text_inputs = {}
#                 for key in t0.keys():
#                     if torch.is_tensor(t0[key]):
#                         text_inputs[key] = torch.stack([b["text_tokens"][key] for b in batch])
#                     else:
#                         dtype = torch.long if key in ("input_ids","attention_mask","token_type_ids") else torch.float32
#                         text_inputs[key] = torch.tensor([b["text_tokens"][key] for b in batch], dtype=dtype)
#             else:
#                 texts = [b.get("text", "") or " " for b in batch]
#                 text_inputs = self.processors["text"].prepare_batch(texts)
#             out["backend_inputs"]["text_inputs"] = text_inputs

#         # images
#         if "image" in self.processors:
#             flat_images, counts = [], []
#             for b in batch:
#                 lst = b.get("images", []) or []
#                 lst = [to_pil(x) for x in lst if x is not None]
#                 counts.append(len(lst))
#                 flat_images.extend(lst)
#             if len(flat_images) > 0:
#                 out["backend_inputs"]["image_inputs"] = self.processors["image"].prepare_batch(flat_images)
#             else:
#                 out["backend_inputs"]["image_inputs"] = {"pixel_values": None}
#             out["backend_inputs"]["image_counts"] = torch.tensor(counts, dtype=torch.long)

#         # audio
#         if "audio" in self.processors:
#             flat, counts = [], []
#             for b in batch:
#                 lst = b.get("audios", []) or []
#                 counts.append(len(lst))
#                 for a in lst:
#                     if isinstance(a, str):
#                         sr = getattr(self.processors["audio"], "sr", self.audio_sr_fallback)
#                         flat.append(load_audio(a, sr))
#                     else:
#                         arr = np.asarray(a, dtype=np.float32)
#                         if arr.ndim>1: arr = np.squeeze(arr)
#                         if arr.ndim>1: arr = arr.reshape(-1)
#                         flat.append(arr)
#             out["backend_inputs"]["audio_counts"] = torch.tensor(counts, dtype=torch.long)
#             out["backend_inputs"]["audio_inputs"] = self.processors["audio"].prepare_batch(flat) if len(flat)>0 else {"input_values": None, "input_features": None, "raw_audios": []}

#         out["backend_inputs"]["batch_size"] = len(batch)
#         return out

# # processors/text.py
# # from mmkit.data.tokenization import BatchTokenizer

# class TextProcessor:
#     modality = "text"
#     def __init__(self, tokenizer, max_length=512, padding="max_length", cache_size=10000, batch_size=256):
#         self.bt = BatchTokenizer(tokenizer, max_length=max_length, cache_size=cache_size, batch_size=batch_size, padding_strategy=padding)
#     def prepare_batch(self, texts):
#         return self.bt.tokenize_batch(texts, use_cache=True)
#     def clear_cache(self):
#         self.bt.clear_cache()

# # processors/image.py
# class ImageProcessor:
#     modality = "image"
#     def __init__(self, hf_processor):
#         self.proc = hf_processor
#     def prepare_batch(self, images):
#         if len(images)==0: return {"pixel_values": None}
#         x = self.proc(images=images, return_tensors="pt")
#         return {"pixel_values": x["pixel_values"]}

# # processors/audio.py
# class ClapAudioProcessor:
#     modality="audio"
#     def __init__(self, hf_processor, sr=48000):
#         self.proc = hf_processor
#         self.sr = sr
#     def prepare_batch(self, raw_list):
#         x = self.proc(audios=raw_list, sampling_rate=self.sr, padding=True, return_tensors="pt")
#         return {"input_features": x["input_features"]}

# class Wav2VecAudioProcessor:
#     modality="audio"
#     def __init__(self, hf_processor, sr=16000):
#         self.proc = hf_processor
#         self.sr = sr
#     def prepare_batch(self, raw_list):
#         x = self.proc(raw_list, sampling_rate=self.sr, padding=True, return_tensors="pt")
#         return {"input_values": x["input_values"]}

# class Wav2ClipAudioProcessor:
#     modality="audio"
#     def __init__(self, sr=16000):
#         self.sr = sr
#     def prepare_batch(self, raw_list):
#         return {"raw_audios": raw_list}  # список np.ndarray; энкодер сам обработает

# # encoders/base.py
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class BaseEncoder(nn.Module):
#     modality: str = "text"
#     embed_dim: int = 0

#     def _normalize(self, x: torch.Tensor) -> torch.Tensor:
#         if x.dim() > 2:
#             x = x.view(x.size(0), -1)
#         return F.normalize(x, dim=-1, eps=1e-12)

# # encoders/text_auto.py
# import torch
# from transformers import AutoModel
# # from mmkit.core.utils import safe_load
# # from .base import BaseEncoder

# class AutoTextEncoder(BaseEncoder):
#     modality="text"
#     def __init__(self, checkpoint="bert-base-multilingual-cased", cache_dir="./model_cache"):
#         super().__init__()
#         self.model = safe_load(AutoModel, checkpoint, cache_dir)
#         self.embed_dim = self.model.config.hidden_size

#     @torch.no_grad()
#     def encode_batch(self, inputs):
#         out = self.model(**inputs)
#         z = out.pooler_output if getattr(out, "pooler_output", None) is not None else out.last_hidden_state.mean(dim=1)
#         return self._normalize(z)

# # encoders/text_clip.py
# import torch
# from transformers import CLIPModel
# # from mmkit.core.utils import safe_load
# # from .base import BaseEncoder

# class CLIPTextEncoder(BaseEncoder):
#     modality="text"
#     def __init__(self, checkpoint="openai/clip-vit-base-patch32", cache_dir="./model_cache"):
#         super().__init__()
#         self.model = safe_load(CLIPModel, checkpoint, cache_dir)
#         self.embed_dim = self.model.config.projection_dim

#     @torch.no_grad()
#     def encode_batch(self, inputs):
#         z = self.model.get_text_features(**inputs)
#         return self._normalize(z)

# # encoders/image_auto.py
# import torch
# from transformers import AutoModel
# # from mmkit.core.utils import safe_load
# # from .base import BaseEncoder

# class AutoImageEncoder(BaseEncoder):
#     modality="image"
#     def __init__(self, checkpoint="google/vit-base-patch16-224", cache_dir="./model_cache"):
#         super().__init__()
#         self.model = safe_load(AutoModel, checkpoint, cache_dir)
#         self.embed_dim = self.model.config.hidden_size

#     @torch.no_grad()
#     def encode_batch(self, inputs):
#         pv = inputs.get("pixel_values", None)
#         if pv is None: 
#             return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
#         out = self.model(pixel_values=pv)
#         z = out.pooler_output if getattr(out, "pooler_output", None) is not None else out.last_hidden_state[:, 0]
#         return self._normalize(z)

# # encoders/image_clip.py
# import torch
# from transformers import CLIPModel
# # from mmkit.core.utils import safe_load
# # from .base import BaseEncoder

# class CLIPImageEncoder(BaseEncoder):
#     modality="image"
#     def __init__(self, checkpoint="openai/clip-vit-base-patch32", cache_dir="./model_cache"):
#         super().__init__()
#         self.model = safe_load(CLIPModel, checkpoint, cache_dir)
#         self.embed_dim = self.model.config.projection_dim

#     @torch.no_grad()
#     def encode_batch(self, inputs):
#         pv = inputs.get("pixel_values", None)
#         if pv is None:
#             return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
#         z = self.model.get_image_features(pixel_values=pv)
#         return self._normalize(z)

# # encoders/audio_clap.py
# import torch
# from transformers import ClapModel
# # from mmkit.core.utils import safe_load
# # from .base import BaseEncoder

# class CLAPAudioEncoder(BaseEncoder):
#     modality="audio"
#     def __init__(self, checkpoint="laion/clap-htsat-unfused", cache_dir="./model_cache"):
#         super().__init__()
#         self.model = safe_load(ClapModel, checkpoint, cache_dir)
#         self.embed_dim = getattr(self.model.config, "projection_dim", 512)

#     @torch.no_grad()
#     def encode_batch(self, inputs):
#         feats = inputs.get("input_features")
#         if feats is None:
#             return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
#         z = self.model.get_audio_features(input_features=feats.float())
#         return self._normalize(z.float())

# # encoders/audio_wav2vec.py
# import torch
# from transformers import AutoModel
# # from mmkit.core.utils import safe_load
# # from .base import BaseEncoder

# class Wav2Vec2Encoder(BaseEncoder):
#     modality="audio"
#     def __init__(self, checkpoint="facebook/wav2vec2-base-960h", cache_dir="./model_cache"):
#         super().__init__()
#         self.model = safe_load(AutoModel, checkpoint, cache_dir)
#         self.embed_dim = getattr(self.model.config, "hidden_size", 768)

#     @torch.no_grad()
#     def encode_batch(self, inputs):
#         iv = inputs.get("input_values")
#         if iv is None:
#             return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
#         out = self.model(input_values=iv.float())
#         z = out.pooler_output if getattr(out, "pooler_output", None) is not None else out.last_hidden_state.mean(dim=1)
#         return self._normalize(z.float())

# # encoders/audio_wav2clip.py
# import numpy as np
# import torch
# # from .base import BaseEncoder

# class Wav2ClipEncoder(BaseEncoder):
#     modality="audio"
#     def __init__(self):
#         super().__init__()
#         try:
#             import wav2clip as w2c
#         except Exception as e:
#             raise RuntimeError("Требуется wav2clip: pip install wav2clip") from e
#         self.w2c = w2c
#         model = None
#         if hasattr(w2c, "get_model"):
#             model = w2c.get_model()
#         elif hasattr(w2c, "model"):
#             m = w2c.model
#             model = m() if callable(m) else m
#         else:
#             raise RuntimeError("wav2clip не содержит get_model()/model")
#         self.model = model
#         self.embed_dim = 512

#     @torch.no_grad()
#     def encode_batch(self, inputs):
#         raws = inputs.get("raw_audios", [])
#         if len(raws)==0:
#             return torch.zeros(0, self.embed_dim)
#         embs = []
#         for arr in raws:
#             a = np.asarray(arr, dtype=np.float32)
#             if a.ndim>1: a = np.squeeze(a)
#             if a.ndim>1: a = a.reshape(-1)
#             if a.size < 512:
#                 a = np.pad(a, (0, 512 - a.size), mode="constant")
#             try:
#                 e = self.w2c.embed_audio(a, self.model)
#                 e = np.asarray(e)
#             except Exception:
#                 x = torch.from_numpy(a).float().unsqueeze(0)
#                 y = self.model(x)
#                 if isinstance(y, (tuple, list)):
#                     y = y[0]
#                 if torch.is_tensor(y):
#                     if y.dim()==2 and y.size(0)==1: y=y.squeeze(0)
#                     e = y.detach().cpu().numpy()
#                 else:
#                     e = np.asarray(y)
#             if e.ndim>1: e = e.reshape(-1)
#             embs.append(torch.as_tensor(e, dtype=torch.float32))
#         z = torch.stack(embs, dim=0)
#         z = torch.nn.functional.normalize(z, dim=-1, eps=1e-12)
#         return z

# # aggregation/item_pool.py
# import torch
# import torch.nn.functional as F

# class ItemAggregator:
#     def aggregate(self, embs: torch.Tensor, counts, max_k: int, how: str):
#         if embs is None or (torch.is_tensor(embs) and embs.numel()==0):
#             D = 0 if embs is None else getattr(embs, "size", lambda *_: 0)(-1)
#             out_dim = D*max_k if how=="concat" else D
#             return torch.zeros((len(counts), out_dim), device=embs.device if torch.is_tensor(embs) else "cpu", dtype=torch.float32)
#         if embs.dim()==1: embs = embs.unsqueeze(0)
#         if embs.dim()>2: embs = embs.view(embs.size(0), -1)
#         N, D = embs.size()
#         out_dim = D*max_k if how=="concat" else D
#         out = torch.zeros((len(counts), out_dim), device=embs.device, dtype=embs.dtype)
#         off = 0
#         for i, c in enumerate(counts):
#             if c<=0 or off>=N: continue
#             take = min(c, N-off)
#             sample = embs[off:off+take]
#             off += take
#             if how=="concat":
#                 sample = sample[:max_k]
#                 if sample.size(0)<max_k:
#                     pad = torch.zeros((max_k - sample.size(0), D), device=embs.device, dtype=embs.dtype)
#                     sample = torch.cat([sample, pad], dim=0)
#                 out[i] = sample.reshape(-1)
#             else:
#                 out[i] = sample.mean(dim=0)
#         return F.normalize(out, dim=-1, eps=1e-12)

# # aggregation/fusion.py
# import torch

# class ConcatFusion:
#     def __call__(self, zs: dict) -> torch.Tensor:
#         feats = [zs[k] for k in ["image","text","audio"] if k in zs]
#         return torch.cat(feats, dim=-1)

# class MeanFusion:
#     def __call__(self, zs: dict) -> torch.Tensor:
#         feats = [zs[k] for k in ["image","text","audio"] if k in zs]
#         return torch.stack(feats, dim=0).mean(dim=0)

# # heads/regression.py
# import torch.nn as nn

# class RegressionHead(nn.Module):
#     def __init__(self, in_dim, out_dim, hidden=256, dropout=0.1):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.LayerNorm(in_dim),
#             nn.Linear(in_dim, hidden),
#             nn.GELU(),
#             nn.Dropout(dropout),
#             nn.Linear(hidden, out_dim)
#         )
#     def forward(self, x):
#         return self.net(x)

# # heads/classification.py
# import torch.nn as nn

# class ClassificationHead(nn.Module):
#     def __init__(self, in_dim, num_labels, hidden=512, dropout=0.1):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.LayerNorm(in_dim),
#             nn.Linear(in_dim, hidden),
#             nn.GELU(),
#             nn.Dropout(dropout),
#             nn.Linear(hidden, num_labels)
#         )
#     def forward(self, x): return self.net(x)

# # model/multimodal.py
# from typing import Dict
# import torch
# import torch.nn as nn
# # from mmkit.aggregation.item_pool import ItemAggregator

# class MultiModalModel(nn.Module):
#     """
#     Модель: dict(encoders) + aggregator (для image/audio множественных) + fusion + head.
#     """
#     def __init__(self, encoders: Dict[str, nn.Module], aggregator: ItemAggregator, fusion, head,
#                  image_cfg=None, audio_cfg=None):
#         super().__init__()
#         self.enc = nn.ModuleDict(encoders)
#         self.agg = aggregator
#         self.fusion = fusion
#         self.head = head
#         self.image_cfg = image_cfg or {"max_items":1, "how":"concat"}
#         self.audio_cfg = audio_cfg or {"max_items":1, "how":"concat"}

#     def _fwd_features(self, backend_inputs: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
#         zs = {}
#         if "text" in self.enc and "text_inputs" in backend_inputs:
#             zs["text"] = self.enc["text"].encode_batch(backend_inputs["text_inputs"])
#         if "image" in self.enc and "image_inputs" in backend_inputs:
#             flat = self.enc["image"].encode_batch(backend_inputs["image_inputs"])
#             counts = backend_inputs.get("image_counts", torch.tensor([0]*backend_inputs.get("batch_size", flat.size(0)))).tolist()
#             zs["image"] = self.agg.aggregate(flat, counts, self.image_cfg["max_items"], self.image_cfg["how"])
#         if "audio" in self.enc and "audio_inputs" in backend_inputs:
#             flat = self.enc["audio"].encode_batch(backend_inputs["audio_inputs"])
#             counts = backend_inputs.get("audio_counts", torch.tensor([0]*backend_inputs.get("batch_size", flat.size(0)))).tolist()
#             zs["audio"] = self.agg.aggregate(flat, counts, self.audio_cfg["max_items"], self.audio_cfg["how"])
#         return zs

#     def forward(self, backend_inputs: Dict[str, torch.Tensor], labels: torch.Tensor = None):
#         zs = self._fwd_features(backend_inputs)
#         if not zs:
#             raise ValueError("Энкодеры не вернули эмбеддинги")
#         fused = self.fusion(zs)
#         logits = self.head(fused)
#         return {"logits": logits}

#     @torch.no_grad()
#     def get_embeddings(self, backend_inputs: Dict[str, torch.Tensor], return_per_modality: bool = False):
#         zs = self._fwd_features(backend_inputs)
#         fused = self.fusion(zs)
#         return (fused, zs) if return_per_modality else fused

# # train/trainer_hf.py
# from typing import Optional
# import torch
# import torch.nn.functional as F
# from transformers import Trainer

# class MSETrainer(Trainer):
#     def compute_loss(
#         self,
#         model,
#         inputs,
#         return_outputs: bool = False,
#         num_items_in_batch: Optional[int] = None,
#         **kwargs
#     ):
#         labels = inputs.pop("labels").to(torch.float32)
#         backend_inputs = inputs.pop("backend_inputs", None)
#         if backend_inputs is None:
#             # fallback: все остальное считаем backend_inputs
#             backend_inputs = inputs

#         out = model(backend_inputs=backend_inputs)  # передаем строго backend_inputs
#         logits = out["logits"]
#         preds = logits if logits.dim() == 1 else (logits.squeeze(-1) if logits.size(-1) == 1 else logits)
#         labels = labels.view_as(preds)
#         loss = F.mse_loss(preds, labels)
#         return (loss, out) if return_outputs else loss

# class WeightedCETrainer(Trainer):
#     def __init__(self, *args, class_weights=None, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.class_weights = (
#             torch.as_tensor(class_weights, dtype=torch.float32) if class_weights is not None else None
#         )

#     def compute_loss(
#         self,
#         model,
#         inputs,
#         return_outputs: bool = False,
#         num_items_in_batch: Optional[int] = None,
#         **kwargs
#     ):
#         labels = inputs.pop("labels")
#         backend_inputs = inputs.pop("backend_inputs", None)
#         if backend_inputs is None:
#             backend_inputs = inputs

#         out = model(backend_inputs=backend_inputs)  # только backend_inputs
#         logits = out["logits"]

#         weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
#         loss = F.cross_entropy(logits, labels.long(), weight=weight)
#         return (loss, out) if return_outputs else loss

# # train/callbacks.py
# from transformers.trainer_callback import TrainerCallback
# from tqdm.auto import tqdm

# class PbarConsoleLogger(TrainerCallback):
#     def __init__(self, pbar):
#         self.pbar = pbar
#         self.last_logs = {}
#         self.last_train_loss = None
#         self.printed_eval_steps = set()
#         self.tqdm = tqdm

#     def _step(self, state) -> int:
#         return int(state.global_step or 0)

#     def _fmt_postfix(self):
#         parts = []
#         if 'loss' in self.last_logs:
#             parts.append(f"loss {self.last_logs['loss']:.4f}")
#         if 'eval_loss' in self.last_logs:
#             parts.append(f"val {self.last_logs['eval_loss']:.4f}")
#         for k, v in self.last_logs.items():
#             if k.startswith('eval_') and k not in (
#                 'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'
#             ):
#                 parts.append(f"{k.replace('eval_', '')} {v:.4f}")
#         return " | ".join(parts)

#     def on_step_end(self, args, state, control, **kwargs):
#         n = min(self._step(state), self.pbar.total)
#         if n > self.pbar.n:
#             self.pbar.update(n - self.pbar.n)
#         if self.last_logs:
#             self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
#         self.pbar.refresh()

#     def on_log(self, args, state, control, logs=None, **kwargs):
#         if not logs:
#             return
#         for k, v in logs.items():
#             if isinstance(v, (int, float)):
#                 self.last_logs[k] = float(v)
#         if 'loss' in logs and isinstance(logs['loss'], (int, float)):
#             self.last_train_loss = float(logs['loss'])

#         self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
#         self.pbar.refresh()

#         if any(k.startswith('eval_') for k in logs.keys()):
#             step = self._step(state)
#             if step in self.printed_eval_steps:
#                 return
#             self.printed_eval_steps.add(step)
#             train_loss_str = f"{self.last_train_loss:.10f}" if self.last_train_loss is not None else "n/a"
#             val_loss = logs.get('eval_loss', None)
#             val_loss_str = f"{float(val_loss):.10g}" if isinstance(val_loss, (int, float)) else "n/a"
#             exclude = {'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'}
#             extra_parts = []
#             for k, v in logs.items():
#                 if k.startswith('eval_') and k not in exclude:
#                     metric_name = k.replace('eval_', '')
#                     extra_parts.append(f"val {metric_name}: {float(v):.10f}")
#             line = f"step: {step}, train loss: {train_loss_str}, val loss: {val_loss_str}"
#             if extra_parts:
#                 line += ", " + ", ".join(extra_parts)
#             self.tqdm.write(line)

#     def on_train_end(self, args, state, control, **kwargs):
#         n = min(self._step(state), self.pbar.total)
#         if n > self.pbar.n:
#             self.pbar.update(n - self.pbar.n)
#         self.pbar.refresh()

# # train/metrics.py
# import numpy as np

# def build_regression_metrics(name: str):
#     name = name.lower()
#     if name not in ("rmse", "mae", "r2"):
#         raise ValueError('metric_name для регрессии должен быть "rmse", "mae" или "r2"')

#     def compute(p):
#         preds = p.predictions
#         y = p.label_ids
#         preds = preds.squeeze(-1) if preds.ndim == 2 and preds.shape[-1] == 1 else preds
#         y = y.squeeze(-1) if y.ndim == 2 and y.shape[-1] == 1 else y
#         axis = 1 if preds.ndim == 2 else None
#         if name == "rmse":
#             err = preds - y
#             mse = np.mean(err**2, axis=axis)
#             rmse = np.sqrt(mse)
#             return {"rmse": float(np.mean(rmse))}
#         elif name == "mae":
#             mae = np.mean(np.abs(preds - y), axis=axis)
#             return {"mae": float(np.mean(mae))}
#         else:
#             y_mean = np.mean(y, axis=axis, keepdims=True) if preds.ndim == 2 else np.mean(y)
#             ss_res = np.sum((y - preds) ** 2, axis=axis)
#             ss_tot = np.sum((y - y_mean) ** 2, axis=axis)
#             r2 = 1.0 - (ss_res / (ss_tot + 1e-12))
#             return {"r2": float(np.mean(r2))}
#     return compute

# # train/recipes.py
# from transformers import CLIPTokenizer, CLIPImageProcessor, AutoTokenizer, AutoImageProcessor, ClapProcessor, AutoProcessor
# # from mmkit.processors.text import TextProcessor
# # from mmkit.processors.image import ImageProcessor
# # from mmkit.processors.audio import ClapAudioProcessor, Wav2VecAudioProcessor, Wav2ClipAudioProcessor

# # from mmkit.encoders.text_clip import CLIPTextEncoder
# # from mmkit.encoders.text_auto import AutoTextEncoder
# # from mmkit.encoders.image_clip import CLIPImageEncoder
# # from mmkit.encoders.image_auto import AutoImageEncoder
# # from mmkit.encoders.audio_clap import CLAPAudioEncoder
# # from mmkit.encoders.audio_wav2vec import Wav2Vec2Encoder
# # from mmkit.encoders.audio_wav2clip import Wav2ClipEncoder

# # from mmkit.aggregation.item_pool import ItemAggregator
# # from mmkit.aggregation.fusion import ConcatFusion, MeanFusion
# # from mmkit.heads.regression import RegressionHead
# # from mmkit.model.multimodal import MultiModalModel

# def build_auto_regression(modalities,
#                           fusion="concat",
#                           # text config
#                           text_model_type="clip", text_checkpoint="openai/clip-vit-base-patch32", text_max_length=77, text_padding="max_length",
#                           # image config
#                           image_model_type="clip", image_checkpoint="openai/clip-vit-base-patch32", max_images=1, image_agg="concat",
#                           # audio config
#                           audio_model_type="clap", audio_checkpoint="laion/clap-htsat-unfused", audio_sr=48000, max_audios=1, audio_agg="concat",
#                           # head config
#                           out_dim=1, hidden=256, dropout=0.1,
#                           cache_dir="./model_cache"):
#     encoders = {}
#     processors = {}
#     # text
#     if "text" in modalities:
#         if text_model_type == "clip":
#             tok = CLIPTokenizer.from_pretrained(text_checkpoint, cache_dir=cache_dir)
#             processors["text"] = TextProcessor(tok, max_length=text_max_length, padding=text_padding)
#             encoders["text"] = CLIPTextEncoder(text_checkpoint, cache_dir=cache_dir)
#             text_dim = encoders["text"].embed_dim
#         else:
#             tok = AutoTokenizer.from_pretrained(text_checkpoint, cache_dir=cache_dir)
#             processors["text"] = TextProcessor(tok, max_length=text_max_length, padding=text_padding)
#             encoders["text"] = AutoTextEncoder(text_checkpoint, cache_dir=cache_dir)
#             text_dim = encoders["text"].embed_dim
#     # image
#     image_cfg = None
#     if "image" in modalities:
#         if image_model_type == "clip":
#             ip = CLIPImageProcessor.from_pretrained(image_checkpoint, cache_dir=cache_dir)
#             processors["image"] = ImageProcessor(ip)
#             encoders["image"] = CLIPImageEncoder(image_checkpoint, cache_dir=cache_dir)
#         else:
#             ip = AutoImageProcessor.from_pretrained(image_checkpoint, cache_dir=cache_dir)
#             processors["image"] = ImageProcessor(ip)
#             encoders["image"] = AutoImageEncoder(image_checkpoint, cache_dir=cache_dir)
#         image_dim = encoders["image"].embed_dim
#         image_cfg = {"max_items": max_images, "how": image_agg}
#     # audio
#     audio_cfg = None
#     if "audio" in modalities:
#         if audio_model_type == "clap":
#             ap = ClapProcessor.from_pretrained(audio_checkpoint, cache_dir=cache_dir)
#             processors["audio"] = ClapAudioProcessor(ap, sr=audio_sr)
#             encoders["audio"] = CLAPAudioEncoder(audio_checkpoint, cache_dir=cache_dir)
#         elif audio_model_type == "wav2vec":
#             ap = AutoProcessor.from_pretrained(audio_checkpoint, cache_dir=cache_dir)
#             processors["audio"] = Wav2VecAudioProcessor(ap, sr=audio_sr)
#             encoders["audio"] = Wav2Vec2Encoder(audio_checkpoint, cache_dir=cache_dir)
#         else:  # wav2clip
#             processors["audio"] = Wav2ClipAudioProcessor(sr=audio_sr)
#             encoders["audio"] = Wav2ClipEncoder()
#         audio_dim = encoders["audio"].embed_dim
#         audio_cfg = {"max_items": max_audios, "how": audio_agg}

#     agg = ItemAggregator()
#     fusion_mod = ConcatFusion() if fusion=="concat" else MeanFusion()
#     # compute in_dim
#     dims = []
#     for m in ["image","text","audio"]:
#         if m in encoders:
#             dims.append(encoders[m].embed_dim)
#     if fusion=="concat":
#         in_dim = sum(dims)
#     else:
#         assert len(set(dims))==1, "Для fusion=mean требуется одинаковая размерность эмбеддингов"
#         in_dim = dims[0]
#     head = RegressionHead(in_dim, out_dim=out_dim, hidden=hidden, dropout=dropout)
#     model = MultiModalModel(encoders, agg, fusion_mod, head, image_cfg=image_cfg, audio_cfg=audio_cfg)
#     return processors, model

# # pipelines/regression.py
# import math
# import gc
# from typing import Any, Dict, List, Optional
# import numpy as np
# import pandas as pd
# import torch
# from torch.utils.data import DataLoader
# from tqdm.auto import tqdm

# from transformers import TrainingArguments
# # from mmkit.core.utils import set_seed
# # from mmkit.data.dataset import MultiModalDataset
# # from mmkit.data.collate import MultiModalCollator
# # from mmkit.train.trainer_hf import MSETrainer
# # from mmkit.train.callbacks import PbarConsoleLogger
# # from mmkit.train.metrics import build_regression_metrics
# # from mmkit.train.recipes import build_auto_regression

# class MultiModalRegressionPipeline:
#     """
#     Высокоуровневый пайплайн мультимодальной регрессии (multi-target).
#     Поддерживает: text/image/audio, dynamic/max padding, предтокенизацию, чанки, RMSE/MAE/R2.
#     """
#     def __init__(self,
#                  modalities: List[str],
#                  target_column_names: List[str],
#                  text_columns: Optional[List[str]] = None,
#                  image_columns: Optional[List[str]] = None,
#                  audio_columns: Optional[List[str]] = None,
#                  # recipe params
#                  backend: str = "auto",
#                  clip_checkpoint: str = "openai/clip-vit-base-patch32",
#                  clap_checkpoint: str = "laion/clap-htsat-unfused",
#                  text_model_config: Optional[Dict[str, Any]] = None,
#                  image_model_config: Optional[Dict[str, Any]] = None,
#                  audio_model_config: Optional[Dict[str, Any]] = None,
#                  fusion: str = "concat",
#                  # tokenizer
#                  text_padding: str = "max_length",
#                  text_max_length: int = 77,
#                  use_batch_tokenizer: bool = True,
#                  pretokenize_data: bool = True,
#                  pretokenize_batch_size: int = 256,
#                  tokenizer_cache_size: int = 10000,
#                  max_pretokenize_samples: int = 100000,
#                  local_cache_dir: str = "./model_cache"):
#         self.modalities = sorted(list(set(modalities)))
#         self.target_column_names = list(target_column_names)
#         self.text_columns = text_columns or []
#         self.image_columns = image_columns or []
#         self.audio_columns = audio_columns or []
#         self.backend_name = backend.lower()
#         self.clip_checkpoint = clip_checkpoint
#         self.clap_checkpoint = clap_checkpoint
#         self.text_model_config = text_model_config
#         self.image_model_config = image_model_config
#         self.audio_model_config = audio_model_config
#         self.fusion = fusion

#         self.text_padding = text_padding
#         self.text_max_length = text_max_length
#         self.use_batch_tokenizer = use_batch_tokenizer
#         self.pretokenize_data = pretokenize_data
#         self.pretokenize_batch_size = pretokenize_batch_size
#         self.tokenizer_cache_size = tokenizer_cache_size
#         self.max_pretokenize_samples = max_pretokenize_samples
#         self.local_cache_dir = local_cache_dir

#         self._target_vec_col = "__target_vector__"
#         self.processors = {}
#         self.model = None
#         self.trainer = None

#     def _attach_target_vector(self, df: pd.DataFrame, fill_zeros: bool = False) -> pd.DataFrame:
#         df_c = df.copy()
#         K = len(self.target_column_names)
#         if fill_zeros:
#             df_c[self._target_vec_col] = [np.zeros(K, dtype=np.float32) for _ in range(len(df_c))]
#         else:
#             def _row_to_vec(row):
#                 vals = [row[c] for c in self.target_column_names]
#                 return np.asarray(vals, dtype=np.float32)
#             df_c[self._target_vec_col] = df_c.apply(_row_to_vec, axis=1)
#         return df_c

#     def _validate_modalities(self, df: pd.DataFrame):
#         if "text" in self.modalities:
#             assert self.text_columns, "Вы выбрали text, но text_columns пустой"
#             missing = [c for c in self.text_columns if c not in df.columns]
#             if missing: raise ValueError(f"Нет текстовых колонок: {missing}")
#         if "image" in self.modalities:
#             assert self.image_columns, "Вы выбрали image, но image_columns пуст"
#             missing = [c for c in self.image_columns if c not in df.columns]
#             if missing: raise ValueError(f"Нет колонок изображений: {missing}")
#         if "audio" in self.modalities:
#             assert self.audio_columns, "Вы выбрали audio, но audio_columns пуст"
#             missing = [c for c in self.audio_columns if c not in df.columns]
#             if missing: raise ValueError(f"Нет колонок аудио: {missing}")

#     def _validate_targets(self, df: pd.DataFrame):
#         miss = [c for c in self.target_column_names if c not in df.columns]
#         if miss:
#             raise ValueError(f"В DataFrame отсутствуют целевые колонки: {miss}")

#     def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42):
#         df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
#         n_eval = int(math.ceil(len(df) * test_size))
#         return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

#     def _build_model(self, num_outputs: int, image_cfg: Dict[str, Any], audio_cfg: Dict[str, Any]):
#         # Авто-рецепт: CLIP для text+image, CLAP для audio (или явные конфиги)
#         if self.backend_name == "auto":
#             # auto defaults
#             txt_type = "clip" if "image" in self.modalities else ("auto" if "text" in self.modalities else None)
#             img_type = "clip" if "image" in self.modalities else None
#             aud_type = "clap" if "audio" in self.modalities else None

#             txt_cp = self.clip_checkpoint if txt_type=="clip" else (self.text_model_config["checkpoint"] if self.text_model_config else "bert-base-multilingual-cased")
#             img_cp = self.clip_checkpoint if img_type=="clip" else (self.image_model_config["checkpoint"] if self.image_model_config else "google/vit-base-patch16-224")
#             aud_cp = self.clap_checkpoint if aud_type=="clap" else (self.audio_model_config["checkpoint"] if self.audio_model_config else "facebook/wav2vec2-base-960h")

#             if aud_type is None and "audio" in self.modalities and self.audio_model_config:
#                 aud_type = self.audio_model_config.get("model_type","clap")

#             processors, model = build_auto_regression(
#                 modalities=self.modalities,
#                 fusion=self.fusion,
#                 text_model_type="clip" if txt_type=="clip" else "auto",
#                 text_checkpoint=txt_cp,
#                 text_max_length=self.text_max_length,
#                 text_padding=self.text_padding,
#                 image_model_type="clip" if img_type=="clip" else "auto",
#                 image_checkpoint=img_cp,
#                 max_images=image_cfg.get("max_images", 1),
#                 image_agg=image_cfg.get("image_agg", "concat"),
#                 audio_model_type=aud_type if aud_type else "clap",
#                 audio_checkpoint=aud_cp,
#                 audio_sr=audio_cfg.get("sr", 48000),
#                 max_audios=audio_cfg.get("max_audios", 1),
#                 audio_agg=audio_cfg.get("audio_agg", "concat"),
#                 out_dim=num_outputs,
#                 hidden=256,
#                 dropout=0.1,
#                 cache_dir=self.local_cache_dir
#             )
#             return processors, model

#         # Иначе — можно сделать фабрику из переданных конфигов (опущено для краткости)
#         raise NotImplementedError("Ручная сборка (не auto) не реализована в этой версии")

#     def fit(
#         self,
#         train_data: pd.DataFrame,
#         epochs: int = 3,
#         test_size: float = 0.2,
#         test_data: Optional[pd.DataFrame] = None,
#         per_device_train_batch_size: int = 16,
#         gradient_accumulation_steps: int = 1,
#         learning_rate: float = 2e-4,
#         metric_name: str = "rmse",       # "rmse" | "mae" | "r2"
#         fp16: bool = True,
#         logging_steps: int = 50,
#         eval_steps: int = 200,
#         output_dir: str = "./result_reg",
#         seed: int = 42,
#         gradient_checkpointing: bool = False
#     ):
#         import os
#         from transformers import TrainingArguments
    
#         # 1) Проверки и подготовка
#         self._validate_modalities(train_data)
#         self._validate_targets(train_data)
#         set_seed(seed)
    
#         df_train, df_eval = (train_data, test_data) if test_data is not None else self._split(train_data, test_size, seed)
#         if test_data is not None:
#             self._validate_targets(test_data)
    
#         # Преобразуем таргеты в одну вектор-колонку
#         df_train_ext = self._attach_target_vector(df_train, fill_zeros=False)
#         df_eval_ext  = self._attach_target_vector(df_eval,  fill_zeros=False)
    
#         # 2) Модель и процессоры (auto-рецепт)
#         image_cfg = {"max_images": 1, "image_agg": "concat"}
#         audio_cfg = {"sr": 48000, "max_audios": 1, "audio_agg": "concat"}
    
#         self.processors, self.model = self._build_model(
#             num_outputs=len(self.target_column_names),
#             image_cfg=image_cfg,
#             audio_cfg=audio_cfg
#         )
    
#         if gradient_checkpointing:
#             try:
#                 self.model.gradient_checkpointing_enable()
#             except Exception:
#                 pass
    
#         # 3) Datasets (без чанков)
#         has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
#         pretokenize_train = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
#                              and len(df_train_ext) <= self.max_pretokenize_samples)
#         pretokenize_eval  = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
#                              and len(df_eval_ext)  <= self.max_pretokenize_samples)
    
#         ds_train = MultiModalDataset(
#             df=df_train_ext,
#             target_col="__target_vector__",
#             task="regression",
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_train) else None,
#             pretokenize=pretokenize_train,
#             pretokenize_batch_size=self.pretokenize_batch_size
#         )
#         ds_eval = MultiModalDataset(
#             df=df_eval_ext,
#             target_col="__target_vector__",
#             task="regression",
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_eval) else None,
#             pretokenize=pretokenize_eval,
#             pretokenize_batch_size=self.pretokenize_batch_size
#         )
    
#         collate = MultiModalCollator(self.processors, task="regression")
#         compute_metrics = build_regression_metrics(metric_name)
    
#         # 4) Аргументы обучения (eval_strategy)
#         args = TrainingArguments(
#             output_dir=output_dir,
#             num_train_epochs=epochs,
#             per_device_train_batch_size=per_device_train_batch_size,
#             per_device_eval_batch_size=max(4, per_device_train_batch_size // 2),
#             gradient_accumulation_steps=gradient_accumulation_steps,
#             eval_accumulation_steps=max(1, gradient_accumulation_steps * 2),
#             learning_rate=learning_rate,
#             warmup_ratio=0.1,
#             lr_scheduler_type="cosine",
#             weight_decay=0.0,
#             eval_strategy="steps",
#             eval_steps=eval_steps,
#             save_strategy="steps",
#             save_steps=eval_steps,
#             load_best_model_at_end=True,
#             metric_for_best_model=f"eval_{metric_name}",
#             save_total_limit=1,
#             logging_strategy="steps",
#             logging_steps=logging_steps,
#             report_to="none",
#             fp16=fp16 and torch.cuda.is_available(),
#             dataloader_num_workers=min(4, os.cpu_count() or 4),
#             seed=seed,
#             remove_unused_columns=False,
#             gradient_checkpointing=gradient_checkpointing,
#             dataloader_pin_memory=True,
#             ddp_find_unused_parameters=False,
#             disable_tqdm=False
#         )
    
#         # 5) Trainer и старт обучения
#         self.trainer = MSETrainer(
#             model=self.model,
#             args=args,
#             train_dataset=ds_train,
#             eval_dataset=ds_eval,
#             data_collator=collate,
#             compute_metrics=compute_metrics
#         )
    
#         try:
#             from transformers.trainer_callback import PrinterCallback
#             self.trainer.remove_callback(PrinterCallback)
#         except Exception:
#             pass
    
#         self.trainer.train()
    
#         if has_bt:
#             try: self.processors["text"].clear_cache()
#             except Exception: pass
    
#         return self

#     def predict(self, df: pd.DataFrame, batch_size: Optional[int] = None) -> np.ndarray:
#         if self.trainer is None:
#             raise RuntimeError("Модель не обучена. Вызовите .fit().")

#         df_c = self._attach_target_vector(df, fill_zeros=True)
#         has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")

#         ds = MultiModalDataset(
#             df=df_c,
#             target_col="__target_vector__",
#             task="regression",
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if has_bt else None,
#             pretokenize=(self.pretokenize_data and has_bt and len(df_c) < 10000 and self.text_padding != "dynamic"),
#             pretokenize_batch_size=self.pretokenize_batch_size
#         )
#         collate = MultiModalCollator(self.processors, task="regression")

#         if batch_size:
#             original_bs = self.trainer.args.per_device_eval_batch_size
#             self.trainer.args.per_device_eval_batch_size = batch_size

#         effective_batch = batch_size or self.trainer.args.per_device_eval_batch_size
#         num_batches = (len(df_c) + effective_batch - 1) // effective_batch
#         print(f"Running predictions (batch_size={effective_batch}, num_batches={num_batches})...")

#         original_disable_tqdm = self.trainer.args.disable_tqdm
#         self.trainer.args.disable_tqdm = False

#         preds = self.trainer.predict(test_dataset=ds, metric_key_prefix="predict")

#         self.trainer.args.disable_tqdm = original_disable_tqdm
#         if batch_size:
#             self.trainer.args.per_device_eval_batch_size = original_bs

#         ds.clear_cache()
#         y = preds.predictions
#         y = y.squeeze(-1) if y.ndim == 2 and y.shape[-1] == 1 else y
#         return y

#     def get_embeddings(self, df: pd.DataFrame, batch_size: int = 32, return_per_modality: bool = False):
#         if self.trainer is None or self.model is None:
#             raise RuntimeError("Модель не обучена. Вызовите .fit().")
#         device = next(self.trainer.model.parameters()).device
#         self.model.to(device).eval()

#         df_c = self._attach_target_vector(df, fill_zeros=True)
#         has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")

#         ds = MultiModalDataset(
#             df=df_c,
#             target_col="__target_vector__",
#             task="regression",
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if has_bt else None,
#             pretokenize=False
#         )
#         collate = MultiModalCollator(self.processors, task="regression")
#         loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)

#         fused_list = []
#         per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

#         num_batches = (len(df_c) + batch_size - 1) // batch_size
#         print(f"Extracting embeddings (batch_size={batch_size}, num_batches={num_batches})...")

#         with torch.no_grad():
#             for batch in tqdm(loader, total=num_batches, desc="Extracting embeddings", unit="batch", leave=True):
#                 bi = batch["backend_inputs"]
#                 def move_to_device(obj):
#                     if torch.is_tensor(obj): return obj.to(device)
#                     if isinstance(obj, dict): return {k: move_to_device(v) for k, v in obj.items()}
#                     if isinstance(obj, list): return [move_to_device(v) for v in obj]
#                     return obj
#                 bi = move_to_device(bi)
#                 fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
#                 fused_list.append(fused.cpu().numpy())
#                 if return_per_modality:
#                     for m in per_mod_lists.keys():
#                         if m in per:
#                             per_mod_lists[m].append(per[m].cpu().numpy())

#         fused_arr = np.vstack(fused_list)
#         if not return_per_modality:
#             print(f"✓ Embeddings shape: {fused_arr.shape}")
#             return fused_arr
#         per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
#         print(f"✓ Fused embeddings shape: {fused_arr.shape}")
#         for m, arr in per_mod.items():
#             print(f"✓ {m.capitalize()} embeddings shape: {arr.shape}")
#         return fused_arr, per_mod

# # pipelines/classification.py
# import os
# import math
# import gc
# from typing import Any, Dict, List, Optional, Tuple

# import numpy as np
# import pandas as pd
# import torch
# from torch.utils.data import DataLoader
# from tqdm.auto import tqdm
# from transformers import TrainingArguments

# # from mmkit.core.utils import set_seed
# # from mmkit.data.dataset import MultiModalDataset
# # from mmkit.data.collate import MultiModalCollator

# # from mmkit.train.trainer_hf import WeightedCETrainer
# # from mmkit.train.callbacks import PbarConsoleLogger

# # from mmkit.aggregation.item_pool import ItemAggregator
# # from mmkit.aggregation.fusion import ConcatFusion, MeanFusion
# # from mmkit.heads.classification import ClassificationHead
# # from mmkit.model.multimodal import MultiModalModel

# # Процессоры
# from transformers import (
#     CLIPTokenizer,
#     CLIPImageProcessor,
#     AutoTokenizer,
#     AutoImageProcessor,
#     ClapProcessor,
#     AutoProcessor,
# )

# # from mmkit.processors.text import TextProcessor
# # from mmkit.processors.image import ImageProcessor
# # from mmkit.processors.audio import (
# #     ClapAudioProcessor,
# #     Wav2VecAudioProcessor,
# #     Wav2ClipAudioProcessor,
# # )

# # # Энкодеры
# # from mmkit.encoders.text_clip import CLIPTextEncoder
# # from mmkit.encoders.text_auto import AutoTextEncoder
# # from mmkit.encoders.image_clip import CLIPImageEncoder
# # from mmkit.encoders.image_auto import AutoImageEncoder
# # from mmkit.encoders.audio_clap import CLAPAudioEncoder
# # from mmkit.encoders.audio_wav2vec import Wav2Vec2Encoder
# # from mmkit.encoders.audio_wav2clip import Wav2ClipEncoder


# class MultiModalClassificationPipeline:
#     """
#     Высокоуровневый пайплайн мультимодальной классификации (single/multi-class).
#     Поддерживает text/image/audio, dynamic/max padding, предтокенизацию, чанковую тренировку,
#     accuracy/f1 метрики, class weights, predict и извлечение эмбеддингов.

#     По умолчанию (backend="auto"):
#       - если есть image: CLIP для text и image;
#       - если есть audio: CLAP (или wav2vec/wav2clip по желанию — см. параметры),
#       - для чистого текста: Auto (BERT) по умолчанию.

#     Параметры:
#       - modalities: список модальностей среди ["text","image","audio"]
#       - target_column_name: имя колонки с метками (строки/инты)
#       - num_labels: число классов (если None — возьмётся из train_data по unique)
#       - text_columns, image_columns, audio_columns: колонки данных
#       - fusion: "concat" | "mean" (для mean размеры эмбеддингов должны совпадать)
#       - text_padding: "max_length" | "dynamic"
#       - text_max_length: длина токенов
#       - pretokenize_data: предтокенизация датасета (выключится при dynamic padding)
#       - tokenizer_cache_size: LRU-кэш токенизатора
#       - max_pretokenize_samples: лимит на предтокенизацию в чанке
#       - local_cache_dir: кэш HF

#       - audio_backend: "clap" | "wav2vec" | "wav2clip" (для audio)
#       - audio_sr: sampling rate для аудио-процессоров
#       - max_images_per_sample, image_agg: агрегация нескольких картинок ("concat"/"mean")
#       - max_audios_per_sample, audio_agg: агрегация для аудио
#     """
#     def __init__(
#         self,
#         modalities: List[str],
#         target_column_name: str,
#         num_labels: Optional[int] = None,
#         text_columns: Optional[List[str]] = None,
#         image_columns: Optional[List[str]] = None,
#         audio_columns: Optional[List[str]] = None,

#         # backend/модели
#         backend: str = "auto",
#         clip_checkpoint: str = "openai/clip-vit-base-patch32",
#         clap_checkpoint: str = "laion/clap-htsat-unfused",
#         text_model_config: Optional[Dict[str, Any]] = None,
#         image_model_config: Optional[Dict[str, Any]] = None,
#         audio_model_config: Optional[Dict[str, Any]] = None,

#         fusion: str = "concat",

#         # токенизация текста
#         text_padding: str = "max_length",
#         text_max_length: int = 77,
#         use_batch_tokenizer: bool = True,
#         pretokenize_data: bool = True,
#         pretokenize_batch_size: int = 256,
#         tokenizer_cache_size: int = 10000,
#         max_pretokenize_samples: int = 100000,
#         local_cache_dir: str = "./model_cache",

#         # изображение/аудио агрегация
#         max_images_per_sample: int = 1,
#         image_agg: str = "concat",
#         audio_backend: str = "clap",  # "clap" | "wav2vec" | "wav2clip"
#         audio_sr: int = 48000,
#         max_audios_per_sample: int = 1,
#         audio_agg: str = "concat",
#     ):
#         self.modalities = sorted(list(set(modalities)))
#         self.target_column_name = target_column_name
#         self.num_labels = num_labels

#         self.text_columns = text_columns or []
#         self.image_columns = image_columns or []
#         self.audio_columns = audio_columns or []

#         self.backend_name = backend.lower()
#         self.clip_checkpoint = clip_checkpoint
#         self.clap_checkpoint = clap_checkpoint
#         self.text_model_config = text_model_config
#         self.image_model_config = image_model_config
#         self.audio_model_config = audio_model_config

#         self.fusion = fusion
#         self.text_padding = text_padding
#         self.text_max_length = text_max_length
#         self.use_batch_tokenizer = use_batch_tokenizer
#         self.pretokenize_data = pretokenize_data
#         self.pretokenize_batch_size = pretokenize_batch_size
#         self.tokenizer_cache_size = tokenizer_cache_size
#         self.max_pretokenize_samples = max_pretokenize_samples
#         self.local_cache_dir = local_cache_dir

#         self.max_images_per_sample = int(max_images_per_sample)
#         self.image_agg = image_agg
#         self.audio_backend = audio_backend
#         self.audio_sr = int(audio_sr)
#         self.max_audios_per_sample = int(max_audios_per_sample)
#         self.audio_agg = audio_agg

#         # Внутренние поля
#         self.label2id: Optional[Dict[Any, int]] = None
#         self.id2label: Optional[Dict[int, str]] = None
#         self.processors: Dict[str, Any] = {}
#         self.model: Optional[MultiModalModel] = None
#         self.trainer: Optional[WeightedCETrainer] = None

#     # --------------------------
#     # Валидации и подсобки
#     # --------------------------
#     def _validate_modalities(self, df: pd.DataFrame):
#         if "text" in self.modalities:
#             assert self.text_columns, "Вы выбрали text, но text_columns пустой"
#             missing = [c for c in self.text_columns if c not in df.columns]
#             if missing:
#                 raise ValueError(f"Нет текстовых колонок: {missing}")
#         if "image" in self.modalities:
#             assert self.image_columns, "Вы выбрали image, но image_columns пуст"
#             missing = [c for c in self.image_columns if c not in df.columns]
#             if missing:
#                 raise ValueError(f"Нет колонок изображений: {missing}")
#         if "audio" in self.modalities:
#             assert self.audio_columns, "Вы выбрали audio, но audio_columns пуст"
#             missing = [c for c in self.audio_columns if c not in df.columns]
#             if missing:
#                 raise ValueError(f"Нет колонок аудио: {missing}")

#     def _ensure_label_mapping(self, df_train: pd.DataFrame):
#         classes = sorted(df_train[self.target_column_name].unique().tolist())
#         if self.num_labels is None:
#             self.num_labels = len(classes)
#         elif self.num_labels != len(classes):
#             print(f"Warning: num_labels={self.num_labels} != len(classes)={len(classes)}")
#         self.label2id = {c: i for i, c in enumerate(classes)}
#         self.id2label = {i: str(c) for c, i in self.label2id.items()}

#     def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
#         df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
#         n_eval = int(math.ceil(len(df) * test_size))
#         return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

#     def _ensure_label_column(self, df: pd.DataFrame) -> pd.DataFrame:
#         df_c = df.copy()
#         if self.target_column_name not in df_c.columns:
#             # Если нет метки — подставим первую известную
#             fake_label = next(iter(self.label2id.keys()))
#             df_c[self.target_column_name] = [fake_label] * len(df_c)
#         return df_c

#     # --------------------------
#     # Модель и процессоры (auto)
#     # --------------------------
#     def _build_model_and_processors(self) -> Tuple[Dict[str, Any], MultiModalModel]:
#         encoders = {}
#         processors = {}

#         # text
#         if "text" in self.modalities:
#             # Если есть image — используем CLIP-текст, иначе Auto (BERT)
#             use_clip_text = ("image" in self.modalities)
#             if use_clip_text:
#                 tok = CLIPTokenizer.from_pretrained(self.clip_checkpoint, cache_dir=self.local_cache_dir)
#                 processors["text"] = TextProcessor(
#                     tok, max_length=self.text_max_length, padding=self.text_padding,
#                     cache_size=self.tokenizer_cache_size, batch_size=self.pretokenize_batch_size
#                 )
#                 encoders["text"] = CLIPTextEncoder(self.clip_checkpoint, cache_dir=self.local_cache_dir)
#             else:
#                 tok = AutoTokenizer.from_pretrained(
#                     (self.text_model_config or {}).get("checkpoint", "bert-base-multilingual-cased"),
#                     cache_dir=self.local_cache_dir
#                 )
#                 processors["text"] = TextProcessor(
#                     tok, max_length=self.text_max_length, padding=self.text_padding,
#                     cache_size=self.tokenizer_cache_size, batch_size=self.pretokenize_batch_size
#                 )
#                 encoders["text"] = AutoTextEncoder(
#                     (self.text_model_config or {}).get("checkpoint", "bert-base-multilingual-cased"),
#                     cache_dir=self.local_cache_dir
#                 )

#         # image
#         image_cfg = None
#         if "image" in self.modalities:
#             if (self.image_model_config or {}).get("model_type", "clip") == "clip":
#                 ip = CLIPImageProcessor.from_pretrained(self.clip_checkpoint, cache_dir=self.local_cache_dir)
#                 processors["image"] = ImageProcessor(ip)
#                 encoders["image"] = CLIPImageEncoder(self.clip_checkpoint, cache_dir=self.local_cache_dir)
#             else:
#                 cp = (self.image_model_config or {}).get("checkpoint", "google/vit-base-patch16-224")
#                 ip = AutoImageProcessor.from_pretrained(cp, cache_dir=self.local_cache_dir)
#                 processors["image"] = ImageProcessor(ip)
#                 encoders["image"] = AutoImageEncoder(cp, cache_dir=self.local_cache_dir)
#             image_cfg = {"max_items": self.max_images_per_sample, "how": self.image_agg}

#         # audio
#         audio_cfg = None
#         if "audio" in self.modalities:
#             ab = (self.audio_model_config or {}).get("model_type", self.audio_backend)
#             if ab == "clap":
#                 cp = (self.audio_model_config or {}).get("checkpoint", self.clap_checkpoint)
#                 ap = ClapProcessor.from_pretrained(cp, cache_dir=self.local_cache_dir)
#                 processors["audio"] = ClapAudioProcessor(ap, sr=self.audio_sr)
#                 encoders["audio"] = CLAPAudioEncoder(cp, cache_dir=self.local_cache_dir)
#             elif ab == "wav2vec":
#                 cp = (self.audio_model_config or {}).get("checkpoint", "facebook/wav2vec2-base-960h")
#                 ap = AutoProcessor.from_pretrained(cp, cache_dir=self.local_cache_dir)
#                 processors["audio"] = Wav2VecAudioProcessor(ap, sr=self.audio_sr)
#                 encoders["audio"] = Wav2Vec2Encoder(cp, cache_dir=self.local_cache_dir)
#             else:  # wav2clip
#                 processors["audio"] = Wav2ClipAudioProcessor(sr=self.audio_sr)
#                 encoders["audio"] = Wav2ClipEncoder()
#             audio_cfg = {"max_items": self.max_audios_per_sample, "how": self.audio_agg}

#         # Fusion + Head
#         agg = ItemAggregator()
#         fusion_mod = ConcatFusion() if self.fusion == "concat" else MeanFusion()
#         dims = [encoders[m].embed_dim for m in ["image", "text", "audio"] if m in encoders]
#         if self.fusion == "concat":
#             in_dim = sum(dims)
#         else:
#             assert len(set(dims)) == 1, "Для fusion='mean' размеры эмбеддингов всех модальностей должны совпадать"
#             in_dim = dims[0]
#         head = ClassificationHead(in_dim=in_dim, num_labels=self.num_labels, hidden=512, dropout=0.1)
#         model = MultiModalModel(encoders, agg, fusion_mod, head, image_cfg=image_cfg, audio_cfg=audio_cfg)
#         return processors, model

#     # --------------------------
#     # Метрики
#     # --------------------------
#     def _build_compute_metrics(self, metric_name: str):
#         metric_name = metric_name.lower()
#         if metric_name == "f1":
#             try:
#                 import evaluate
#                 f1_metric = evaluate.load("f1")
#                 def compute(p):
#                     preds = p.predictions.argmax(-1)
#                     return f1_metric.compute(predictions=preds, references=p.label_ids, average="weighted")
#                 return compute
#             except Exception:
#                 print("evaluate не доступен, использую accuracy")
#                 metric_name = "accuracy"

#         if metric_name == "accuracy":
#             try:
#                 import evaluate
#                 acc = evaluate.load("accuracy")
#                 def compute(p):
#                     preds = p.predictions.argmax(-1)
#                     return acc.compute(predictions=preds, references=p.label_ids)
#                 return compute
#             except Exception:
#                 def compute(p):
#                     preds = p.predictions.argmax(-1)
#                     y = p.label_ids
#                     return {"accuracy": float(np.mean(preds == y))}
#                 return compute

#         raise ValueError('metric_name должен быть "f1" или "accuracy"')

#     # --------------------------
#     # Обучение
#     # --------------------------
#     def fit(
#         self,
#         train_data: pd.DataFrame,
#         epochs: int = 3,
#         test_size: float = 0.2,
#         test_data: Optional[pd.DataFrame] = None,
#         per_device_train_batch_size: int = 16,
#         gradient_accumulation_steps: int = 1,
#         learning_rate: float = 2e-4,
#         metric_name: str = "accuracy",   # "accuracy" | "f1"
#         fp16: bool = True,
#         logging_steps: int = 50,
#         eval_steps: int = 200,
#         output_dir: str = "./result_cls",
#         seed: int = 42,
#         gradient_checkpointing: bool = False,
#         class_weights: Optional[np.ndarray] = None,  # если None — авто-расчёт по train_data
#     ):
#         import os
#         from transformers import TrainingArguments
    
#         # 1) Проверки и подготовка
#         self._validate_modalities(train_data)
#         set_seed(seed)
    
#         # train/val split
#         df_train, df_eval = (train_data, test_data) if test_data is not None else self._split(train_data, test_size, seed)
    
#         # labels mapping
#         self._ensure_label_mapping(df_train)
    
#         # class weights
#         if class_weights is None:
#             y_train_all = np.array([self.label2id[y] for y in df_train[self.target_column_name].tolist()], dtype=int)
#             counts = np.bincount(y_train_all, minlength=self.num_labels)
#             n_all = counts.sum()
#             cw = np.zeros(self.num_labels, dtype=np.float32)
#             nz = counts > 0
#             cw[nz] = n_all / (self.num_labels * counts[nz].astype(np.float32))
#             class_weights = cw
    
#         # 2) Модель и процессоры (auto)
#         self.processors, self.model = self._build_model_and_processors()
#         if gradient_checkpointing:
#             try:
#                 self.model.gradient_checkpointing_enable()  # если поддерживается
#             except Exception:
#                 pass
    
#         # 3) Datasets (без чанков)
#         has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
#         pretokenize_train = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
#                              and len(df_train) <= self.max_pretokenize_samples)
#         pretokenize_eval  = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
#                              and len(df_eval)  <= self.max_pretokenize_samples)
    
#         ds_train = MultiModalDataset(
#             df=df_train,
#             target_col=self.target_column_name,
#             task="classification",
#             label2id=self.label2id,
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_train) else None,
#             pretokenize=pretokenize_train,
#             pretokenize_batch_size=self.pretokenize_batch_size
#         )
#         ds_eval = MultiModalDataset(
#             df=df_eval,
#             target_col=self.target_column_name,
#             task="classification",
#             label2id=self.label2id,
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_eval) else None,
#             pretokenize=pretokenize_eval,
#             pretokenize_batch_size=self.pretokenize_batch_size
#         )
    
#         collate = MultiModalCollator(self.processors, task="classification")
#         compute_metrics = self._build_compute_metrics(metric_name)

#         args = TrainingArguments(
#             output_dir=output_dir,
#             num_train_epochs=epochs,
#             per_device_train_batch_size=per_device_train_batch_size,
#             per_device_eval_batch_size=max(4, per_device_train_batch_size // 2),
#             gradient_accumulation_steps=gradient_accumulation_steps,
#             eval_accumulation_steps=max(1, gradient_accumulation_steps * 2),
#             learning_rate=learning_rate,
#             warmup_ratio=0.1,
#             lr_scheduler_type="cosine",
#             weight_decay=0.01,
#             eval_strategy="steps",
#             eval_steps=eval_steps,
#             save_strategy="steps",
#             save_steps=eval_steps,
#             load_best_model_at_end=True,
#             metric_for_best_model=f"eval_{metric_name}",
#             save_total_limit=1,
#             logging_strategy="steps",
#             logging_steps=logging_steps,
#             report_to="none",
#             fp16=fp16 and torch.cuda.is_available(),
#             dataloader_num_workers=min(4, os.cpu_count() or 4),
#             seed=seed,
#             remove_unused_columns=False,
#             gradient_checkpointing=gradient_checkpointing,
#             dataloader_pin_memory=True,
#             ddp_find_unused_parameters=False,
#             disable_tqdm=False
#         )
    
#         # 5) Trainer и старт обучения
#         self.trainer = WeightedCETrainer(
#             model=self.model,
#             args=args,
#             train_dataset=ds_train,
#             eval_dataset=ds_eval,
#             data_collator=collate,
#             compute_metrics=compute_metrics,
#             class_weights=class_weights
#         )
    
#         # (необязательно) убрать стандартный принтер-коллбек
#         try:
#             from transformers.trainer_callback import PrinterCallback
#             self.trainer.remove_callback(PrinterCallback)
#         except Exception:
#             pass
    
#         self.trainer.train()
    
#         # очистить кэш токенизатора (если был)
#         if has_bt:
#             try: self.processors["text"].clear_cache()
#             except Exception: pass
    
#         return self

#     # --------------------------
#     # Предсказания
#     # --------------------------
#     def predict(
#         self,
#         df: pd.DataFrame,
#         return_label_str: bool = False,
#         return_proba: bool = False,
#         batch_size: Optional[int] = None
#     ) -> np.ndarray:
#         if self.trainer is None:
#             raise RuntimeError("Модель не обучена. Вызовите .fit().")

#         df_c = self._ensure_label_column(df)

#         has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
#         ds = MultiModalDataset(
#             df=df_c,
#             target_col=self.target_column_name,
#             task="classification",
#             label2id=self.label2id,
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if has_bt else None,
#             pretokenize=(self.pretokenize_data and has_bt and len(df_c) < 10000 and self.text_padding != "dynamic"),
#             pretokenize_batch_size=self.pretokenize_batch_size
#         )
#         collate = MultiModalCollator(self.processors, task="classification")

#         # override bs
#         if batch_size:
#             original_bs = self.trainer.args.per_device_eval_batch_size
#             self.trainer.args.per_device_eval_batch_size = batch_size

#         effective_batch_size = batch_size or self.trainer.args.per_device_eval_batch_size
#         num_batches = (len(df_c) + effective_batch_size - 1) // effective_batch_size
#         print(f"Running predictions (batch_size={effective_batch_size}, num_batches={num_batches})...")

#         original_disable_tqdm = self.trainer.args.disable_tqdm
#         self.trainer.args.disable_tqdm = False

#         preds = self.trainer.predict(test_dataset=ds, metric_key_prefix="predict")

#         self.trainer.args.disable_tqdm = original_disable_tqdm
#         if batch_size:
#             self.trainer.args.per_device_eval_batch_size = original_bs

#         ds.clear_cache()

#         logits = preds.predictions
#         if return_proba:
#             exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
#             probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
#             return probabilities

#         y_pred = np.argmax(logits, axis=-1)
#         if return_label_str:
#             return np.array([self.id2label[int(i)] for i in y_pred])
#         return y_pred

#     # --------------------------
#     # Эмбеддинги
#     # --------------------------
#     def get_embeddings(
#         self,
#         df: pd.DataFrame,
#         batch_size: int = 32,
#         return_per_modality: bool = False
#     ):
#         if self.trainer is None or self.model is None:
#             raise RuntimeError("Модель не обучена. Вызовите .fit().")

#         device = next(self.trainer.model.parameters()).device
#         self.model.to(device).eval()

#         df_c = self._ensure_label_column(df)

#         has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
#         ds = MultiModalDataset(
#             df=df_c,
#             target_col=self.target_column_name,
#             task="classification",
#             label2id=self.label2id,
#             text_columns=self.text_columns,
#             image_columns=self.image_columns,
#             audio_columns=self.audio_columns,
#             text_tokenizer=self.processors["text"].bt if has_bt else None,
#             pretokenize=False
#         )
#         collate = MultiModalCollator(self.processors, task="classification")
#         loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)

#         fused_list = []
#         per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

#         num_batches = (len(df_c) + batch_size - 1) // batch_size
#         print(f"Extracting embeddings (batch_size={batch_size}, num_batches={num_batches})...")

#         with torch.no_grad():
#             for batch in tqdm(loader, total=num_batches, desc="Extracting embeddings", unit="batch", leave=True):
#                 bi = batch["backend_inputs"]

#                 def move_to_device(obj):
#                     if torch.is_tensor(obj): return obj.to(device)
#                     if isinstance(obj, dict): return {k: move_to_device(v) for k, v in obj.items()}
#                     if isinstance(obj, list): return [move_to_device(v) for v in obj]
#                     return obj

#                 bi = move_to_device(bi)
#                 fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
#                 fused_list.append(fused.cpu().numpy())
#                 if return_per_modality:
#                     for m in per_mod_lists.keys():
#                         if m in per:
#                             per_mod_lists[m].append(per[m].cpu().numpy())

#         fused_arr = np.vstack(fused_list)
#         if not return_per_modality:
#             print(f"✓ Embeddings shape: {fused_arr.shape}")
#             return fused_arr

#         per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
#         print(f"✓ Fused embeddings shape: {fused_arr.shape}")
#         for m, arr in per_mod.items():
#             print(f"✓ {m.capitalize()} embeddings shape: {arr.shape}")

#         return fused_arr, per_mod

# # ensembles/stacking.py
# from typing import List, Optional, Literal, Tuple, Union
# import numpy as np

# from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.preprocessing import StandardScaler, LabelEncoder

# Task = Literal["classification", "regression"]
# MetaType = Literal["auto", "logistic", "ridge", "linear"]


# def _softmax(x: np.ndarray, axis: int = 1) -> np.ndarray:
#     x = x - np.max(x, axis=axis, keepdims=True)
#     e = np.exp(x)
#     return e / np.sum(e, axis=axis, keepdims=True)


# def _check_shapes(outputs_list: List[np.ndarray]) -> Tuple[int, Tuple[int, ...]]:
#     if len(outputs_list) == 0:
#         raise ValueError("Пустой список выходов базовых моделей")
#     n = outputs_list[0].shape[0]
#     for i, a in enumerate(outputs_list):
#         if a.ndim != 2:
#             raise ValueError(f"Ожидался массив 2D для модели #{i}, получено shape={a.shape}")
#         if a.shape[0] != n:
#             raise ValueError(f"Несогласованная размерность N: outputs[0].shape[0]={n} != outputs[{i}].shape[0]={a.shape[0]}")
#     return n, outputs_list[0].shape


# class StackingMetaLearner:
#     """
#     Стакинг-ансамбль с обучением только мета-модели (склейка выходов базовых моделей):
#       - classification: LogisticRegression по признакам = concat(логиты|вероятности) базовых моделей
#       - regression: Ridge (по умолчанию) или LinearRegression по concat предсказаний базовых моделей

#     Теперь поддерживает текстовые метки классов (y как список/массив строк).
#     Метки автоматически кодируются во внутренние индексы через LabelEncoder.

#     Как использовать:
#       - classification:
#           outputs_list = [logits_model1, logits_model2, ...], каждый [N, C]
#           y = [N] (int или str — можно строки-классы)
#           meta.fit(outputs_list, y)
#           meta.predict(outputs_list_new, return_labels=True) -> строковые метки
#       - regression:
#           outputs_list = [preds_model1, preds_model2, ...], каждый [N, K]
#           y = [N] или [N, K]
#     """
#     def __init__(
#         self,
#         task: Task = "classification",
#         meta: MetaType = "auto",
#         use_proba: bool = True,
#         standardize: bool = False,
#         ridge_alpha: float = 1.0,
#         logistic_C: float = 1.0,
#         logistic_penalty: Union[str, None] = "l2",
#         class_weight: Optional[Union[str, dict]] = None,  # dict: можно передавать маппинг по строковым классам
#         fit_intercept: bool = True,
#         random_state: Optional[int] = None
#     ):
#         self.task = task
#         self.meta = meta
#         self.use_proba = use_proba
#         self.standardize = standardize
#         self.ridge_alpha = ridge_alpha
#         self.logistic_C = logistic_C
#         self.logistic_penalty = logistic_penalty
#         self.class_weight = class_weight
#         self.fit_intercept = fit_intercept
#         self.random_state = random_state

#         self.model = None
#         self.scaler: Optional[StandardScaler] = None

#         # Информация о форме признаков/классов
#         self.n_models_: Optional[int] = None
#         self.n_classes_: Optional[int] = None
#         self.n_targets_: Optional[int] = None  # для регрессии

#         # Для классификации: энкодер и классы
#         self.label_encoder_: Optional[LabelEncoder] = None
#         self.classes_: Optional[np.ndarray] = None

#         # Внутреннее преобразование весов классов (после энкодинга)
#         self._class_weight_enc: Optional[Union[str, dict]] = None

#     # ------------- Вспомогательное -------------
#     def _stack_features_classification(self, logits_list: List[np.ndarray]) -> np.ndarray:
#         """
#         Превращает список логитов/вероятностей в матрицу признаков X.
#         - Если use_proba=True: применяем softmax к каждому [N,C] и конкатенируем -> [N, M*C]
#         - Иначе: конкатенируем логиты -> [N, M*C]
#         """
#         n, (N, C) = _check_shapes(logits_list)
#         self.n_models_ = len(logits_list)
#         self.n_classes_ = C
#         parts = [(_softmax(z) if self.use_proba else z) for z in logits_list]
#         X = np.hstack(parts).astype(np.float64, copy=False)
#         return X

#     def _stack_features_regression(self, preds_list: List[np.ndarray]) -> np.ndarray:
#         """
#         Превращает список предсказаний регрессии в матрицу признаков X.
#         Каждый элемент: [N, K_i] (обычно K_i совпадает у всех моделей), конкатенация -> [N, sum(K_i)]
#         """
#         n, _ = _check_shapes(preds_list)
#         self.n_models_ = len(preds_list)
#         # Воспримем K по первой модели
#         K = preds_list[0].shape[1]
#         self.n_targets_ = K
#         X = np.hstack(preds_list).astype(np.float64, copy=False)
#         return X

#     def _build_X(self, outputs_list: List[np.ndarray]) -> np.ndarray:
#         if self.task == "classification":
#             return self._stack_features_classification(outputs_list)
#         else:
#             return self._stack_features_regression(outputs_list)

#     def _init_model(self, y_encoded: np.ndarray):
#         if self.task == "classification":
#             if self.meta in ("auto", "logistic"):
#                 solver = "lbfgs" if (self.logistic_penalty in ("l2", None, "none")) else "saga"
#                 penalty = None if self.logistic_penalty in (None, "none") else self.logistic_penalty
#                 self.model = LogisticRegression(
#                     C=self.logistic_C,
#                     penalty=penalty,
#                     class_weight=self._class_weight_enc,  # уже преобразованные веса
#                     fit_intercept=self.fit_intercept,
#                     multi_class="auto",
#                     solver=solver,
#                     max_iter=2000,
#                     random_state=self.random_state
#                 )
#             else:
#                 raise ValueError("Для classification поддерживается только meta='logistic' (или 'auto').")
#         else:
#             # regression
#             if self.meta in ("auto", "ridge"):
#                 base = Ridge(alpha=self.ridge_alpha, fit_intercept=self.fit_intercept, random_state=self.random_state)
#             elif self.meta == "linear":
#                 base = LinearRegression(fit_intercept=self.fit_intercept)
#             else:
#                 raise ValueError("Для regression meta должно быть 'ridge'|'linear'|'auto'.")

#             # Мультивыходная регрессия: завернём в MultiOutputRegressor при K>1
#             if y_encoded.ndim == 2 and y_encoded.shape[1] > 1:
#                 self.model = MultiOutputRegressor(base)
#             else:
#                 self.model = base

#     def _encode_class_weights_if_needed(self):
#         """
#         Преобразует словарь весов классов из оригинальных меток в индексы,
#         если была использована LabelEncoder.
#         """
#         if not isinstance(self.class_weight, dict):
#             self._class_weight_enc = self.class_weight  # 'balanced' или None
#             return
#         cw = {}
#         for k, v in self.class_weight.items():
#             if self.label_encoder_ is not None and not isinstance(k, (int, np.integer)):
#                 idx = int(self.label_encoder_.transform([k])[0])
#             else:
#                 idx = int(k)
#             cw[idx] = float(v)
#         self._class_weight_enc = cw

#     # ------------- Публичный API -------------
#     def fit(self, outputs_list: List[np.ndarray], y: np.ndarray):
#         """
#         Обучение только мета-модели.
#         - classification: outputs_list = [logits_i: [N, C]], y: [N] (int или str — допускаются строковые метки)
#         - regression:     outputs_list = [preds_i:  [N, K]], y: [N] или [N, K]
#         """
#         if len(outputs_list) == 0:
#             raise ValueError("outputs_list пуст")

#         X = self._build_X(outputs_list)

#         y = np.asarray(y)
#         if X.shape[0] != y.shape[0]:
#             raise ValueError(f"N samples mismatch: X={X.shape[0]} vs y={y.shape[0]}")

#         # Классификация: допускаем строковые метки
#         if self.task == "classification":
#             if y.ndim != 1:
#                 raise ValueError("Для классификации y должен быть одномерным массивом меток.")

#             # Определяем, нужно ли кодировать метки
#             if y.dtype.kind in ("U", "S", "O"):  # строки/объекты
#                 self.label_encoder_ = LabelEncoder()
#                 y_encoded = self.label_encoder_.fit_transform(y)
#                 self.classes_ = self.label_encoder_.classes_
#             else:
#                 # уже числовые
#                 self.label_encoder_ = None
#                 y_encoded = y.astype(int, copy=False)
#                 # classes_ можно получить после fit у модели, но сохраним явные уникальные
#                 self.classes_ = np.unique(y_encoded)

#             # Стандартизация (опционально)
#             if self.standardize:
#                 self.scaler = StandardScaler(with_mean=True, with_std=True)
#                 X = self.scaler.fit_transform(X)
#             else:
#                 self.scaler = None

#             # Веса классов: преобразуем словарь по необходимости
#             self._encode_class_weights_if_needed()

#             # Инициализация и обучение мета-модели
#             self._init_model(y_encoded)
#             self.model.fit(X, y_encoded.astype(int))

#         else:
#             # Регрессия
#             if self.standardize:
#                 self.scaler = StandardScaler(with_mean=True, with_std=True)
#                 X = self.scaler.fit_transform(X)
#             else:
#                 self.scaler = None

#             y_encoded = y  # для регрессии кодирование не нужно
#             self._init_model(y_encoded)
#             self.model.fit(X, y_encoded)

#         return self

#     def _transform_X(self, outputs_list: List[np.ndarray]) -> np.ndarray:
#         X = self._build_X(outputs_list)
#         if self.scaler is not None:
#             X = self.scaler.transform(X)
#         return X

#     def predict(self, outputs_list: List[np.ndarray], return_labels: bool = False) -> np.ndarray:
#         """
#         - classification: возврат индексов классов [N] по умолчанию; если return_labels=True — строковые метки
#         - regression:     возврат предсказаний [N] или [N, K]
#         """
#         if self.model is None:
#             raise RuntimeError("Мета-модель не обучена. Вызовите fit().")
#         X = self._transform_X(outputs_list)
#         y_pred = self.model.predict(X)

#         if self.task == "classification" and return_labels:
#             if self.label_encoder_ is not None:
#                 return self.label_encoder_.inverse_transform(y_pred.astype(int))
#             else:
#                 # метки изначально были числовые — вернуть как есть
#                 return y_pred

#         return y_pred

#     def predict_labels(self, outputs_list: List[np.ndarray]) -> np.ndarray:
#         """
#         Удобный синоним для классификации: сразу вернуть строковые метки классов [N].
#         Если метки изначально были числовые — вернёт числа.
#         """
#         return self.predict(outputs_list, return_labels=True)

#     def predict_proba(self, outputs_list: List[np.ndarray]) -> np.ndarray:
#         """
#         Только для classification: возврат вероятностей [N, C].
#         """
#         if self.task != "classification":
#             raise RuntimeError("predict_proba доступен только для classification.")
#         if self.model is None:
#             raise RuntimeError("Мета-модель не обучена. Вызовите fit().")
#         X = self._transform_X(outputs_list)
#         if hasattr(self.model, "predict_proba"):
#             return self.model.predict_proba(X)
#         # fallback (не должен понадобиться для LogisticRegression)
#         logits = self.model.decision_function(X)
#         return _softmax(logits)

In [ ]:
# import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!pip install evaluate wav2clip

# core/utils.py
import os
import random
import numpy as np
import torch
from PIL import Image

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def to_pil(x):
    if isinstance(x, Image.Image): return x.convert("RGB")
    if isinstance(x, str): return Image.open(x).convert("RGB")
    if isinstance(x, np.ndarray): return Image.fromarray(x).convert("RGB")
    raise ValueError("Expected path/np.ndarray/PIL.Image")

def load_audio(path: str, target_sr: int) -> np.ndarray:
    try:
        import torchaudio
    except Exception as e:
        raise RuntimeError("Требуется torchaudio: pip install torchaudio") from e
    waveform, sr = torchaudio.load(path)
    if waveform.size(0) > 1: waveform = waveform.mean(dim=0, keepdim=True)
    if sr != target_sr: waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)
    return waveform.squeeze(0).numpy().astype(np.float32)

def safe_load(component_cls, checkpoint: str, cache_dir: str = "./model_cache",
              local_files_only: bool = None, **kwargs):
    if local_files_only is None:
        local_files_only = os.environ.get("HF_HUB_OFFLINE", "0") == "1"
    name = getattr(component_cls, "__name__", "")
    if "Tokenizer" in name:
        kwargs.setdefault("use_fast", True)
    return component_cls.from_pretrained(
        checkpoint, cache_dir=cache_dir, local_files_only=local_files_only, **kwargs
    )

# core/types.py
import torch

def preferred_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

# core/device.py
from typing import Any, Dict, List, Protocol, Literal
import torch

Modality = Literal["text", "image", "audio"]

class Processor(Protocol):
    modality: Modality
    def prepare_batch(self, raw_items: List[Any]) -> Dict[str, torch.Tensor]:
        ...

class Encoder(Protocol):
    modality: Modality
    embed_dim: int
    def encode_batch(self, inputs: Dict[str, torch.Tensor]) -> torch.Tensor:
        ...

class ItemAggregator(Protocol):
    def aggregate(self, embs: torch.Tensor, counts: List[int], max_items: int, how: str) -> torch.Tensor:
        ...

class Fusion(Protocol):
    def __call__(self, zs: Dict[Modality, torch.Tensor]) -> torch.Tensor:
        ...

# data/augmentation.py
import os
import uuid
import numpy as np
import pandas as pd
from typing import Any, List, Optional, Callable
from PIL import Image
from tqdm.auto import tqdm

def augment_data_table(
    df: pd.DataFrame,
    save_dir: str,
    image_columns: Optional[List[str]] = None,
    text_columns: Optional[List[str]] = None,
    audio_columns: Optional[List[str]] = None,
    image_augment_fn: Optional[List[Callable[[Image.Image], Image.Image]]] = None,
    text_augment_fn: Optional[List[Callable[[str], str]]] = None,
    audio_augment_fn: Optional[List[Callable[..., np.ndarray]]] = None,
    id_col: Optional[str] = None,
    audio_sr: int = 16000,
) -> pd.DataFrame:
    """
    Делает простую аугментацию данных.
    - Выполняет аугментацию ТОЛЬКО для тех модальностей, для которых передан список функций.
    - Для каждой функции модальности создаёт ОДИН вариант датасета (без перемножения между модальностями).
    - Картинки/аудио сохраняются в save_dir, в таблице подставляются пути к новым файлам.
    - Возвращает конкатенацию всех вариантов с колонкой __aug.
    """
    image_columns = image_columns or []
    text_columns  = text_columns or []
    audio_columns = audio_columns or []

    os.makedirs(save_dir, exist_ok=True)
    run_tag = uuid.uuid4().hex[:8]

    def _row_id(i: int) -> str:
        if id_col and id_col in df.columns:
            return str(df.iloc[i][id_col])
        return f"row{i}"

    def _to_pil(v: Any) -> Image.Image:
        if isinstance(v, Image.Image): return v.convert("RGB")
        if isinstance(v, str):         return Image.open(v).convert("RGB")
        if isinstance(v, np.ndarray):
            if v.dtype != np.uint8: v = np.clip(v, 0, 255).astype(np.uint8)
            return Image.fromarray(v)
        raise TypeError("Unsupported image type")

    def _save_img(im: Image.Image, out_dir: str, name: str) -> str:
        os.makedirs(out_dir, exist_ok=True)
        path = os.path.join(out_dir, f"{name}.png")
        im.save(path)
        return path

    def _read_audio(x: Any) -> tuple[np.ndarray, int]:
        if isinstance(x, str):
            try:
                import soundfile as sf
                y, sr = sf.read(x, always_2d=False)
            except Exception:
                from scipy.io import wavfile
                sr, y = wavfile.read(x)
            y = np.asarray(y)
            if y.ndim > 1: y = y.mean(axis=-1)
            y = (y.astype(np.float32) / (32768.0 if y.dtype.kind in "iu" else 1.0))
            return np.clip(y, -1.0, 1.0), sr
        elif isinstance(x, np.ndarray):
            y = x
            if y.ndim > 1: y = y.mean(axis=-1)
            y = y.astype(np.float32, copy=False)
            if y.dtype.kind in "iu": y = y / 32768.0
            return np.clip(y, -1.0, 1.0), audio_sr
        raise TypeError("Unsupported audio type")

    def _save_wav(y: np.ndarray, sr: int, out_dir: str, name: str) -> str:
        os.makedirs(out_dir, exist_ok=True)
        path = os.path.join(out_dir, f"{name}.wav")
        try:
            import soundfile as sf
            sf.write(path, y.astype(np.float32, copy=False), sr, subtype="PCM_16")
        except Exception:
            from scipy.io import wavfile
            wavfile.write(path, sr, (np.clip(y, -1, 1) * 32767).astype(np.int16))
        return path

    out_parts = []

    # ТОЛЬКО если переданы функции и есть столбцы изображений
    if image_columns and image_augment_fn:
        for k, fn in tqdm(list(enumerate(image_augment_fn)), desc='Аугментация картинок'):
            df_aug = df.copy(deep=True)
            for col_i, col in enumerate(image_columns):
                out_dir = os.path.join(save_dir, "images", col, f"aug_image_{k}_{run_tag}")
                df_aug[col] = [
                    _save_img(fn(_to_pil(v)), out_dir, f"{_row_id(i)}") if v is not None else v
                    for i, v in tqdm(list(enumerate(df[col].tolist())),
                                     desc=f'Столбец {col_i+1}/{len(image_columns)}', leave=False)
                ]
            df_aug["__aug"] = f"image_{k}"
            out_parts.append(df_aug)

    # ТОЛЬКО если переданы функции и есть текстовые столбцы
    if text_columns and text_augment_fn:
        for k, fn in tqdm(list(enumerate(text_augment_fn)), desc='Аугментация текстов'):
            df_aug = df.copy(deep=True)
            for col_i, col in enumerate(text_columns):
                df_aug[col] = [
                    fn(v) if isinstance(v, str) else v
                    for v in tqdm(df[col].tolist(),
                                  desc=f'Столбец {col_i+1}/{len(text_columns)}', leave=False)
                ]
            df_aug["__aug"] = f"text_{k}"
            out_parts.append(df_aug)

    # ТОЛЬКО если переданы функции и есть аудио столбцы
    if audio_columns and audio_augment_fn:
        for k, fn in tqdm(list(enumerate(audio_augment_fn)), desc='Аугментация звуков'):
            df_aug = df.copy(deep=True)
            for col_i, col in enumerate(audio_columns):
                out_dir = os.path.join(save_dir, "audio", col, f"aug_audio_{k}_{run_tag}")
                new_vals = []
                for i, v in tqdm(list(enumerate(df[col].tolist())),
                                 desc=f'Столбец {col_i+1}/{len(audio_columns)}', leave=False):
                    if v is None:
                        new_vals.append(v); continue
                    y, sr = _read_audio(v)
                    try:    y2 = fn(y, sr)
                    except TypeError: y2 = fn(y)
                    path = _save_wav(np.asarray(y2, dtype=np.float32), sr, out_dir, f"{_row_id(i)}")
                    new_vals.append(path)
                df_aug[col] = new_vals
            df_aug["__aug"] = f"audio_{k}"
            out_parts.append(df_aug)

    if not out_parts:
        raise ValueError("Не переданы функции аугментации ни для одной модальности — нечего аугментировать.")

    return pd.concat(out_parts, ignore_index=True)

# data/tokenization.py
from functools import lru_cache
from typing import Dict, List, Optional
import numpy as np
import torch

class BatchTokenizer:
    def __init__(self, tokenizer, max_length=512, cache_size=10000, batch_size=256, padding_strategy="max_length"):
        assert padding_strategy in {"max_length", "dynamic"}
        self.tok = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size
        self.padding_strategy = padding_strategy
        self._cache = lru_cache(maxsize=cache_size)(self._tokenize_single)
        self.is_fast = hasattr(tokenizer, "is_fast") and tokenizer.is_fast
        if self.is_fast:
            print("✓ Используется Fast Tokenizer")

    def _tokenize_single(self, text: str) -> Dict[str, np.ndarray]:
        res = self.tok(text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        return {k: v.squeeze(0).cpu().numpy() for k, v in res.items()}

    def tokenize_batch(self, texts: List[str], use_cache: bool = True) -> Dict[str, torch.Tensor]:
        if self.padding_strategy == "dynamic":
            res = self.tok(texts, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt")
            return {k: (v.long() if k in ("input_ids","attention_mask","token_type_ids") else v) for k, v in res.items()}
        if use_cache and len(texts) < 100:
            items = [self._cache(t) for t in texts]
            keys = items[0].keys()
            out = {}
            for k in keys:
                dtype = torch.long if k in ("input_ids","attention_mask","token_type_ids") else torch.float32
                out[k] = torch.tensor(np.stack([it[k] for it in items]), dtype=dtype)
            return out
        res = self.tok(texts, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        return {k: (v.long() if k in ("input_ids","attention_mask","token_type_ids") else v) for k, v in res.items()}

    def tokenize_dataset_lazy(self, texts: List[str], batch_size: Optional[int] = None):
        b = batch_size or self.batch_size
        for i in range(0, len(texts), b):
            yield self.tokenize_batch(texts[i:i+b], use_cache=False)

    def clear_cache(self):
        self._cache.cache_clear()

# data/dataset.py
import gc
from typing import Any, Dict, List, Optional, Callable
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

class MultiModalDataset(Dataset):
    """
    Универсальный датасет для классификации и регрессии.
    Поддерживает:
      - text pretokenize: BatchTokenizer или кастомные функции (single/batched)
      - dynamic/max паддинг (dynamic отключает предтокенизацию)
      - мульти-изображения и мульти-аудио на сэмпл
    """
    def __init__(
        self,
        df: pd.DataFrame,
        target_col: Optional[str],                 # для регрессии — колонка с np.ndarray; для классификации — метка
        task: str = "regression",                  # "regression" | "classification"
        label2id: Optional[Dict[Any, int]] = None, # для классификации
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,
        text_tokenizer=None,                       # BatchTokenizer
        text_tokenizer_fn: Optional[Callable] = None,
        text_tokenizer_fn_batched: Optional[Callable] = None,
        special_tokens: Optional[Dict[str, Any]] = None,
        pretokenize: bool = False,
        pretokenize_batch_size: int = 256,
        tokenizer_returns_tensors: bool = False,
        deduplicate_texts: bool = True
    ):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.task = task.lower()
        self.label2id = label2id or {}
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []

        self.batch_tokenizer = text_tokenizer
        self.text_tokenizer_fn = text_tokenizer_fn
        self.text_tokenizer_fn_batched = text_tokenizer_fn_batched

        self.special_tokens = special_tokens or {"sep": " [SEP] "}
        self.tokenizer_returns_tensors = tokenizer_returns_tensors

        self._N = len(self.df)

        # labels
        if self.task == "classification":
            if self.target_col is None:
                y = np.zeros(self._N, dtype=np.int64)
            else:
                y = self.df[self.target_col].map(self.label2id).fillna(0).astype(int).values
            self._labels = torch.tensor(y, dtype=torch.long)
        else:
            self._labels = self._prepare_labels_regression(self.df, self.target_col)

        # lists for images/audios
        self._image_lists = self._collect_multi_values(self.df, self.image_columns) if self.image_columns else None
        self._audio_lists = self._collect_multi_values(self.df, self.audio_columns) if self.audio_columns else None

        self._tok_bank: Optional[Dict[str, torch.Tensor]] = None
        self._has_text = bool(self.text_columns)

        # dynamic padding -> disable pretokenization
        if pretokenize and self.batch_tokenizer is not None and getattr(self.batch_tokenizer, "padding_strategy", "max_length") == "dynamic":
            print("⚠ Предтокенизация отключена: выбран dynamic-паддинг для текста.")
            pretokenize = False

        if self._has_text and pretokenize:
            if self.batch_tokenizer is not None and self.text_tokenizer_fn is None and self.text_tokenizer_fn_batched is None:
                self._pretokenize_with_batch_tokenizer(pretokenize_batch_size)
            else:
                self._pretokenize_with_custom_fn(pretokenize_batch_size, deduplicate_texts)

    # labels for regression: [N, K]
    def _prepare_labels_regression(self, df: pd.DataFrame, target_col: Optional[str]) -> torch.Tensor:
        if target_col is None or target_col not in df.columns:
            return torch.zeros((len(df), 1), dtype=torch.float32)
        labels_list = []
        for i in range(len(df)):
            v = df.iloc[i][target_col]
            if isinstance(v, (list, tuple, np.ndarray)):
                arr = np.asarray(v, dtype=np.float32)
            else:
                try:
                    arr = np.asarray([float(v)], dtype=np.float32)
                except Exception:
                    arr = np.asarray([0.0], dtype=np.float32)
            labels_list.append(arr)
        K = max(a.shape[0] for a in labels_list) if labels_list else 1
        out = np.zeros((len(labels_list), K), dtype=np.float32)
        for i, a in enumerate(labels_list):
            out[i, :a.shape[0]] = a
        return torch.tensor(out, dtype=torch.float32)

    def _join_text(self, row: pd.Series) -> str:
        sep = self.special_tokens.get("sep", " [SEP] ")
        return sep.join([("" if pd.isna(row[c]) else str(row[c])) for c in self.text_columns])

    @staticmethod
    def _as_list(v):
        if v is None or (isinstance(v, float) and np.isnan(v)):
            return []
        if isinstance(v, (list, tuple)):
            return list(v)
        return [v]

    def _collect_multi_values(self, df: pd.DataFrame, columns: List[str]) -> List[List[Any]]:
        out = []
        as_list = self._as_list
        for _, row in df.iterrows():
            lst: List[Any] = []
            for c in columns:
                if c in row:
                    lst.extend([x for x in as_list(row[c]) if x is not None])
            out.append(lst)
        return out

    def _pretokenize_with_batch_tokenizer(self, batch_size: int):
        print("Предтокенизация с BatchTokenizer...")
        texts = [self._join_text(self.df.iloc[i]) for i in range(self._N)]
        banks: Dict[str, List[torch.Tensor]] = {}
        for start in range(0, self._N, batch_size):
            batch = texts[start:start + batch_size]
            tok = self.batch_tokenizer.tokenize_batch(batch, use_cache=False)
            for k in tok:
                if k in ("input_ids","attention_mask","token_type_ids"):
                    tok[k] = tok[k].long()
                else:
                    tok[k] = tok[k].to(torch.float32)
            for k, v in tok.items():
                banks.setdefault(k, []).append(v)
        self._tok_bank = {k: torch.cat(v_parts, dim=0).contiguous() for k, v_parts in banks.items()}
        shapes = {k: tuple(v.shape) for k, v in self._tok_bank.items()}
        print(f"✓ Предтокенизация завершена: {self._N} образцов | keys={list(self._tok_bank.keys())}, shapes={shapes}")

    def _pretokenize_with_custom_fn(self, batch_size: int, deduplicate_texts: bool):
        cols = list(self.text_columns)
        if self.text_tokenizer_fn_batched is not None:
            print("Предтокенизация кастомной batched-функцией...")
            first_end = min(self._N, max(8, batch_size))
            batch_data = []
            for i in range(first_end):
                row = self.df.iloc[i]
                d = {c: ("" if pd.isna(row[c]) else str(row[c])) for c in cols}
                batch_data.append(d)
            first_tok = self.text_tokenizer_fn_batched(batch_data, self.special_tokens)
            if not isinstance(first_tok, dict):
                raise ValueError("text_tokenizer_fn_batched должна возвращать dict тензоров [B, L]")
            bank: Dict[str, torch.Tensor] = {}
            for k, t in first_tok.items():
                if not torch.is_tensor(t): t = torch.tensor(t)
                bank[k] = torch.empty((self._N, t.size(1)), dtype=t.dtype)
                bank[k][:first_end] = t[:first_end]
            for start in range(first_end, self._N, batch_size):
                end = min(self._N, start + batch_size)
                batch_data = []
                for i in range(start, end):
                    row = self.df.iloc[i]
                    d = {c: ("" if pd.isna(row[c]) else str(row[c])) for c in cols}
                    batch_data.append(d)
                tok = self.text_tokenizer_fn_batched(batch_data, self.special_tokens)
                for k, t in tok.items():
                    if not torch.is_tensor(t): t = torch.tensor(t)
                    bank[k][start:end] = t
            self._tok_bank = {k: v.contiguous() for k, v in bank.items()}
            shapes = {k: tuple(v.shape) for k, v in self._tok_bank.items()}
            print(f"✓ Предтокенизация кастомной batched-функцией завершена: shapes={shapes}")
            return

        print("Предтокенизация кастомной single-функцией...")
        col_arrays = [self.df[c].astype(str).where(~self.df[c].isna(), other="").tolist() for c in cols]
        first_td = {c: col_arrays[i][0] for i, c in enumerate(cols)}
        first_tok = self.text_tokenizer_fn(first_td, self.special_tokens)
        if not isinstance(first_tok, dict):
            raise ValueError("custom text_tokenizer_fn должна возвращать dict тензоров")
        for k, t in first_tok.items():
            if not torch.is_tensor(t): first_tok[k] = torch.tensor(t)
        bank: Dict[str, torch.Tensor] = {k: torch.empty((self._N, *t.shape), dtype=t.dtype) for k, t in first_tok.items()}
        for k, t in first_tok.items():
            bank[k][0].copy_(t)

        cache: Dict[tuple, Dict[str, torch.Tensor]] = {}
        if deduplicate_texts:
            key0 = tuple(first_td.get(c, "") for c in cols)
            cache[key0] = {k: v.clone() for k, v in first_tok.items()}

        for i in range(1, self._N):
            td = {c: col_arrays[j][i] for j, c in enumerate(cols)}
            if deduplicate_texts:
                key = tuple(td.get(c, "") for c in cols)
                tok = cache.get(key)
                if tok is None:
                    tok = self.text_tokenizer_fn(td, self.special_tokens)
                    for k, t in tok.items():
                        if not torch.is_tensor(t): tok[k] = torch.tensor(t)
                    cache[key] = {k: v.clone() for k, v in tok.items()}
            else:
                tok = self.text_tokenizer_fn(td, self.special_tokens)
                for k, t in tok.items():
                    if not torch.is_tensor(t): tok[k] = torch.tensor(t)
            for k, t in tok.items():
                bank[k][i].copy_(t)

        self._tok_bank = {k: v.contiguous() for k, v in bank.items()}
        shapes = {k: tuple(v.shape) for k, v in self._tok_bank.items()}
        print(f"✓ Предтокенизация кастомной single-функцией завершена: shapes={shapes}")

    def __len__(self):
        return self._N

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        item: Dict[str, Any] = {}
        item["labels"] = self._labels[idx]  # [K] float32 для регрессии, int64 для классификации
        if self.text_columns:
            if self._tok_bank is not None:
                item["text_tokens"] = {k: v[idx] for k, v in self._tok_bank.items()}
            else:
                item["text"] = self._join_text(self.df.iloc[idx])
        if self._image_lists is not None:
            item["images"] = self._image_lists[idx]
        if self._audio_lists is not None:
            item["audios"] = self._audio_lists[idx]
        return item

    def get_cache_stats(self):
        has = self._tok_bank is not None
        sizes = {k: tuple(v.shape) for k, v in (self._tok_bank or {}).items()}
        return {"has_pretokenized": has, "shapes": sizes, "N": self._N}

    def clear_cache(self):
        self._tok_bank = None
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

# data/collate.py
from typing import Any, Dict, List, Optional
import numpy as np
import torch
# from mmkit.core.utils import to_pil, load_audio

class MultiModalCollator:
    """
    Готовит backend_inputs для модели.
    - Если в item есть "text_tokens": стакает их.
    - Иначе использует processors["text"] для батчевой токенизации.
    - Изображения/аудио: разворачивает в плоский список + считает counts.
    """
    def __init__(self, processors: Dict[str, Any], task: str = "regression", audio_sr_fallback: int = 16000):
        self.processors = processors
        self.task = task.lower()
        self.audio_sr_fallback = audio_sr_fallback

    def __call__(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        labels = [b.get("labels") for b in batch]
        if labels and isinstance(labels[0], torch.Tensor):
            labels = torch.stack(labels)
        else:
            if self.task == "classification":
                labels = torch.tensor(labels, dtype=torch.long)
            else:
                labels = torch.tensor(labels, dtype=torch.float32)

        out = {"labels": labels, "backend_inputs": {}}
        # text
        if "text" in self.processors or any(("text_tokens" in b) for b in batch):
            if "text_tokens" in batch[0]:
                t0 = batch[0]["text_tokens"]
                text_inputs = {}
                for key in t0.keys():
                    if torch.is_tensor(t0[key]):
                        text_inputs[key] = torch.stack([b["text_tokens"][key] for b in batch])
                    else:
                        dtype = torch.long if key in ("input_ids","attention_mask","token_type_ids") else torch.float32
                        text_inputs[key] = torch.tensor([b["text_tokens"][key] for b in batch], dtype=dtype)
            else:
                texts = [b.get("text", "") or " " for b in batch]
                text_inputs = self.processors["text"].prepare_batch(texts)
            out["backend_inputs"]["text_inputs"] = text_inputs

        # images
        if "image" in self.processors:
            flat_images, counts = [], []
            for b in batch:
                lst = b.get("images", []) or []
                lst = [to_pil(x) for x in lst if x is not None]
                counts.append(len(lst))
                flat_images.extend(lst)
            if len(flat_images) > 0:
                out["backend_inputs"]["image_inputs"] = self.processors["image"].prepare_batch(flat_images)
            else:
                out["backend_inputs"]["image_inputs"] = {"pixel_values": None}
            out["backend_inputs"]["image_counts"] = torch.tensor(counts, dtype=torch.long)

        # audio
        if "audio" in self.processors:
            flat, counts = [], []
            for b in batch:
                lst = b.get("audios", []) or []
                counts.append(len(lst))
                for a in lst:
                    if isinstance(a, str):
                        sr = getattr(self.processors["audio"], "sr", self.audio_sr_fallback)
                        flat.append(load_audio(a, sr))
                    else:
                        arr = np.asarray(a, dtype=np.float32)
                        if arr.ndim>1: arr = np.squeeze(arr)
                        if arr.ndim>1: arr = arr.reshape(-1)
                        flat.append(arr)
            out["backend_inputs"]["audio_counts"] = torch.tensor(counts, dtype=torch.long)
            out["backend_inputs"]["audio_inputs"] = self.processors["audio"].prepare_batch(flat) if len(flat)>0 else {"input_values": None, "input_features": None, "raw_audios": []}

        out["backend_inputs"]["batch_size"] = len(batch)
        return out

# processors/text.py
# from mmkit.data.tokenization import BatchTokenizer

class TextProcessor:
    modality = "text"
    def __init__(self, tokenizer, max_length=512, padding="max_length", cache_size=10000, batch_size=256):
        self.bt = BatchTokenizer(tokenizer, max_length=max_length, cache_size=cache_size, batch_size=batch_size, padding_strategy=padding)
    def prepare_batch(self, texts):
        return self.bt.tokenize_batch(texts, use_cache=True)
    def clear_cache(self):
        self.bt.clear_cache()

# processors/image.py
class ImageProcessor:
    modality = "image"
    def __init__(self, hf_processor):
        self.proc = hf_processor
    def prepare_batch(self, images):
        if len(images)==0: return {"pixel_values": None}
        x = self.proc(images=images, return_tensors="pt")
        return {"pixel_values": x["pixel_values"]}

# processors/audio.py
class ClapAudioProcessor:
    modality="audio"
    def __init__(self, hf_processor, sr=48000):
        self.proc = hf_processor
        self.sr = sr
    def prepare_batch(self, raw_list):
        x = self.proc(audios=raw_list, sampling_rate=self.sr, padding=True, return_tensors="pt")
        return {"input_features": x["input_features"]}

class Wav2VecAudioProcessor:
    modality="audio"
    def __init__(self, hf_processor, sr=16000):
        self.proc = hf_processor
        self.sr = sr
    def prepare_batch(self, raw_list):
        x = self.proc(raw_list, sampling_rate=self.sr, padding=True, return_tensors="pt")
        return {"input_values": x["input_values"]}

class Wav2ClipAudioProcessor:
    modality="audio"
    def __init__(self, sr=16000):
        self.sr = sr
    def prepare_batch(self, raw_list):
        return {"raw_audios": raw_list}  # список np.ndarray; энкодер сам обработает

# encoders/base.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class BaseEncoder(nn.Module):
    modality: str = "text"
    embed_dim: int = 0

    def _normalize(self, x: torch.Tensor) -> torch.Tensor:
        if x.dim() > 2:
            x = x.view(x.size(0), -1)
        return F.normalize(x, dim=-1, eps=1e-12)

# encoders/text_auto.py
import torch
from transformers import AutoModel
# from mmkit.core.utils import safe_load
# from .base import BaseEncoder

class AutoTextEncoder(BaseEncoder):
    modality="text"
    def __init__(self, checkpoint="bert-base-multilingual-cased", cache_dir="./model_cache"):
        super().__init__()
        self.model = safe_load(AutoModel, checkpoint, cache_dir)
        self.embed_dim = self.model.config.hidden_size

    @torch.no_grad()
    def encode_batch(self, inputs):
        out = self.model(**inputs)
        z = out.pooler_output if getattr(out, "pooler_output", None) is not None else out.last_hidden_state.mean(dim=1)
        return self._normalize(z)

# encoders/text_clip.py
import torch
from transformers import CLIPModel
# from mmkit.core.utils import safe_load
# from .base import BaseEncoder

class CLIPTextEncoder(BaseEncoder):
    modality="text"
    def __init__(self, checkpoint="openai/clip-vit-base-patch32", cache_dir="./model_cache"):
        super().__init__()
        self.model = safe_load(CLIPModel, checkpoint, cache_dir)
        self.embed_dim = self.model.config.projection_dim

    @torch.no_grad()
    def encode_batch(self, inputs):
        z = self.model.get_text_features(**inputs)
        return self._normalize(z)

# encoders/image_auto.py
import torch
from transformers import AutoModel
# from mmkit.core.utils import safe_load
# from .base import BaseEncoder

class AutoImageEncoder(BaseEncoder):
    modality="image"
    def __init__(self, checkpoint="google/vit-base-patch16-224", cache_dir="./model_cache"):
        super().__init__()
        self.model = safe_load(AutoModel, checkpoint, cache_dir)
        self.embed_dim = self.model.config.hidden_size

    @torch.no_grad()
    def encode_batch(self, inputs):
        pv = inputs.get("pixel_values", None)
        if pv is None: 
            return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
        out = self.model(pixel_values=pv)
        z = out.pooler_output if getattr(out, "pooler_output", None) is not None else out.last_hidden_state[:, 0]
        return self._normalize(z)

# encoders/image_clip.py
import torch
from transformers import CLIPModel
# from mmkit.core.utils import safe_load
# from .base import BaseEncoder

class CLIPImageEncoder(BaseEncoder):
    modality="image"
    def __init__(self, checkpoint="openai/clip-vit-base-patch32", cache_dir="./model_cache"):
        super().__init__()
        self.model = safe_load(CLIPModel, checkpoint, cache_dir)
        self.embed_dim = self.model.config.projection_dim

    @torch.no_grad()
    def encode_batch(self, inputs):
        pv = inputs.get("pixel_values", None)
        if pv is None:
            return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
        z = self.model.get_image_features(pixel_values=pv)
        return self._normalize(z)

# encoders/audio_clap.py
import torch
from transformers import ClapModel
# from mmkit.core.utils import safe_load
# from .base import BaseEncoder

class CLAPAudioEncoder(BaseEncoder):
    modality="audio"
    def __init__(self, checkpoint="laion/clap-htsat-unfused", cache_dir="./model_cache"):
        super().__init__()
        self.model = safe_load(ClapModel, checkpoint, cache_dir)
        self.embed_dim = getattr(self.model.config, "projection_dim", 512)

    @torch.no_grad()
    def encode_batch(self, inputs):
        feats = inputs.get("input_features")
        if feats is None:
            return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
        z = self.model.get_audio_features(input_features=feats.float())
        return self._normalize(z.float())

# encoders/audio_wav2vec.py
import torch
from transformers import AutoModel
# from mmkit.core.utils import safe_load
# from .base import BaseEncoder

class Wav2Vec2Encoder(BaseEncoder):
    modality="audio"
    def __init__(self, checkpoint="facebook/wav2vec2-base-960h", cache_dir="./model_cache"):
        super().__init__()
        self.model = safe_load(AutoModel, checkpoint, cache_dir)
        self.embed_dim = getattr(self.model.config, "hidden_size", 768)

    @torch.no_grad()
    def encode_batch(self, inputs):
        iv = inputs.get("input_values")
        if iv is None:
            return torch.zeros(0, self.embed_dim, device=next(self.model.parameters()).device)
        out = self.model(input_values=iv.float())
        z = out.pooler_output if getattr(out, "pooler_output", None) is not None else out.last_hidden_state.mean(dim=1)
        return self._normalize(z.float())

# encoders/audio_wav2clip.py
import numpy as np
import torch
# from .base import BaseEncoder

class Wav2ClipEncoder(BaseEncoder):
    modality="audio"
    def __init__(self):
        super().__init__()
        try:
            import wav2clip as w2c
        except Exception as e:
            raise RuntimeError("Требуется wav2clip: pip install wav2clip") from e
        self.w2c = w2c
        model = None
        if hasattr(w2c, "get_model"):
            model = w2c.get_model()
        elif hasattr(w2c, "model"):
            m = w2c.model
            model = m() if callable(m) else m
        else:
            raise RuntimeError("wav2clip не содержит get_model()/model")
        self.model = model
        self.embed_dim = 512

    @torch.no_grad()
    def encode_batch(self, inputs):
        raws = inputs.get("raw_audios", [])
        if len(raws)==0:
            return torch.zeros(0, self.embed_dim)
        embs = []
        for arr in raws:
            a = np.asarray(arr, dtype=np.float32)
            if a.ndim>1: a = np.squeeze(a)
            if a.ndim>1: a = a.reshape(-1)
            if a.size < 512:
                a = np.pad(a, (0, 512 - a.size), mode="constant")
            try:
                e = self.w2c.embed_audio(a, self.model)
                e = np.asarray(e)
            except Exception:
                x = torch.from_numpy(a).float().unsqueeze(0)
                y = self.model(x)
                if isinstance(y, (tuple, list)):
                    y = y[0]
                if torch.is_tensor(y):
                    if y.dim()==2 and y.size(0)==1: y=y.squeeze(0)
                    e = y.detach().cpu().numpy()
                else:
                    e = np.asarray(y)
            if e.ndim>1: e = e.reshape(-1)
            embs.append(torch.as_tensor(e, dtype=torch.float32))
        z = torch.stack(embs, dim=0)
        z = torch.nn.functional.normalize(z, dim=-1, eps=1e-12)
        return z

# aggregation/item_pool.py
import torch
import torch.nn.functional as F

class ItemAggregator:
    def aggregate(self, embs: torch.Tensor, counts, max_k: int, how: str):
        if embs is None or (torch.is_tensor(embs) and embs.numel()==0):
            D = 0 if embs is None else getattr(embs, "size", lambda *_: 0)(-1)
            out_dim = D*max_k if how=="concat" else D
            return torch.zeros((len(counts), out_dim), device=embs.device if torch.is_tensor(embs) else "cpu", dtype=torch.float32)
        if embs.dim()==1: embs = embs.unsqueeze(0)
        if embs.dim()>2: embs = embs.view(embs.size(0), -1)
        N, D = embs.size()
        out_dim = D*max_k if how=="concat" else D
        out = torch.zeros((len(counts), out_dim), device=embs.device, dtype=embs.dtype)
        off = 0
        for i, c in enumerate(counts):
            if c<=0 or off>=N: continue
            take = min(c, N-off)
            sample = embs[off:off+take]
            off += take
            if how=="concat":
                sample = sample[:max_k]
                if sample.size(0)<max_k:
                    pad = torch.zeros((max_k - sample.size(0), D), device=embs.device, dtype=embs.dtype)
                    sample = torch.cat([sample, pad], dim=0)
                out[i] = sample.reshape(-1)
            else:
                out[i] = sample.mean(dim=0)
        return F.normalize(out, dim=-1, eps=1e-12)

# aggregation/fusion.py
import torch

class ConcatFusion:
    def __call__(self, zs: dict) -> torch.Tensor:
        feats = [zs[k] for k in ["image","text","audio"] if k in zs]
        return torch.cat(feats, dim=-1)

class MeanFusion:
    def __call__(self, zs: dict) -> torch.Tensor:
        feats = [zs[k] for k in ["image","text","audio"] if k in zs]
        return torch.stack(feats, dim=0).mean(dim=0)

# heads/regression.py
import torch.nn as nn

class RegressionHead(nn.Module):
    def __init__(self, in_dim, out_dim, hidden=256, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(in_dim),
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

# heads/classification.py
import torch.nn as nn

class ClassificationHead(nn.Module):
    def __init__(self, in_dim, num_labels, hidden=512, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(in_dim),
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_labels)
        )
    def forward(self, x): return self.net(x)

# model/multimodal.py
from typing import Dict
import torch
import torch.nn as nn
# from mmkit.aggregation.item_pool import ItemAggregator

class MultiModalModel(nn.Module):
    """
    Модель: dict(encoders) + aggregator (для image/audio множественных) + fusion + head.
    """
    def __init__(self, encoders: Dict[str, nn.Module], aggregator: ItemAggregator, fusion, head,
                 image_cfg=None, audio_cfg=None):
        super().__init__()
        self.enc = nn.ModuleDict(encoders)
        self.agg = aggregator
        self.fusion = fusion
        self.head = head
        self.image_cfg = image_cfg or {"max_items":1, "how":"concat"}
        self.audio_cfg = audio_cfg or {"max_items":1, "how":"concat"}

    def _fwd_features(self, backend_inputs: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        zs = {}
        if "text" in self.enc and "text_inputs" in backend_inputs:
            zs["text"] = self.enc["text"].encode_batch(backend_inputs["text_inputs"])
        if "image" in self.enc and "image_inputs" in backend_inputs:
            flat = self.enc["image"].encode_batch(backend_inputs["image_inputs"])
            counts = backend_inputs.get("image_counts", torch.tensor([0]*backend_inputs.get("batch_size", flat.size(0)))).tolist()
            zs["image"] = self.agg.aggregate(flat, counts, self.image_cfg["max_items"], self.image_cfg["how"])
        if "audio" in self.enc and "audio_inputs" in backend_inputs:
            flat = self.enc["audio"].encode_batch(backend_inputs["audio_inputs"])
            counts = backend_inputs.get("audio_counts", torch.tensor([0]*backend_inputs.get("batch_size", flat.size(0)))).tolist()
            zs["audio"] = self.agg.aggregate(flat, counts, self.audio_cfg["max_items"], self.audio_cfg["how"])
        return zs

    def forward(self, backend_inputs: Dict[str, torch.Tensor], labels: torch.Tensor = None):
        zs = self._fwd_features(backend_inputs)
        if not zs:
            raise ValueError("Энкодеры не вернули эмбеддинги")
        fused = self.fusion(zs)
        logits = self.head(fused)
        return {"logits": logits}

    @torch.no_grad()
    def get_embeddings(self, backend_inputs: Dict[str, torch.Tensor], return_per_modality: bool = False):
        zs = self._fwd_features(backend_inputs)
        fused = self.fusion(zs)
        return (fused, zs) if return_per_modality else fused

# train/trainer_hf.py
from typing import Optional
import torch
import torch.nn.functional as F
from transformers import Trainer

class MSETrainer(Trainer):
    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: Optional[int] = None,
        **kwargs
    ):
        labels = inputs.pop("labels").to(torch.float32)
        backend_inputs = inputs.pop("backend_inputs", None)
        if backend_inputs is None:
            # fallback: все остальное считаем backend_inputs
            backend_inputs = inputs

        out = model(backend_inputs=backend_inputs)  # передаем строго backend_inputs
        logits = out["logits"]
        preds = logits if logits.dim() == 1 else (logits.squeeze(-1) if logits.size(-1) == 1 else logits)
        labels = labels.view_as(preds)
        loss = F.mse_loss(preds, labels)
        return (loss, out) if return_outputs else loss

class WeightedCETrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = (
            torch.as_tensor(class_weights, dtype=torch.float32) if class_weights is not None else None
        )

    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: Optional[int] = None,
        **kwargs
    ):
        labels = inputs.pop("labels")
        backend_inputs = inputs.pop("backend_inputs", None)
        if backend_inputs is None:
            backend_inputs = inputs

        out = model(backend_inputs=backend_inputs)  # только backend_inputs
        logits = out["logits"]

        weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss = F.cross_entropy(logits, labels.long(), weight=weight)
        return (loss, out) if return_outputs else loss

# train/callbacks.py
from transformers.trainer_callback import TrainerCallback
from tqdm.auto import tqdm

class PbarConsoleLogger(TrainerCallback):
    def __init__(self, pbar):
        self.pbar = pbar
        self.last_logs = {}
        self.last_train_loss = None
        self.printed_eval_steps = set()
        self.tqdm = tqdm

    def _step(self, state) -> int:
        return int(state.global_step or 0)

    def _fmt_postfix(self):
        parts = []
        if 'loss' in self.last_logs:
            parts.append(f"loss {self.last_logs['loss']:.4f}")
        if 'eval_loss' in self.last_logs:
            parts.append(f"val {self.last_logs['eval_loss']:.4f}")
        for k, v in self.last_logs.items():
            if k.startswith('eval_') and k not in (
                'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'
            ):
                parts.append(f"{k.replace('eval_', '')} {v:.4f}")
        return " | ".join(parts)

    def on_step_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        if self.last_logs:
            self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return
        for k, v in logs.items():
            if isinstance(v, (int, float)):
                self.last_logs[k] = float(v)
        if 'loss' in logs and isinstance(logs['loss'], (int, float)):
            self.last_train_loss = float(logs['loss'])

        self.pbar.set_postfix_str(self._fmt_postfix(), refresh=False)
        self.pbar.refresh()

        if any(k.startswith('eval_') for k in logs.keys()):
            step = self._step(state)
            if step in self.printed_eval_steps:
                return
            self.printed_eval_steps.add(step)
            train_loss_str = f"{self.last_train_loss:.10f}" if self.last_train_loss is not None else "n/a"
            val_loss = logs.get('eval_loss', None)
            val_loss_str = f"{float(val_loss):.10g}" if isinstance(val_loss, (int, float)) else "n/a"
            exclude = {'eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'}
            extra_parts = []
            for k, v in logs.items():
                if k.startswith('eval_') and k not in exclude:
                    metric_name = k.replace('eval_', '')
                    extra_parts.append(f"val {metric_name}: {float(v):.10f}")
            line = f"step: {step}, train loss: {train_loss_str}, val loss: {val_loss_str}"
            if extra_parts:
                line += ", " + ", ".join(extra_parts)
            self.tqdm.write(line)

    def on_train_end(self, args, state, control, **kwargs):
        n = min(self._step(state), self.pbar.total)
        if n > self.pbar.n:
            self.pbar.update(n - self.pbar.n)
        self.pbar.refresh()

# train/metrics.py
import numpy as np

def build_regression_metrics(name: str):
    name = name.lower()
    if name not in ("rmse", "mae", "r2"):
        raise ValueError('metric_name для регрессии должен быть "rmse", "mae" или "r2"')

    def compute(p):
        preds = p.predictions
        y = p.label_ids
        preds = preds.squeeze(-1) if preds.ndim == 2 and preds.shape[-1] == 1 else preds
        y = y.squeeze(-1) if y.ndim == 2 and y.shape[-1] == 1 else y
        axis = 1 if preds.ndim == 2 else None
        if name == "rmse":
            err = preds - y
            mse = np.mean(err**2, axis=axis)
            rmse = np.sqrt(mse)
            return {"rmse": float(np.mean(rmse))}
        elif name == "mae":
            mae = np.mean(np.abs(preds - y), axis=axis)
            return {"mae": float(np.mean(mae))}
        else:
            y_mean = np.mean(y, axis=axis, keepdims=True) if preds.ndim == 2 else np.mean(y)
            ss_res = np.sum((y - preds) ** 2, axis=axis)
            ss_tot = np.sum((y - y_mean) ** 2, axis=axis)
            r2 = 1.0 - (ss_res / (ss_tot + 1e-12))
            return {"r2": float(np.mean(r2))}
    return compute

# train/recipes.py
from transformers import CLIPTokenizer, CLIPImageProcessor, AutoTokenizer, AutoImageProcessor, ClapProcessor, AutoProcessor
# from mmkit.processors.text import TextProcessor
# from mmkit.processors.image import ImageProcessor
# from mmkit.processors.audio import ClapAudioProcessor, Wav2VecAudioProcessor, Wav2ClipAudioProcessor

# from mmkit.encoders.text_clip import CLIPTextEncoder
# from mmkit.encoders.text_auto import AutoTextEncoder
# from mmkit.encoders.image_clip import CLIPImageEncoder
# from mmkit.encoders.image_auto import AutoImageEncoder
# from mmkit.encoders.audio_clap import CLAPAudioEncoder
# from mmkit.encoders.audio_wav2vec import Wav2Vec2Encoder
# from mmkit.encoders.audio_wav2clip import Wav2ClipEncoder

# from mmkit.aggregation.item_pool import ItemAggregator
# from mmkit.aggregation.fusion import ConcatFusion, MeanFusion
# from mmkit.heads.regression import RegressionHead
# from mmkit.model.multimodal import MultiModalModel

def build_auto_regression(modalities,
                          fusion="concat",
                          # text config
                          text_model_type="clip", text_checkpoint="openai/clip-vit-base-patch32", text_max_length=77, text_padding="max_length",
                          # image config
                          image_model_type="clip", image_checkpoint="openai/clip-vit-base-patch32", max_images=1, image_agg="concat",
                          # audio config
                          audio_model_type="clap", audio_checkpoint="laion/clap-htsat-unfused", audio_sr=48000, max_audios=1, audio_agg="concat",
                          # head config
                          out_dim=1, hidden=256, dropout=0.1,
                          cache_dir="./model_cache"):
    encoders = {}
    processors = {}
    # text
    if "text" in modalities:
        if text_model_type == "clip":
            tok = CLIPTokenizer.from_pretrained(text_checkpoint, cache_dir=cache_dir)
            processors["text"] = TextProcessor(tok, max_length=text_max_length, padding=text_padding)
            encoders["text"] = CLIPTextEncoder(text_checkpoint, cache_dir=cache_dir)
            text_dim = encoders["text"].embed_dim
        else:
            tok = AutoTokenizer.from_pretrained(text_checkpoint, cache_dir=cache_dir)
            processors["text"] = TextProcessor(tok, max_length=text_max_length, padding=text_padding)
            encoders["text"] = AutoTextEncoder(text_checkpoint, cache_dir=cache_dir)
            text_dim = encoders["text"].embed_dim
    # image
    image_cfg = None
    if "image" in modalities:
        if image_model_type == "clip":
            ip = CLIPImageProcessor.from_pretrained(image_checkpoint, cache_dir=cache_dir)
            processors["image"] = ImageProcessor(ip)
            encoders["image"] = CLIPImageEncoder(image_checkpoint, cache_dir=cache_dir)
        else:
            ip = AutoImageProcessor.from_pretrained(image_checkpoint, cache_dir=cache_dir)
            processors["image"] = ImageProcessor(ip)
            encoders["image"] = AutoImageEncoder(image_checkpoint, cache_dir=cache_dir)
        image_dim = encoders["image"].embed_dim
        image_cfg = {"max_items": max_images, "how": image_agg}
    # audio
    audio_cfg = None
    if "audio" in modalities:
        if audio_model_type == "clap":
            ap = ClapProcessor.from_pretrained(audio_checkpoint, cache_dir=cache_dir)
            processors["audio"] = ClapAudioProcessor(ap, sr=audio_sr)
            encoders["audio"] = CLAPAudioEncoder(audio_checkpoint, cache_dir=cache_dir)
        elif audio_model_type == "wav2vec":
            ap = AutoProcessor.from_pretrained(audio_checkpoint, cache_dir=cache_dir)
            processors["audio"] = Wav2VecAudioProcessor(ap, sr=audio_sr)
            encoders["audio"] = Wav2Vec2Encoder(audio_checkpoint, cache_dir=cache_dir)
        else:  # wav2clip
            processors["audio"] = Wav2ClipAudioProcessor(sr=audio_sr)
            encoders["audio"] = Wav2ClipEncoder()
        audio_dim = encoders["audio"].embed_dim
        audio_cfg = {"max_items": max_audios, "how": audio_agg}

    agg = ItemAggregator()
    fusion_mod = ConcatFusion() if fusion=="concat" else MeanFusion()
    # compute in_dim
    dims = []
    for m in ["image","text","audio"]:
        if m in encoders:
            dims.append(encoders[m].embed_dim)
    if fusion=="concat":
        in_dim = sum(dims)
    else:
        assert len(set(dims))==1, "Для fusion=mean требуется одинаковая размерность эмбеддингов"
        in_dim = dims[0]
    head = RegressionHead(in_dim, out_dim=out_dim, hidden=hidden, dropout=dropout)
    model = MultiModalModel(encoders, agg, fusion_mod, head, image_cfg=image_cfg, audio_cfg=audio_cfg)
    return processors, model

# pipelines/regression.py
import math
import gc
from typing import Any, Dict, List, Optional
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from transformers import TrainingArguments
# from mmkit.core.utils import set_seed
# from mmkit.data.dataset import MultiModalDataset
# from mmkit.data.collate import MultiModalCollator
# from mmkit.train.trainer_hf import MSETrainer
# from mmkit.train.callbacks import PbarConsoleLogger
# from mmkit.train.metrics import build_regression_metrics
# from mmkit.train.recipes import build_auto_regression

class MultiModalRegressionPipeline:
    """
    Высокоуровневый пайплайн мультимодальной регрессии (multi-target).
    Поддерживает: text/image/audio, dynamic/max padding, предтокенизацию, чанки, RMSE/MAE/R2.
    """
    def __init__(self,
                 modalities: List[str],
                 target_column_names: List[str],
                 text_columns: Optional[List[str]] = None,
                 image_columns: Optional[List[str]] = None,
                 audio_columns: Optional[List[str]] = None,
                 # recipe params
                 backend: str = "auto",
                 clip_checkpoint: str = "openai/clip-vit-base-patch32",
                 clap_checkpoint: str = "laion/clap-htsat-unfused",
                 text_model_config: Optional[Dict[str, Any]] = None,
                 image_model_config: Optional[Dict[str, Any]] = None,
                 audio_model_config: Optional[Dict[str, Any]] = None,
                 fusion: str = "concat",
                 # tokenizer
                 text_padding: str = "max_length",
                 text_max_length: int = 77,
                 use_batch_tokenizer: bool = True,
                 pretokenize_data: bool = True,
                 pretokenize_batch_size: int = 256,
                 tokenizer_cache_size: int = 10000,
                 max_pretokenize_samples: int = 100000,
                 local_cache_dir: str = "./model_cache"):
        self.modalities = sorted(list(set(modalities)))
        self.target_column_names = list(target_column_names)
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.backend_name = backend.lower()
        self.clip_checkpoint = clip_checkpoint
        self.clap_checkpoint = clap_checkpoint
        self.text_model_config = text_model_config
        self.image_model_config = image_model_config
        self.audio_model_config = audio_model_config
        self.fusion = fusion

        self.text_padding = text_padding
        self.text_max_length = text_max_length
        self.use_batch_tokenizer = use_batch_tokenizer
        self.pretokenize_data = pretokenize_data
        self.pretokenize_batch_size = pretokenize_batch_size
        self.tokenizer_cache_size = tokenizer_cache_size
        self.max_pretokenize_samples = max_pretokenize_samples
        self.local_cache_dir = local_cache_dir

        self._target_vec_col = "__target_vector__"
        self.processors = {}
        self.model = None
        self.trainer = None

    def _attach_target_vector(self, df: pd.DataFrame, fill_zeros: bool = False) -> pd.DataFrame:
        df_c = df.copy()
        K = len(self.target_column_names)
        if fill_zeros:
            df_c[self._target_vec_col] = [np.zeros(K, dtype=np.float32) for _ in range(len(df_c))]
        else:
            def _row_to_vec(row):
                vals = [row[c] for c in self.target_column_names]
                return np.asarray(vals, dtype=np.float32)
            df_c[self._target_vec_col] = df_c.apply(_row_to_vec, axis=1)
        return df_c

    def _validate_modalities(self, df: pd.DataFrame):
        if "text" in self.modalities:
            assert self.text_columns, "Вы выбрали text, но text_columns пустой"
            missing = [c for c in self.text_columns if c not in df.columns]
            if missing: raise ValueError(f"Нет текстовых колонок: {missing}")
        if "image" in self.modalities:
            assert self.image_columns, "Вы выбрали image, но image_columns пуст"
            missing = [c for c in self.image_columns if c not in df.columns]
            if missing: raise ValueError(f"Нет колонок изображений: {missing}")
        if "audio" in self.modalities:
            assert self.audio_columns, "Вы выбрали audio, но audio_columns пуст"
            missing = [c for c in self.audio_columns if c not in df.columns]
            if missing: raise ValueError(f"Нет колонок аудио: {missing}")

    def _validate_targets(self, df: pd.DataFrame):
        miss = [c for c in self.target_column_names if c not in df.columns]
        if miss:
            raise ValueError(f"В DataFrame отсутствуют целевые колонки: {miss}")

    def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42):
        df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
        n_eval = int(math.ceil(len(df) * test_size))
        return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

    def _build_model(self, num_outputs: int, image_cfg: Dict[str, Any], audio_cfg: Dict[str, Any]):
        # Авто-рецепт: CLIP для text+image, CLAP для audio (или явные конфиги)
        if self.backend_name == "auto":
            # auto defaults
            txt_type = "clip" if "image" in self.modalities else ("auto" if "text" in self.modalities else None)
            img_type = "clip" if "image" in self.modalities else None
            aud_type = "clap" if "audio" in self.modalities else None

            txt_cp = self.clip_checkpoint if txt_type=="clip" else (self.text_model_config["checkpoint"] if self.text_model_config else "bert-base-multilingual-cased")
            img_cp = self.clip_checkpoint if img_type=="clip" else (self.image_model_config["checkpoint"] if self.image_model_config else "google/vit-base-patch16-224")
            aud_cp = self.clap_checkpoint if aud_type=="clap" else (self.audio_model_config["checkpoint"] if self.audio_model_config else "facebook/wav2vec2-base-960h")

            if aud_type is None and "audio" in self.modalities and self.audio_model_config:
                aud_type = self.audio_model_config.get("model_type","clap")

            processors, model = build_auto_regression(
                modalities=self.modalities,
                fusion=self.fusion,
                text_model_type="clip" if txt_type=="clip" else "auto",
                text_checkpoint=txt_cp,
                text_max_length=self.text_max_length,
                text_padding=self.text_padding,
                image_model_type="clip" if img_type=="clip" else "auto",
                image_checkpoint=img_cp,
                max_images=image_cfg.get("max_images", 1),
                image_agg=image_cfg.get("image_agg", "concat"),
                audio_model_type=aud_type if aud_type else "clap",
                audio_checkpoint=aud_cp,
                audio_sr=audio_cfg.get("sr", 48000),
                max_audios=audio_cfg.get("max_audios", 1),
                audio_agg=audio_cfg.get("audio_agg", "concat"),
                out_dim=num_outputs,
                hidden=256,
                dropout=0.1,
                cache_dir=self.local_cache_dir
            )
            return processors, model

        # Иначе — можно сделать фабрику из переданных конфигов (опущено для краткости)
        raise NotImplementedError("Ручная сборка (не auto) не реализована в этой версии")

    def fit(
        self,
        train_data: pd.DataFrame,
        epochs: int = 3,
        test_size: float = 0.2,
        test_data: Optional[pd.DataFrame] = None,
        per_device_train_batch_size: int = 16,
        gradient_accumulation_steps: int = 1,
        learning_rate: float = 2e-4,
        metric_name: str = "rmse",       # "rmse" | "mae" | "r2"
        fp16: bool = True,
        logging_steps: int = 50,
        eval_steps: int = 200,
        output_dir: str = "./result_reg",
        seed: int = 42,
        gradient_checkpointing: bool = False
    ):
        import os
        from transformers import TrainingArguments
    
        # 1) Проверки и подготовка
        self._validate_modalities(train_data)
        self._validate_targets(train_data)
        set_seed(seed)
    
        df_train, df_eval = (train_data, test_data) if test_data is not None else self._split(train_data, test_size, seed)
        if test_data is not None:
            self._validate_targets(test_data)
    
        # Преобразуем таргеты в одну вектор-колонку
        df_train_ext = self._attach_target_vector(df_train, fill_zeros=False)
        df_eval_ext  = self._attach_target_vector(df_eval,  fill_zeros=False)
    
        # 2) Модель и процессоры (auto-рецепт)
        image_cfg = {"max_images": 1, "image_agg": "concat"}
        audio_cfg = {"sr": 48000, "max_audios": 1, "audio_agg": "concat"}
    
        self.processors, self.model = self._build_model(
            num_outputs=len(self.target_column_names),
            image_cfg=image_cfg,
            audio_cfg=audio_cfg
        )
    
        if gradient_checkpointing:
            try:
                self.model.gradient_checkpointing_enable()
            except Exception:
                pass
    
        # 3) Datasets (без чанков)
        has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
        pretokenize_train = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
                             and len(df_train_ext) <= self.max_pretokenize_samples)
        pretokenize_eval  = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
                             and len(df_eval_ext)  <= self.max_pretokenize_samples)
    
        ds_train = MultiModalDataset(
            df=df_train_ext,
            target_col="__target_vector__",
            task="regression",
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_train) else None,
            pretokenize=pretokenize_train,
            pretokenize_batch_size=self.pretokenize_batch_size
        )
        ds_eval = MultiModalDataset(
            df=df_eval_ext,
            target_col="__target_vector__",
            task="regression",
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_eval) else None,
            pretokenize=pretokenize_eval,
            pretokenize_batch_size=self.pretokenize_batch_size
        )
    
        collate = MultiModalCollator(self.processors, task="regression")
        compute_metrics = build_regression_metrics(metric_name)
    
        # 4) Аргументы обучения (eval_strategy)
        args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=max(4, per_device_train_batch_size // 2),
            gradient_accumulation_steps=gradient_accumulation_steps,
            eval_accumulation_steps=max(1, gradient_accumulation_steps * 2),
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.0,
            eval_strategy="steps",
            eval_steps=eval_steps,
            save_strategy="steps",
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=f"eval_{metric_name}",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            report_to="none",
            fp16=fp16 and torch.cuda.is_available(),
            dataloader_num_workers=min(4, os.cpu_count() or 4),
            seed=seed,
            remove_unused_columns=False,
            gradient_checkpointing=gradient_checkpointing,
            dataloader_pin_memory=True,
            ddp_find_unused_parameters=False,
            disable_tqdm=False
        )
    
        # 5) Trainer и старт обучения
        self.trainer = MSETrainer(
            model=self.model,
            args=args,
            train_dataset=ds_train,
            eval_dataset=ds_eval,
            data_collator=collate,
            compute_metrics=compute_metrics
        )
    
        try:
            from transformers.trainer_callback import PrinterCallback
            self.trainer.remove_callback(PrinterCallback)
        except Exception:
            pass
    
        self.trainer.train()
    
        if has_bt:
            try: self.processors["text"].clear_cache()
            except Exception: pass
    
        return self

    def predict(self, df: pd.DataFrame, batch_size: Optional[int] = None) -> np.ndarray:
        if self.trainer is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        df_c = self._attach_target_vector(df, fill_zeros=True)
        has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")

        ds = MultiModalDataset(
            df=df_c,
            target_col="__target_vector__",
            task="regression",
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if has_bt else None,
            pretokenize=(self.pretokenize_data and has_bt and len(df_c) < 10000 and self.text_padding != "dynamic"),
            pretokenize_batch_size=self.pretokenize_batch_size
        )
        collate = MultiModalCollator(self.processors, task="regression")

        if batch_size:
            original_bs = self.trainer.args.per_device_eval_batch_size
            self.trainer.args.per_device_eval_batch_size = batch_size

        effective_batch = batch_size or self.trainer.args.per_device_eval_batch_size
        num_batches = (len(df_c) + effective_batch - 1) // effective_batch
        print(f"Running predictions (batch_size={effective_batch}, num_batches={num_batches})...")

        original_disable_tqdm = self.trainer.args.disable_tqdm
        self.trainer.args.disable_tqdm = False

        preds = self.trainer.predict(test_dataset=ds, metric_key_prefix="predict")

        self.trainer.args.disable_tqdm = original_disable_tqdm
        if batch_size:
            self.trainer.args.per_device_eval_batch_size = original_bs

        ds.clear_cache()
        y = preds.predictions
        y = y.squeeze(-1) if y.ndim == 2 and y.shape[-1] == 1 else y
        return y

    def get_embeddings(self, df: pd.DataFrame, batch_size: int = 32, return_per_modality: bool = False):
        if self.trainer is None or self.model is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")
        device = next(self.trainer.model.parameters()).device
        self.model.to(device).eval()

        df_c = self._attach_target_vector(df, fill_zeros=True)
        has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")

        ds = MultiModalDataset(
            df=df_c,
            target_col="__target_vector__",
            task="regression",
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if has_bt else None,
            pretokenize=False
        )
        collate = MultiModalCollator(self.processors, task="regression")
        loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)

        fused_list = []
        per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

        num_batches = (len(df_c) + batch_size - 1) // batch_size
        print(f"Extracting embeddings (batch_size={batch_size}, num_batches={num_batches})...")

        with torch.no_grad():
            for batch in tqdm(loader, total=num_batches, desc="Extracting embeddings", unit="batch", leave=True):
                bi = batch["backend_inputs"]
                def move_to_device(obj):
                    if torch.is_tensor(obj): return obj.to(device)
                    if isinstance(obj, dict): return {k: move_to_device(v) for k, v in obj.items()}
                    if isinstance(obj, list): return [move_to_device(v) for v in obj]
                    return obj
                bi = move_to_device(bi)
                fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
                fused_list.append(fused.cpu().numpy())
                if return_per_modality:
                    for m in per_mod_lists.keys():
                        if m in per:
                            per_mod_lists[m].append(per[m].cpu().numpy())

        fused_arr = np.vstack(fused_list)
        if not return_per_modality:
            print(f"✓ Embeddings shape: {fused_arr.shape}")
            return fused_arr
        per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
        print(f"✓ Fused embeddings shape: {fused_arr.shape}")
        for m, arr in per_mod.items():
            print(f"✓ {m.capitalize()} embeddings shape: {arr.shape}")
        return fused_arr, per_mod

# pipelines/classification.py
import os
import math
import gc
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import TrainingArguments

# from mmkit.core.utils import set_seed
# from mmkit.data.dataset import MultiModalDataset
# from mmkit.data.collate import MultiModalCollator

# from mmkit.train.trainer_hf import WeightedCETrainer
# from mmkit.train.callbacks import PbarConsoleLogger

# from mmkit.aggregation.item_pool import ItemAggregator
# from mmkit.aggregation.fusion import ConcatFusion, MeanFusion
# from mmkit.heads.classification import ClassificationHead
# from mmkit.model.multimodal import MultiModalModel

# Процессоры
from transformers import (
    CLIPTokenizer,
    CLIPImageProcessor,
    AutoTokenizer,
    AutoImageProcessor,
    ClapProcessor,
    AutoProcessor,
)

# from mmkit.processors.text import TextProcessor
# from mmkit.processors.image import ImageProcessor
# from mmkit.processors.audio import (
#     ClapAudioProcessor,
#     Wav2VecAudioProcessor,
#     Wav2ClipAudioProcessor,
# )

# # Энкодеры
# from mmkit.encoders.text_clip import CLIPTextEncoder
# from mmkit.encoders.text_auto import AutoTextEncoder
# from mmkit.encoders.image_clip import CLIPImageEncoder
# from mmkit.encoders.image_auto import AutoImageEncoder
# from mmkit.encoders.audio_clap import CLAPAudioEncoder
# from mmkit.encoders.audio_wav2vec import Wav2Vec2Encoder
# from mmkit.encoders.audio_wav2clip import Wav2ClipEncoder


class MultiModalClassificationPipeline:
    """
    Высокоуровневый пайплайн мультимодальной классификации (single/multi-class).
    Поддерживает text/image/audio, dynamic/max padding, предтокенизацию, чанковую тренировку,
    accuracy/f1 метрики, class weights, predict и извлечение эмбеддингов.

    По умолчанию (backend="auto"):
      - если есть image: CLIP для text и image;
      - если есть audio: CLAP (или wav2vec/wav2clip по желанию — см. параметры),
      - для чистого текста: Auto (BERT) по умолчанию.

    Параметры:
      - modalities: список модальностей среди ["text","image","audio"]
      - target_column_name: имя колонки с метками (строки/инты)
      - num_labels: число классов (если None — возьмётся из train_data по unique)
      - text_columns, image_columns, audio_columns: колонки данных
      - fusion: "concat" | "mean" (для mean размеры эмбеддингов должны совпадать)
      - text_padding: "max_length" | "dynamic"
      - text_max_length: длина токенов
      - pretokenize_data: предтокенизация датасета (выключится при dynamic padding)
      - tokenizer_cache_size: LRU-кэш токенизатора
      - max_pretokenize_samples: лимит на предтокенизацию в чанке
      - local_cache_dir: кэш HF

      - audio_backend: "clap" | "wav2vec" | "wav2clip" (для audio)
      - audio_sr: sampling rate для аудио-процессоров
      - max_images_per_sample, image_agg: агрегация нескольких картинок ("concat"/"mean")
      - max_audios_per_sample, audio_agg: агрегация для аудио
    """
    def __init__(
        self,
        modalities: List[str],
        target_column_name: str,
        num_labels: Optional[int] = None,
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,

        # backend/модели
        backend: str = "auto",
        clip_checkpoint: str = "openai/clip-vit-base-patch32",
        clap_checkpoint: str = "laion/clap-htsat-unfused",
        text_model_config: Optional[Dict[str, Any]] = None,
        image_model_config: Optional[Dict[str, Any]] = None,
        audio_model_config: Optional[Dict[str, Any]] = None,

        fusion: str = "concat",

        # токенизация текста
        text_padding: str = "max_length",
        text_max_length: int = 77,
        use_batch_tokenizer: bool = True,
        pretokenize_data: bool = True,
        pretokenize_batch_size: int = 256,
        tokenizer_cache_size: int = 10000,
        max_pretokenize_samples: int = 100000,
        local_cache_dir: str = "./model_cache",

        # изображение/аудио агрегация
        max_images_per_sample: int = 1,
        image_agg: str = "concat",
        audio_backend: str = "clap",  # "clap" | "wav2vec" | "wav2clip"
        audio_sr: int = 48000,
        max_audios_per_sample: int = 1,
        audio_agg: str = "concat",
    ):
        self.modalities = sorted(list(set(modalities)))
        self.target_column_name = target_column_name
        self.num_labels = num_labels

        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []

        self.backend_name = backend.lower()
        self.clip_checkpoint = clip_checkpoint
        self.clap_checkpoint = clap_checkpoint
        self.text_model_config = text_model_config
        self.image_model_config = image_model_config
        self.audio_model_config = audio_model_config

        self.fusion = fusion
        self.text_padding = text_padding
        self.text_max_length = text_max_length
        self.use_batch_tokenizer = use_batch_tokenizer
        self.pretokenize_data = pretokenize_data
        self.pretokenize_batch_size = pretokenize_batch_size
        self.tokenizer_cache_size = tokenizer_cache_size
        self.max_pretokenize_samples = max_pretokenize_samples
        self.local_cache_dir = local_cache_dir

        self.max_images_per_sample = int(max_images_per_sample)
        self.image_agg = image_agg
        self.audio_backend = audio_backend
        self.audio_sr = int(audio_sr)
        self.max_audios_per_sample = int(max_audios_per_sample)
        self.audio_agg = audio_agg

        # Внутренние поля
        self.label2id: Optional[Dict[Any, int]] = None
        self.id2label: Optional[Dict[int, str]] = None
        self.processors: Dict[str, Any] = {}
        self.model: Optional[MultiModalModel] = None
        self.trainer: Optional[WeightedCETrainer] = None

    # --------------------------
    # Валидации и подсобки
    # --------------------------
    def _validate_modalities(self, df: pd.DataFrame):
        if "text" in self.modalities:
            assert self.text_columns, "Вы выбрали text, но text_columns пустой"
            missing = [c for c in self.text_columns if c not in df.columns]
            if missing:
                raise ValueError(f"Нет текстовых колонок: {missing}")
        if "image" in self.modalities:
            assert self.image_columns, "Вы выбрали image, но image_columns пуст"
            missing = [c for c in self.image_columns if c not in df.columns]
            if missing:
                raise ValueError(f"Нет колонок изображений: {missing}")
        if "audio" in self.modalities:
            assert self.audio_columns, "Вы выбрали audio, но audio_columns пуст"
            missing = [c for c in self.audio_columns if c not in df.columns]
            if missing:
                raise ValueError(f"Нет колонок аудио: {missing}")

    def _ensure_label_mapping(self, df_train: pd.DataFrame):
        classes = sorted(df_train[self.target_column_name].unique().tolist())
        if self.num_labels is None:
            self.num_labels = len(classes)
        elif self.num_labels != len(classes):
            print(f"Warning: num_labels={self.num_labels} != len(classes)={len(classes)}")
        self.label2id = {c: i for i, c in enumerate(classes)}
        self.id2label = {i: str(c) for c, i in self.label2id.items()}

    def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
        df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
        n_eval = int(math.ceil(len(df) * test_size))
        return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

    def _ensure_label_column(self, df: pd.DataFrame) -> pd.DataFrame:
        df_c = df.copy()
        if self.target_column_name not in df_c.columns:
            # Если нет метки — подставим первую известную
            fake_label = next(iter(self.label2id.keys()))
            df_c[self.target_column_name] = [fake_label] * len(df_c)
        return df_c

    # --------------------------
    # Модель и процессоры (auto)
    # --------------------------
    def _build_model_and_processors(self) -> Tuple[Dict[str, Any], MultiModalModel]:
        encoders = {}
        processors = {}

        # text
        if "text" in self.modalities:
            # Если есть image — используем CLIP-текст, иначе Auto (BERT)
            use_clip_text = ("image" in self.modalities)
            if use_clip_text:
                tok = CLIPTokenizer.from_pretrained(self.clip_checkpoint, cache_dir=self.local_cache_dir)
                processors["text"] = TextProcessor(
                    tok, max_length=self.text_max_length, padding=self.text_padding,
                    cache_size=self.tokenizer_cache_size, batch_size=self.pretokenize_batch_size
                )
                encoders["text"] = CLIPTextEncoder(self.clip_checkpoint, cache_dir=self.local_cache_dir)
            else:
                tok = AutoTokenizer.from_pretrained(
                    (self.text_model_config or {}).get("checkpoint", "bert-base-multilingual-cased"),
                    cache_dir=self.local_cache_dir
                )
                processors["text"] = TextProcessor(
                    tok, max_length=self.text_max_length, padding=self.text_padding,
                    cache_size=self.tokenizer_cache_size, batch_size=self.pretokenize_batch_size
                )
                encoders["text"] = AutoTextEncoder(
                    (self.text_model_config or {}).get("checkpoint", "bert-base-multilingual-cased"),
                    cache_dir=self.local_cache_dir
                )

        # image
        image_cfg = None
        if "image" in self.modalities:
            if (self.image_model_config or {}).get("model_type", "clip") == "clip":
                ip = CLIPImageProcessor.from_pretrained(self.clip_checkpoint, cache_dir=self.local_cache_dir)
                processors["image"] = ImageProcessor(ip)
                encoders["image"] = CLIPImageEncoder(self.clip_checkpoint, cache_dir=self.local_cache_dir)
            else:
                cp = (self.image_model_config or {}).get("checkpoint", "google/vit-base-patch16-224")
                ip = AutoImageProcessor.from_pretrained(cp, cache_dir=self.local_cache_dir)
                processors["image"] = ImageProcessor(ip)
                encoders["image"] = AutoImageEncoder(cp, cache_dir=self.local_cache_dir)
            image_cfg = {"max_items": self.max_images_per_sample, "how": self.image_agg}

        # audio
        audio_cfg = None
        if "audio" in self.modalities:
            ab = (self.audio_model_config or {}).get("model_type", self.audio_backend)
            if ab == "clap":
                cp = (self.audio_model_config or {}).get("checkpoint", self.clap_checkpoint)
                ap = ClapProcessor.from_pretrained(cp, cache_dir=self.local_cache_dir)
                processors["audio"] = ClapAudioProcessor(ap, sr=self.audio_sr)
                encoders["audio"] = CLAPAudioEncoder(cp, cache_dir=self.local_cache_dir)
            elif ab == "wav2vec":
                cp = (self.audio_model_config or {}).get("checkpoint", "facebook/wav2vec2-base-960h")
                ap = AutoProcessor.from_pretrained(cp, cache_dir=self.local_cache_dir)
                processors["audio"] = Wav2VecAudioProcessor(ap, sr=self.audio_sr)
                encoders["audio"] = Wav2Vec2Encoder(cp, cache_dir=self.local_cache_dir)
            else:  # wav2clip
                processors["audio"] = Wav2ClipAudioProcessor(sr=self.audio_sr)
                encoders["audio"] = Wav2ClipEncoder()
            audio_cfg = {"max_items": self.max_audios_per_sample, "how": self.audio_agg}

        # Fusion + Head
        agg = ItemAggregator()
        fusion_mod = ConcatFusion() if self.fusion == "concat" else MeanFusion()
        dims = [encoders[m].embed_dim for m in ["image", "text", "audio"] if m in encoders]
        if self.fusion == "concat":
            in_dim = sum(dims)
        else:
            assert len(set(dims)) == 1, "Для fusion='mean' размеры эмбеддингов всех модальностей должны совпадать"
            in_dim = dims[0]
        head = ClassificationHead(in_dim=in_dim, num_labels=self.num_labels, hidden=512, dropout=0.1)
        model = MultiModalModel(encoders, agg, fusion_mod, head, image_cfg=image_cfg, audio_cfg=audio_cfg)
        return processors, model

    # --------------------------
    # Метрики
    # --------------------------
    def _build_compute_metrics(self, metric_name: str):
        metric_name = metric_name.lower()
        if metric_name == "f1":
            try:
                import evaluate
                f1_metric = evaluate.load("f1")
                def compute(p):
                    preds = p.predictions.argmax(-1)
                    return f1_metric.compute(predictions=preds, references=p.label_ids, average="macro")
                return compute
            except Exception:
                print("evaluate не доступен, использую accuracy")
                metric_name = "accuracy"

        if metric_name == "accuracy":
            try:
                import evaluate
                acc = evaluate.load("accuracy")
                def compute(p):
                    preds = p.predictions.argmax(-1)
                    return acc.compute(predictions=preds, references=p.label_ids)
                return compute
            except Exception:
                def compute(p):
                    preds = p.predictions.argmax(-1)
                    y = p.label_ids
                    return {"accuracy": float(np.mean(preds == y))}
                return compute

        raise ValueError('metric_name должен быть "f1" или "accuracy"')

    # --------------------------
    # Обучение
    # --------------------------
    def fit(
        self,
        train_data: pd.DataFrame,
        epochs: int = 3,
        test_size: float = 0.2,
        test_data: Optional[pd.DataFrame] = None,
        per_device_train_batch_size: int = 16,
        gradient_accumulation_steps: int = 1,
        learning_rate: float = 2e-4,
        metric_name: str = "accuracy",   # "accuracy" | "f1"
        fp16: bool = True,
        logging_steps: int = 50,
        eval_steps: int = 200,
        output_dir: str = "./result_cls",
        seed: int = 42,
        gradient_checkpointing: bool = False,
        class_weights: Optional[np.ndarray] = None,  # если None — авто-расчёт по train_data
    ):
        import os
        from transformers import TrainingArguments
    
        # 1) Проверки и подготовка
        self._validate_modalities(train_data)
        set_seed(seed)
    
        # train/val split
        df_train, df_eval = (train_data, test_data) if test_data is not None else self._split(train_data, test_size, seed)
    
        # labels mapping
        self._ensure_label_mapping(df_train)
    
        # class weights
        if class_weights is None:
            y_train_all = np.array([self.label2id[y] for y in df_train[self.target_column_name].tolist()], dtype=int)
            counts = np.bincount(y_train_all, minlength=self.num_labels)
            n_all = counts.sum()
            cw = np.zeros(self.num_labels, dtype=np.float32)
            nz = counts > 0
            cw[nz] = n_all / (self.num_labels * counts[nz].astype(np.float32))
            class_weights = cw
    
        # 2) Модель и процессоры (auto)
        self.processors, self.model = self._build_model_and_processors()
        if gradient_checkpointing:
            try:
                self.model.gradient_checkpointing_enable()  # если поддерживается
            except Exception:
                pass
    
        # 3) Datasets (без чанков)
        has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
        pretokenize_train = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
                             and len(df_train) <= self.max_pretokenize_samples)
        pretokenize_eval  = (self.pretokenize_data and has_bt and self.text_padding != "dynamic"
                             and len(df_eval)  <= self.max_pretokenize_samples)
    
        ds_train = MultiModalDataset(
            df=df_train,
            target_col=self.target_column_name,
            task="classification",
            label2id=self.label2id,
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_train) else None,
            pretokenize=pretokenize_train,
            pretokenize_batch_size=self.pretokenize_batch_size
        )
        ds_eval = MultiModalDataset(
            df=df_eval,
            target_col=self.target_column_name,
            task="classification",
            label2id=self.label2id,
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if (has_bt and pretokenize_eval) else None,
            pretokenize=pretokenize_eval,
            pretokenize_batch_size=self.pretokenize_batch_size
        )
    
        collate = MultiModalCollator(self.processors, task="classification")
        compute_metrics = self._build_compute_metrics(metric_name)

        args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=max(4, per_device_train_batch_size // 2),
            gradient_accumulation_steps=gradient_accumulation_steps,
            eval_accumulation_steps=max(1, gradient_accumulation_steps * 2),
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.01,
            eval_strategy="steps",
            eval_steps=eval_steps,
            save_strategy="steps",
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=f"eval_{metric_name}",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            report_to="none",
            fp16=fp16 and torch.cuda.is_available(),
            dataloader_num_workers=min(4, os.cpu_count() or 4),
            seed=seed,
            remove_unused_columns=False,
            gradient_checkpointing=gradient_checkpointing,
            dataloader_pin_memory=True,
            ddp_find_unused_parameters=False,
            disable_tqdm=False
        )
    
        # 5) Trainer и старт обучения
        self.trainer = WeightedCETrainer(
            model=self.model,
            args=args,
            train_dataset=ds_train,
            eval_dataset=ds_eval,
            data_collator=collate,
            compute_metrics=compute_metrics,
            class_weights=class_weights
        )
    
        # (необязательно) убрать стандартный принтер-коллбек
        try:
            from transformers.trainer_callback import PrinterCallback
            self.trainer.remove_callback(PrinterCallback)
        except Exception:
            pass
    
        self.trainer.train()
    
        # очистить кэш токенизатора (если был)
        if has_bt:
            try: self.processors["text"].clear_cache()
            except Exception: pass
    
        return self

    # --------------------------
    # Предсказания
    # --------------------------
    def predict(
        self,
        df: pd.DataFrame,
        return_label_str: bool = False,
        return_proba: bool = False,
        batch_size: Optional[int] = None
    ) -> np.ndarray:
        if self.trainer is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        df_c = self._ensure_label_column(df)

        has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
        ds = MultiModalDataset(
            df=df_c,
            target_col=self.target_column_name,
            task="classification",
            label2id=self.label2id,
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if has_bt else None,
            pretokenize=(self.pretokenize_data and has_bt and len(df_c) < 10000 and self.text_padding != "dynamic"),
            pretokenize_batch_size=self.pretokenize_batch_size
        )
        collate = MultiModalCollator(self.processors, task="classification")

        # override bs
        if batch_size:
            original_bs = self.trainer.args.per_device_eval_batch_size
            self.trainer.args.per_device_eval_batch_size = batch_size

        effective_batch_size = batch_size or self.trainer.args.per_device_eval_batch_size
        num_batches = (len(df_c) + effective_batch_size - 1) // effective_batch_size
        print(f"Running predictions (batch_size={effective_batch_size}, num_batches={num_batches})...")

        original_disable_tqdm = self.trainer.args.disable_tqdm
        self.trainer.args.disable_tqdm = False

        preds = self.trainer.predict(test_dataset=ds, metric_key_prefix="predict")

        self.trainer.args.disable_tqdm = original_disable_tqdm
        if batch_size:
            self.trainer.args.per_device_eval_batch_size = original_bs

        ds.clear_cache()

        logits = preds.predictions
        if return_proba:
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
            return probabilities

        y_pred = np.argmax(logits, axis=-1)
        if return_label_str:
            return np.array([self.id2label[int(i)] for i in y_pred])
        return y_pred

    # --------------------------
    # Эмбеддинги
    # --------------------------
    def get_embeddings(
        self,
        df: pd.DataFrame,
        batch_size: int = 32,
        return_per_modality: bool = False
    ):
        if self.trainer is None or self.model is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        device = next(self.trainer.model.parameters()).device
        self.model.to(device).eval()

        df_c = self._ensure_label_column(df)

        has_bt = "text" in self.processors and hasattr(self.processors["text"], "bt")
        ds = MultiModalDataset(
            df=df_c,
            target_col=self.target_column_name,
            task="classification",
            label2id=self.label2id,
            text_columns=self.text_columns,
            image_columns=self.image_columns,
            audio_columns=self.audio_columns,
            text_tokenizer=self.processors["text"].bt if has_bt else None,
            pretokenize=False
        )
        collate = MultiModalCollator(self.processors, task="classification")
        loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)

        fused_list = []
        per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

        num_batches = (len(df_c) + batch_size - 1) // batch_size
        print(f"Extracting embeddings (batch_size={batch_size}, num_batches={num_batches})...")

        with torch.no_grad():
            for batch in tqdm(loader, total=num_batches, desc="Extracting embeddings", unit="batch", leave=True):
                bi = batch["backend_inputs"]

                def move_to_device(obj):
                    if torch.is_tensor(obj): return obj.to(device)
                    if isinstance(obj, dict): return {k: move_to_device(v) for k, v in obj.items()}
                    if isinstance(obj, list): return [move_to_device(v) for v in obj]
                    return obj

                bi = move_to_device(bi)
                fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
                fused_list.append(fused.cpu().numpy())
                if return_per_modality:
                    for m in per_mod_lists.keys():
                        if m in per:
                            per_mod_lists[m].append(per[m].cpu().numpy())

        fused_arr = np.vstack(fused_list)
        if not return_per_modality:
            print(f"✓ Embeddings shape: {fused_arr.shape}")
            return fused_arr

        per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
        print(f"✓ Fused embeddings shape: {fused_arr.shape}")
        for m, arr in per_mod.items():
            print(f"✓ {m.capitalize()} embeddings shape: {arr.shape}")

        return fused_arr, per_mod

In [ ]:
!pip install ultralytics

import os
import cv2
import json
import shutil
import tempfile
import warnings
import random
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from ultralytics import YOLO
from ultralytics.utils.downloads import attempt_download_asset

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold


class YOLODetectionPipeline:
    """
    YOLO-пайплайн из pandas DataFrame с опциональным подбором гиперпараметров подсчёта
    (plain или area-gated) и калибровкой линейной моделью (Ridge по резидуалу).

    Входные данные (DataFrame):
      - image_path (str): путь к изображению (jpg/png)
      - boxes_col: GT-боксы в YOLO-нормировке [0..1] (списки/массивы/строка)

    Основной сценарий:
      - fit(): обучает YOLO; (опц.) тюнинг порогов (plain/area-gated + iou/max_det);
               (опц.) калибрует Ridge; (опц.) валидирует RMSE/MAE.
      - predict(): возвращает детекции (boxes_json + count). Калибровка НЕ применяется.
      - predict_counts(): возвращает числовой подсчёт; применяет лучшие пороги/калибровку, если включены.

    Управление временем тюнинга:
      - tune_val_subsample: подвыборка валидации (int — кол-во, float — доля 0..1).
      - tune_max_combinations: ограничивает число проверяемых комбо (случайно из сетки).
    """

    def __init__(self,
                 model_ckpt: str = "yolov8n.pt",
                 data_root: str | None = None,
                 image_col: str = "image_path",
                 boxes_col: str = "boxes",
                 class_names: list[str] | None = None,
                 use_symlinks: bool = True,
                 verbose: bool = True,
                 # переключатели
                 enable_tuning: bool = True,
                 enable_ridge: bool = True,
                 validate_count: bool = True,
                 # режим тюнинга plain vs area-gated
                 enable_area_gate: bool = True,        # True → тюним (conf_small, conf_big, area_thr) + (iou, max_det)
                 enable_tta_flip: bool = False,        # True → TTA flip при plain-подсчёте
                 # управление временем тюнинга
                 tune_val_subsample: int | float | None = None,  # int=кол-во; float=доля [0..1]
                 tune_max_combinations: int | None = 100,
                 random_state: int = 42,
                 # сетки для plain-тюнинга
                 tune_conf_grid = (0.20, 0.25, 0.30, 0.35),
                 tune_iou_grid  = (0.55, 0.60),
                 tune_max_det_grid = (300, 600),
                 # сетки для area-gated (под мелкие объекты из ваших стат)
                 tune_conf_small_grid = (0.10, 0.12, 0.14, 0.18),
                 tune_conf_big_grid   = (0.30, 0.40, 0.50),
                 tune_area_thr_grid   = (0.0008, 0.0010, 0.0012, 0.0015),
                 gate_conf_base: float = 0.07,  # базовый conf для извлечения кандидатов при area-gated
                 # сетка для Ridge
                 ridge_alpha_grid = (0.3, 1.0, 3.0),
                 # пороги нормированной площади для фич Ridge/plain
                 small_thr: float = 0.0010,
                 big_thr: float   = 0.003):
        self.model_ckpt = model_ckpt
        self.image_col = image_col
        self.boxes_col = boxes_col
        self.class_names = class_names or ["obj"]
        self.use_symlinks = use_symlinks
        self.verbose = verbose

        self.enable_tuning = enable_tuning
        self.enable_ridge = enable_ridge
        self.validate_count = validate_count
        self.enable_area_gate = enable_area_gate
        self.enable_tta_flip = enable_tta_flip

        self.tune_val_subsample = tune_val_subsample
        self.tune_max_combinations = tune_max_combinations
        self.random_state = random_state
        random.seed(random_state)
        np.random.seed(random_state)

        # рабочая папка
        self._tmpdir_owned = False
        if data_root is None:
            self.data_root = tempfile.mkdtemp(prefix="yolo_ds_")
            self._tmpdir_owned = True
        else:
            self.data_root = os.path.abspath(data_root)
            os.makedirs(self.data_root, exist_ok=True)

        self.dataset_yaml = os.path.join(self.data_root, "dataset.yaml")
        self.model_path = None
        self._model = None
        self._device = None  # строка устройства, использованная при fit()

        # сетки и пороги
        self.tune_conf_grid = tuple(float(x) for x in tune_conf_grid)
        self.tune_iou_grid  = tuple(float(x) for x in tune_iou_grid)
        self.tune_max_det_grid = tuple(int(x) for x in tune_max_det_grid)

        self.tune_conf_small_grid = tuple(float(x) for x in tune_conf_small_grid)
        self.tune_conf_big_grid   = tuple(float(x) for x in tune_conf_big_grid)
        self.tune_area_thr_grid   = tuple(float(x) for x in tune_area_thr_grid)
        self.gate_conf_base = float(gate_conf_base)

        self.ridge_alpha_grid = tuple(float(x) for x in ridge_alpha_grid)
        self.small_thr = float(small_thr)
        self.big_thr   = float(big_thr)

        # сохранённые результаты тюнинга/калибровки
        self.calib_ = dict(
            # plain режим:
            best_conf=None, best_iou=None, best_max_det=None,
            # area-gated:
            gate_conf_small=None, gate_conf_big=None, gate_area_thr=None, gate_conf_base=None,
            # Ridge:
            ridge_alpha=None, ridge_model=None, ridge_mu=None, ridge_sd=None,
            # общий:
            imgsz=None
        )

    # -------------------- device helpers --------------------
    @staticmethod
    def _resolve_device(device: str | int | None) -> str:
        """
        Выбор устройства для обучения:
          - None / "auto": "0,1,...,N-1" при наличии CUDA, иначе "cpu"
          - иначе вернуть строку как есть (например, "0" или "cpu")
        """
        if device is None or str(device).lower() == "auto":
            if torch.cuda.is_available() and torch.cuda.device_count() > 0:
                n = torch.cuda.device_count()
                return ",".join(str(i) for i in range(n))
            return "cpu"
        return str(device)

    def _infer_device(self) -> str:
        """
        Устройство для инференса/тюнинга:
          - если тренировались на '0,1,...' → берём первую карту '0'
          - если тренировались на 'k' → её же
          - иначе авто: '0' при наличии CUDA, 'cpu' без GPU
        """
        if getattr(self, "_device", None):
            if isinstance(self._device, str) and "," in self._device:
                return self._device.split(",")[0]
            return self._device
        return "0" if (torch.cuda.is_available() and torch.cuda.device_count() > 0) else "cpu"

    # -------------------- helpers: разметка → YOLO-тxt --------------------
    @staticmethod
    def _is_nan_like(x):
        if x is None: return True
        if isinstance(x, float) and np.isnan(x): return True
        if isinstance(x, str) and x.strip()=="": return True
        return False

    def _parse_boxes(self, row):
        boxes_raw = row[self.boxes_col] if self.boxes_col in row else None
        if self._is_nan_like(boxes_raw): return []
        out = []
        if isinstance(boxes_raw, (list, tuple, np.ndarray)):
            for it in boxes_raw:
                vals = list(map(float, it))
                if len(vals) >= 4:
                    x,y,w,h = vals[:4]
                    if 0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1:
                        out.append((0, x,y,w,h))
        elif isinstance(boxes_raw, str):
            lines = [ln.strip() for ln in boxes_raw.strip().splitlines() if ln.strip()]
            for ln in lines:
                parts = ln.split()
                vals = list(map(float, parts))
                if len(vals) == 4:
                    x,y,w,h = vals
                    out.append((0, x,y,w,h))
                elif len(vals) >= 5:
                    cls,x,y,w,h = int(vals[0]), *vals[1:5]
                    out.append((cls, float(x),float(y),float(w),float(h)))
        return out

    def _link_or_copy(self, src, dst):
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        if self.use_symlinks:
            try:
                if os.path.lexists(dst): os.remove(dst)
                os.symlink(os.path.abspath(src), dst)
                return
            except Exception:
                pass
        shutil.copy2(src, dst)

    def _write_label_file(self, label_path, boxes):
        os.makedirs(os.path.dirname(label_path), exist_ok=True)
        with open(label_path, "w", encoding="utf-8") as f:
            for cls, x,y,w,h in boxes:
                f.write(f"{int(cls)} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

    def _materialize(self, train_df, val_df, train_split="train", val_split="val"):
        for split_name, df in [(train_split, train_df), (val_split, val_df)]:
            img_dir = os.path.join(self.data_root, "images", split_name)
            lbl_dir = os.path.join(self.data_root, "labels", split_name)
            os.makedirs(img_dir, exist_ok=True); os.makedirs(lbl_dir, exist_ok=True)
            it = df.iterrows()
            if self.verbose: it = tqdm(it, total=len(df), desc=f"[build] {split_name}")
            for _, row in it:
                src = row[self.image_col]
                if not os.path.exists(src):
                    raise FileNotFoundError(f"Image not found: {src}")
                fname = os.path.basename(src)
                stem, _ = os.path.splitext(fname)
                self._link_or_copy(src, os.path.join(img_dir, fname))
                self._write_label_file(os.path.join(lbl_dir, stem + ".txt"), self._parse_boxes(row))

        with open(self.dataset_yaml, "w", encoding="utf-8") as f:
            f.write(f"path: {self.data_root}\ntrain: images/{train_split}\nval: images/{val_split}\nnames:\n")
            for i, name in enumerate(self.class_names):
                f.write(f"  {i}: {name}\n")

    # -------------------- fit: train + (tune/ridge/validate) --------------------
    def fit(self,
            train_df: pd.DataFrame,
            val_df: pd.DataFrame | None = None,
            test_size: float = 0.2,
            epochs: int = 50,
            imgsz: int = 640,
            batch: int = 16,
            device: str | int | None = "auto",
            workers: int = 4,
            patience: int = 50,
            optimizer: str = "auto",
            augment: bool = True,
            seed: int = 42,
            close_mosaic: int | None = 10,
            cos_lr: bool = True,
            rect: bool = False,
            iou: float = 0.7,
            **extra_train_kwargs):

        # выбрать устройство для тренировки и сохранить
        self._device = self._resolve_device(device)
        if self.verbose:
            print(f"[device] training device='{self._device}'")

        np.random.seed(seed); random.seed(seed)

        # если val_df не задан — делаем простую стратификацию по бинам count
        if val_df is None:
            tmp = train_df.copy()
            counts = [len(self._parse_boxes(r)) for _, r in tmp.iterrows()]
            tmp["_bins"] = np.clip((np.array(counts)//5).astype(int), 0, 50)
            val_mask = tmp.groupby("_bins", group_keys=False).apply(
                lambda g: g.sample(frac=test_size, random_state=seed)).index
            val_df = train_df.loc[val_mask]
            train_df = train_df.drop(index=val_mask)
            train_df = train_df.reset_index(drop=True); val_df = val_df.reset_index(drop=True)

        self._materialize(train_df, val_df)

        # загрузить/скачать чекпоинт
        if not os.path.exists(self.model_ckpt) and self.model_ckpt.endswith(".pt"):
            try:
                if self.verbose: print(f"Checkpoint '{self.model_ckpt}' not found. Attempting to download...")
                attempt_download_asset(self.model_ckpt)
            except Exception as e:
                raise FileNotFoundError(f"Failed to download '{self.model_ckpt}'. Error: {e}")

        # train args
        train_args = {
            'data': self.dataset_yaml, 'epochs': epochs, 'imgsz': imgsz, 'batch': batch,
            'device': self._device, 'workers': workers, 'patience': patience, 'optimizer': optimizer,
            'augment': augment, 'seed': seed, 'close_mosaic': close_mosaic, 'cos_lr': cos_lr,
            'rect': rect, 'iou': iou, 'verbose': self.verbose
        }
        train_args.update(extra_train_kwargs)

        # обучение
        model = YOLO(self.model_ckpt)
        model.train(**train_args)

        # ГАРАНТИРОВАННО берём лучший чекпоинт
        best_path = None
        if hasattr(model, "trainer") and getattr(model.trainer, "best", None):
            best_path = str(model.trainer.best)     # .../runs/detect/exp/weights/best.pt
        elif getattr(model, "ckpt_path", None):
            best_path = str(model.ckpt_path)
        else:
            best_path = self.model_ckpt
        self.model_path = best_path
        if self.verbose:
            print(f"[fit] best model: {self.model_path}")

        # тюнинг/калибровка/валидация
        try:
            self._tune_and_or_calibrate(val_df, imgsz=imgsz)
            if self.validate_count:
                self._validate_counting(val_df)
        except Exception as e:
            if self.verbose:
                print(f"[post-fit] skipped tuning/calibration/validation due to: {e}")

        return self.model_path

    # -------------------- инференс-хелперы --------------------
    def _ensure_model(self):
        if self._model is None:
            path = self.model_path or self.model_ckpt
            self._model = YOLO(path)

    @torch.no_grad()
    def _raw_counts(self, paths, imgsz, conf, iou, max_det):
        """Plain len(detections). Если enable_tta_flip=True — усреднение с augment=True."""
        self._ensure_model()
        out = []
        dev = self._infer_device()
        if not self.enable_tta_flip:
            for i in tqdm(range(0, len(paths), 64), disable=not self.verbose, desc="[counts]"):
                batch = paths[i:i+64]
                res = self._model.predict(batch, imgsz=imgsz, conf=conf, iou=iou,
                                          max_det=max_det, device=dev, verbose=False)
                for r in res:
                    out.append(int(len(r.boxes) if (r.boxes is not None) else 0))
        else:
            for i in tqdm(range(0, len(paths), 64), disable=not self.verbose, desc="[counts-tta]"):
                batch = paths[i:i+64]
                r1 = self._model.predict(batch, imgsz=imgsz, conf=conf, iou=iou,
                                         max_det=max_det, device=dev, verbose=False)
                r2 = self._model.predict(batch, imgsz=imgsz, conf=conf, iou=iou,
                                         max_det=max_det, device=dev, verbose=False, augment=True)
                for a, b in zip(r1, r2):
                    n1 = int(len(a.boxes) if (a.boxes is not None) else 0)
                    n2 = int(len(b.boxes) if (b.boxes is not None) else 0)
                    out.append(0.5 * (n1 + n2))
        return np.array(out, dtype=float)

    @torch.no_grad()
    def _yolo_feats(self, paths, imgsz, conf, iou, max_det):
        """Фичи для Ridge: [n, conf_sum, conf_mean, conf_max, area_mean, frac_small, frac_mid, frac_big]."""
        self._ensure_model()
        rows = []
        dev = self._infer_device()
        for i in tqdm(range(0, len(paths), 64), disable=not self.verbose, desc="[feats]"):
            batch = paths[i:i+64]
            res = self._model.predict(batch, imgsz=imgsz, conf=conf, iou=iou,
                                      max_det=max_det, device=dev, verbose=False)
            for r in res:
                if r.boxes is None or len(r.boxes) == 0:
                    rows.append(dict(n=0, conf_sum=0, conf_mean=0, conf_max=0,
                                     area_mean=0, frac_small=0, frac_mid=0, frac_big=0))
                    continue
                confs = r.boxes.conf.cpu().numpy()
                xywhn = r.boxes.xywhn.cpu().numpy()
                areas = (xywhn[:, 2] * xywhn[:, 3]).clip(0, 1)
                rows.append(dict(
                    n=len(confs),
                    conf_sum=float(confs.sum()),
                    conf_mean=float(confs.mean()),
                    conf_max=float(confs.max()),
                    area_mean=float(areas.mean()),
                    frac_small=float((areas < self.small_thr).mean()),
                    frac_mid=float(((areas >= self.small_thr) & (areas <= self.big_thr)).mean()),
                    frac_big=float((areas > self.big_thr).mean())
                ))
        return pd.DataFrame(rows).to_numpy()

    @torch.no_grad()
    def _detect_conf_area(self, paths, imgsz, conf_base, iou, max_det):
        """Возвращает список массивов Nx2 [conf, area] при базовом пороге conf_base (для area-gated)."""
        self._ensure_model()
        dev = self._infer_device()
        out = []
        for i in tqdm(range(0, len(paths), 64), disable=not self.verbose, desc="[boxes]"):
            batch = paths[i:i+64]
            res = self._model.predict(batch, imgsz=imgsz, conf=conf_base, iou=iou,
                                      max_det=max_det, device=dev, verbose=False)
            for r in res:
                if r.boxes is None or len(r.boxes) == 0:
                    out.append(np.empty((0, 2), dtype=np.float32))
                    continue
                confs = r.boxes.conf.cpu().numpy()
                xywhn = r.boxes.xywhn.cpu().numpy()
                areas = (xywhn[:, 2] * xywhn[:, 3]).clip(0, 1)
                out.append(np.stack([confs, areas], axis=1))
        return out

    @staticmethod
    def _count_with_area_gate(conf_area_list, conf_small, conf_big, area_thr):
        """Подсчёт с двупороговой фильтрацией по площади."""
        counts = []
        for ca in conf_area_list:
            if ca.size == 0:
                counts.append(0); continue
            conf = ca[:, 0]; area = ca[:, 1]
            small_mask = (area < area_thr)  & (conf >= conf_small)
            big_mask   = (area >= area_thr) & (conf >= conf_big)
            counts.append(int(small_mask.sum() + big_mask.sum()))
        return np.array(counts, dtype=float)

    # -------------------- подвыборка валидации --------------------
    def _subset_val(self, val_df: pd.DataFrame) -> pd.DataFrame:
        if self.tune_val_subsample is None:
            return val_df
        n = len(val_df)
        if isinstance(self.tune_val_subsample, float):
            k = max(1, int(round(n * self.tune_val_subsample)))
        else:
            k = int(self.tune_val_subsample)
        k = min(k, n)
        return val_df.sample(n=k, random_state=self.random_state).reset_index(drop=True)

    # -------------------- Ridge по резидуалу с K-fold CV --------------------
    def _fit_ridge_cv_on_residual(self, X: np.ndarray, y_true: np.ndarray, y_plain: np.ndarray):
        alphas = self.ridge_alpha_grid
        k = min(5, len(y_true)) if len(y_true) >= 3 else 2
        kf = KFold(n_splits=k, shuffle=True, random_state=self.random_state)

        # стандартизация фич
        mu = X.mean(axis=0)
        sd = X.std(axis=0); sd[sd == 0] = 1.0
        Xs = (X - mu) / sd
        r = y_true - y_plain

        best_alpha, best_cv = None, 1e9
        for a in alphas:
            cv_scores = []
            for tr, va in kf.split(Xs):
                m = Ridge(alpha=float(a)).fit(Xs[tr], r[tr])
                pr = m.predict(Xs[va])
                cv_scores.append(mean_squared_error(r[va], pr, squared=False))
            cv_rmse = float(np.mean(cv_scores))
            if cv_rmse < best_cv:
                best_alpha, best_cv = float(a), cv_rmse

        # финальная подгонка на всех
        model = Ridge(alpha=best_alpha).fit(Xs, r)
        return dict(model=model, alpha=best_alpha, mu=mu, sd=sd, cv_rmse=best_cv)

    # -------------------- тюнинг и/или калибровка --------------------
    def _tune_and_or_calibrate(self, val_df: pd.DataFrame, imgsz: int):
        val_sub = self._subset_val(val_df)
        paths = val_sub[self.image_col].tolist()
        y_true = np.array([len(self._parse_boxes(r)) for _, r in val_sub.iterrows()], dtype=float)

        # 1) Тюнинг plain или area-gated
        if self.enable_tuning:
            if self.enable_area_gate:
                # Подготовим базовый порог для сбора кандидатов, согласованный с минимальным cs
                min_cs = min(self.tune_conf_small_grid) if len(self.tune_conf_small_grid) else 0.10
                base_collect = max(0.03, min(self.gate_conf_base, min_cs - 0.02))
                if self.verbose:
                    print(f"[tune-gate] base_collect={base_collect:.3f} (min_cs={min_cs:.3f})")

                # 1) Предрасчёт списков [conf, area] для всех (iou, max_det)
                iou_grid = tuple(self.tune_iou_grid)
                md_grid  = tuple(self.tune_max_det_grid)
                conf_area_by_key = {}
                total_prepasses = len(iou_grid) * len(md_grid)
                if self.verbose:
                    print(f"[tune-gate] precomputing boxes for {total_prepasses} (iou,max_det) pairs...")
                for iou_ in iou_grid:
                    for md_ in md_grid:
                        conf_area_by_key[(iou_, md_)] = self._detect_conf_area(
                            paths, imgsz=imgsz,
                            conf_base=float(base_collect),
                            iou=float(iou_), max_det=int(md_)
                        )

                # 2) Подбор cs/cb/area_thr + iou/max_det
                all_combos = []
                for iou_ in iou_grid:
                    for md_ in md_grid:
                        for cs in self.tune_conf_small_grid:
                            # эффективная база (ниже cs)
                            base_eff = max(0.03, min(float(self.gate_conf_base), float(cs) - 0.02))
                            for cb in self.tune_conf_big_grid:
                                for at in self.tune_area_thr_grid:
                                    all_combos.append((float(iou_), int(md_), float(cs), float(cb), float(at), float(base_eff)))

                random.shuffle(all_combos)
                full_space = len(all_combos)
                if self.tune_max_combinations is not None:
                    all_combos = all_combos[:int(self.tune_max_combinations)]
                if self.verbose:
                    print(f"[tune-gate] search combos: {len(all_combos)} (cap), full={full_space}")

                best = dict(rmse=1e9, iou=None, max_det=None, cs=None, cb=None, at=None, base=None)
                for (iou_, md_, cs, cb, at, base_eff) in all_combos:
                    conf_area = conf_area_by_key[(iou_, md_)]
                    y_pred = self._count_with_area_gate(conf_area, cs, cb, at)
                    rmse = mean_squared_error(y_true, y_pred, squared=False)
                    if rmse < best["rmse"]:
                        best.update(dict(rmse=rmse, iou=iou_, max_det=md_,
                                         cs=cs, cb=cb, at=at, base=base_eff))

                if self.verbose:
                    print(f"[tune-gate] best: cs={best['cs']:.3f}, cb={best['cb']:.3f}, "
                          f"area_thr={best['at']:.4f}, iou={best['iou']}, max_det={best['max_det']}  RMSE={best['rmse']:.3f}")

                self.calib_.update(dict(
                    best_conf=None, best_iou=float(best['iou']), best_max_det=int(best['max_det']),
                    gate_conf_small=float(best['cs']), gate_conf_big=float(best['cb']),
                    gate_area_thr=float(best['at']), gate_conf_base=float(best['base'])
                ))
            else:
                combos = [(float(c), float(i), int(m))
                          for i in self.tune_iou_grid
                          for m in self.tune_max_det_grid
                          for c in self.tune_conf_grid]
                random.shuffle(combos)
                full_space = len(combos)
                if self.tune_max_combinations is not None:
                    combos = combos[:int(self.tune_max_combinations)]
                if self.verbose:
                    print(f"[tune-plain] search combos: {len(combos)} (cap), full={full_space}")

                best = dict(rmse=1e9, conf=None, iou=None, max_det=None)
                for conf, iou_v, max_det in combos:
                    y_pred = self._raw_counts(paths, imgsz=imgsz, conf=conf, iou=iou_v, max_det=max_det)
                    rmse = mean_squared_error(y_true, y_pred, squared=False)
                    if rmse < best["rmse"]:
                        best.update(dict(rmse=rmse, conf=conf, iou=iou_v, max_det=max_det))
                if self.verbose:
                    print(f"[tune] best plain count: conf={best['conf']}, iou={best['iou']}, max_det={best['max_det']}  RMSE={best['rmse']:.3f}")

                self.calib_.update(dict(
                    best_conf=best['conf'], best_iou=best['iou'], best_max_det=best['max_det'],
                    gate_conf_small=None, gate_conf_big=None, gate_area_thr=None, gate_conf_base=None
                ))
        else:
            # без тюнинга — дефолты для plain
            self.calib_.update(dict(
                best_conf=0.25, best_iou=0.5, best_max_det=1000,
                gate_conf_small=None, gate_conf_big=None, gate_area_thr=None, gate_conf_base=None
            ))

        # 2) Калибровка Ridge (по резидуалу, с CV)
        ridge_model, ridge_alpha = None, None
        if self.enable_ridge:
            # строим y_plain на том же сабсете val_sub
            if self.enable_tuning and self.enable_area_gate and self.calib_.get("gate_conf_small") is not None:
                iou_use = float(self.calib_["best_iou"])
                md_use  = int(self.calib_["best_max_det"])
                conf_area = self._detect_conf_area(paths, imgsz=imgsz,
                                                   conf_base=float(self.calib_["gate_conf_base"]),
                                                   iou=iou_use, max_det=md_use)
                y_plain = self._count_with_area_gate(conf_area,
                                                     float(self.calib_["gate_conf_small"]),
                                                     float(self.calib_["gate_conf_big"]),
                                                     float(self.calib_["gate_area_thr"]))
                X = self._yolo_feats(paths, imgsz=imgsz,
                                     conf=float(self.calib_["gate_conf_base"]),
                                     iou=iou_use, max_det=md_use)
            else:
                conf_use = float(self.calib_["best_conf"] if self.calib_.get("best_conf") is not None else 0.25)
                iou_use  = float(self.calib_["best_iou"]  if self.calib_.get("best_iou")  is not None else 0.5)
                md_use   = int(self.calib_["best_max_det"] if self.calib_.get("best_max_det") is not None else 1000)
                y_plain  = self._raw_counts(paths, imgsz=imgsz, conf=conf_use, iou=iou_use, max_det=md_use)
                X = self._yolo_feats(paths, imgsz=imgsz, conf=conf_use, iou=iou_use, max_det=md_use)

            pack = self._fit_ridge_cv_on_residual(X, y_true, y_plain)
            ridge_model, ridge_alpha = pack["model"], pack["alpha"]
            if self.verbose:
                print(f"[calib] Ridge(residual) alpha={ridge_alpha}  CV-RMSE(resid)={pack['cv_rmse']:.3f}")

            # сохраним стандартизацию
            self.calib_.update(dict(
                ridge_alpha=ridge_alpha, ridge_model=ridge_model,
                ridge_mu=pack["mu"], ridge_sd=pack["sd"], imgsz=imgsz
            ))
        else:
            self.calib_.update(dict(ridge_alpha=None, ridge_model=None, imgsz=imgsz))

    # -------------------- финальная валидация подсчёта --------------------
    def _validate_counting(self, val_df: pd.DataFrame):
        paths = val_df[self.image_col].tolist()
        y_true = np.array([len(self._parse_boxes(r)) for _, r in val_df.iterrows()], dtype=float)

        imgsz = self.calib_['imgsz'] or 640

        # plain/gate
        if self.enable_tuning and self.enable_area_gate and self.calib_.get("gate_conf_small") is not None:
            iou_use = float(self.calib_["best_iou"])
            md_use  = int(self.calib_["best_max_det"])
            conf_area = self._detect_conf_area(paths, imgsz=imgsz, conf_base=self.calib_["gate_conf_base"],
                                               iou=iou_use, max_det=md_use)
            y_plain = self._count_with_area_gate(conf_area,
                                                 self.calib_["gate_conf_small"],
                                                 self.calib_["gate_conf_big"],
                                                 self.calib_["gate_area_thr"])
        else:
            conf  = self.calib_['best_conf'] if self.enable_tuning else 0.25
            iou_v   = self.calib_['best_iou']  if self.enable_tuning else 0.5
            max_det = self.calib_['best_max_det'] if self.enable_tuning else 1000
            y_plain = self._raw_counts(paths, imgsz, conf, iou_v, max_det)

        rmse_plain = mean_squared_error(y_true, y_plain, squared=False)
        mae_plain  = mean_absolute_error(y_true, y_plain)

        # calibrated
        if self.enable_ridge and self.calib_['ridge_model'] is not None:
            if self.enable_tuning and self.enable_area_gate and self.calib_.get("gate_conf_small") is not None:
                X = self._yolo_feats(paths, imgsz=imgsz, conf=float(self.calib_["gate_conf_base"]),
                                     iou=float(self.calib_["best_iou"]), max_det=int(self.calib_["best_max_det"]))
            else:
                X = self._yolo_feats(paths, imgsz=imgsz,
                                     conf=(self.calib_["best_conf"] if self.calib_["best_conf"] is not None else 0.25),
                                     iou=(self.calib_["best_iou"] if self.calib_["best_iou"] is not None else 0.5),
                                     max_det=(self.calib_["best_max_det"] if self.calib_["best_max_det"] is not None else 1000))
            mu = self.calib_.get("ridge_mu"); sd = self.calib_.get("ridge_sd")
            Xs = (X - mu) / sd
            resid = self.calib_['ridge_model'].predict(Xs)
            y_cal = np.clip(y_plain + resid, 0, None)
            rmse_cal = mean_squared_error(y_true, y_cal, squared=False)
            mae_cal  = mean_absolute_error(y_true, y_cal)
            print(f"[val-count] plain: RMSE={rmse_plain:.3f}, MAE={mae_plain:.3f}  |  calibrated: RMSE={rmse_cal:.3f}, MAE={mae_cal:.3f}")
        else:
            print(f"[val-count] plain: RMSE={rmse_plain:.3f}, MAE={mae_plain:.3f}  |  calibrated: (disabled)")

    # -------------------- публичный инференс: детекции --------------------
    @torch.no_grad()
    def predict(self, df: pd.DataFrame,
                conf: float = 0.25, iou: float = 0.6,
                imgsz: int = 640, device: str | int | None = "auto",
                max_det: int = 300, agnostic_nms: bool = False) -> pd.DataFrame:
        """Детекции (калибровка НЕ используется)."""
        assert self.image_col in df.columns
        if self._model is None:
            self._model = YOLO(self.model_path or self.model_ckpt)

        dev = self._resolve_device(device if device is not None else self._infer_device())

        paths = df[self.image_col].tolist()
        preds = []
        for i in tqdm(range(0, len(paths), 64), disable=not self.verbose, desc="[predict]"):
            batch = paths[i:i+64]
            res = self._model(batch, conf=conf, iou=iou, imgsz=imgsz,
                              device=dev, verbose=False, max_det=max_det,
                              agnostic_nms=agnostic_nms)
            for r in res:
                boxes = []
                if r.boxes is not None and len(r.boxes):
                    xywhn = r.boxes.xywhn.cpu().numpy()
                    confv = r.boxes.conf.cpu().numpy()
                    clsv  = r.boxes.cls.cpu().numpy().astype(int)
                    for (x,y,w,h), c, k in zip(xywhn, confv, clsv):
                        boxes.append({"cls": int(k), "conf": float(c),
                                      "x": float(x), "y": float(y), "w": float(w), "h": float(h)})
                preds.append({
                    self.image_col: r.path,
                    "count": len(boxes),
                    "boxes_json": json.dumps(boxes, ensure_ascii=False)
                })
        return pd.DataFrame(preds)

    # -------------------- публичный инференс: подсчёт --------------------
    @torch.no_grad()
    def predict_counts(self, df: pd.DataFrame,
                       imgsz: int | None = None,
                       conf: float | None = None,
                       iou: float | None = None,
                       max_det: int | None = None,
                       device: str | int | None = "auto",
                       clamp_nonneg: bool = True,
                       do_round: bool = False) -> pd.DataFrame:
        """
        Подсчёт объектов.
          - Если enable_ridge=True и калибратор обучен → y = y_plain + Ridge(residual).
          - Иначе → plain len(dets).
          - Если enable_area_gate=True и тюнинг выполнен → двупороговая фильтрация (conf_small/conf_big) по area.
        """
        assert self.image_col in df.columns
        if self._model is None:
            self._model = YOLO(self.model_path or self.model_ckpt)

        dev = self._resolve_device(device if device is not None else self._infer_device())
        imgsz = imgsz or self.calib_['imgsz'] or 640
        paths = df[self.image_col].tolist()

        # area-gated путь
        if self.enable_tuning and self.enable_area_gate and self.calib_.get("gate_conf_small") is not None:
            conf_base = float(self.calib_["gate_conf_base"])
            iou_use   = float(self.calib_["best_iou"])
            max_det_use = int(self.calib_["best_max_det"])
            conf_area = self._detect_conf_area(paths, imgsz=imgsz, conf_base=conf_base, iou=iou_use, max_det=max_det_use)
            cs, cb, at = float(self.calib_["gate_conf_small"]), float(self.calib_["gate_conf_big"]), float(self.calib_["gate_area_thr"])
            y_plain = self._count_with_area_gate(conf_area, cs, cb, at)

            if self.enable_ridge and self.calib_.get("ridge_model") is not None:
                X = self._yolo_feats(paths, imgsz=imgsz, conf=conf_base, iou=iou_use, max_det=max_det_use)
                # стандартизация и предсказание резидуала
                mu = self.calib_.get("ridge_mu"); sd = self.calib_.get("ridge_sd")
                Xs = (X - mu) / sd
                resid = self.calib_['ridge_model'].predict(Xs)
                y = y_plain + resid
            else:
                y = y_plain

        else:
            # обычный plain путь
            if self.enable_tuning and self.calib_.get("best_conf") is not None:
                conf_def, iou_def, max_det_def = self.calib_['best_conf'], self.calib_['best_iou'], self.calib_['best_max_det']
            else:
                conf_def, iou_def, max_det_def = 0.25, 0.5, 1000

            use_conf  = conf    if conf    is not None else conf_def
            use_iou   = iou     if iou     is not None else iou_def
            use_maxdet= max_det if max_det is not None else max_det_def

            if self.enable_ridge and self.calib_.get("ridge_model") is not None:
                X = self._yolo_feats(paths, imgsz=imgsz, conf=use_conf, iou=use_iou, max_det=use_maxdet)
                mu = self.calib_.get("ridge_mu"); sd = self.calib_.get("ridge_sd")
                Xs = (X - mu) / sd
                resid = self.calib_['ridge_model'].predict(Xs)
                # базовый plain-счёт для сложения с резидуалом:
                y_plain = self._raw_counts(paths, imgsz=imgsz, conf=use_conf, iou=use_iou, max_det=use_maxdet)
                y = y_plain + resid
            else:
                y = self._raw_counts(paths, imgsz=imgsz, conf=use_conf, iou=use_iou, max_det=use_maxdet)

        if clamp_nonneg: y = np.clip(y, 0, None)
        if do_round:     y = np.rint(y)

        out = df[[self.image_col]].copy()
        out["label"] = y
        return out

    # -------------------- housekeeping --------------------
    def cleanup(self):
        if self._tmpdir_owned and os.path.isdir(self.data_root):
            shutil.rmtree(self.data_root, ignore_errors=True)

# pipe = YOLODetectionPipeline(
#     model_ckpt="yolov8l.pt",
#     image_col="image_path",
#     boxes_col="label",
#     class_names=["obj"],
#     verbose=True,
#     use_symlinks=True,

#     enable_tuning=True,
#     enable_ridge=True,
#     validate_count=True,

#     enable_area_gate=True,
#     enable_tta_flip=False,

#     tune_val_subsample=None,
#     tune_max_combinations=500,

#     tune_conf_grid=(0.18, 0.21, 0.24, 0.27, 0.30, 0.33, 0.36, 0.39, 0.42),
#     tune_iou_grid=(0.4, 0.45, 0.50, 0.55, 0.60, 0.65),
#     tune_max_det_grid=(50, 100, 200, 300, 600),
#     tune_conf_small_grid=(0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20),
#     tune_conf_big_grid=(0.28, 0.32, 0.36, 0.40, 0.45, 0.50, 0.55),
#     tune_area_thr_grid=(0.0006, 0.0008, 0.0010, 0.0012, 0.0015, 0.0018),
#     ridge_alpha_grid=(0.01, 0.03, 0.05, 0.075, 0.1, 0.3, 0.6, 1.0, 2.0),

#     gate_conf_base=0.07,
#     small_thr=0.0010,
#     big_thr=0.003,

#     random_state=42
# )

# pipe.fit(
#     train_df=train_df,
#     val_df=val_df,
#     epochs=60,
#     imgsz=640,
#     batch=32,
#     device='0,1',
#     workers=4,
#     lr0=0.01,
#     cos_lr=True,
#     rect=True,
#     iou=0.5
# )

# pipeline = YOLODetectionPipeline(
#     model_ckpt="yolov10n.pt",
#     image_col="image_path",
#     boxes_col="label",
#     enable_tuning=True,
#     enable_ridge=True,
#     validate_count=True,
#     tune_val_subsample=None,  # вся валидация для подбора параметров для подсчёта объектов
#     tune_max_combinations=100,  ######################### 50
#     random_state=42,
#     verbose=True
# )
# pipeline.fit(
#     train_df=train_df,
#     val_df=val_df,
#     lr0=0.01,
#     epochs=70,  ######################### 40
#     imgsz=640,
#     batch=32,
#     iou=0.5,
#     rect=True,
#     device='0,1'
# )

In [ ]:
encoder_params = []
head_params = []

for name, param in model.named_parameters():
    if 'classifier' in name or 'head' in name or 'projection' in name:
        head_params.append(param)
    else:
        encoder_params.append(param)

optimizer_grouped_parameters = [
    {'params': encoder_params, 'lr': 3e-5},
    {'params': head_params, 'lr': 6e-5}
]

optimizer = torch.optim.AdamW(optimizer_grouped_parameters)

trainer = Trainer(
    ...,
    optimizers=(optimizer, None)
)